In [1]:
!nvidia-smi

Mon Apr  1 05:22:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:01:00.0 Off |                  Off |
| 30%   34C    P8              20W / 300W |     12MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
!pip install transformers --upgrade
!pip install accelerate
!pip install -U flash-attn --no-build-isolation


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import time
from accelerate import Accelerator

## model loading

In [6]:
start = time.time()
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1", device_map="auto", revision="refs/pr/5")
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1",torch_dtype=torch.float32, device_map="auto", revision="refs/pr/5")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1")


# record end time
end = time.time()

temp = end-start
print(temp)
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes

total_time = "%dh:%dm:%ds" %(hours,minutes,seconds)

if hours == 0:
    print('The time of execution of downloading and loading the model : %dm:%ds' %(minutes,seconds))

else:
    print('The time of execution of downloading and loading the model : %dh:%dm:%ds' %(hours,minutes,seconds))


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

595.948174238205
The time of execution of downloading and loading the model : 9m:55s


In [7]:
!nvidia-smi

Mon Apr  1 05:33:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:01:00.0 Off |                  Off |
| 30%   36C    P2              71W / 300W |  34187MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## MG: text

In [47]:
item = '''
Mohandas Karamchand Gandhi (2 October 1869 – 30 January 1948) was an Indian lawyer, anti-colonial nationalist and political ethicist who employed nonviolent resistance to lead the successful campaign for India's independence from British rule. He inspired movements for civil rights and freedom across the world. The honorific Mahātmā (from Sanskrit 'great-souled, venerable'), first applied to him in South Africa in 1914, is now used throughout the world.
Born and raised in a Hindu family in coastal Gujarat, Gandhi trained in the law at the Inner Temple, London, and was called to the bar at age 22 in June 1891. After two uncertain years in India, where he was unable to start a successful law practice, he moved to South Africa in 1893 to represent an Indian merchant in a lawsuit. He went on to live in South Africa for 21 years. It was here that Gandhi raised a family and first employed nonviolent resistance in a campaign for civil rights. In 1915, aged 45, he returned to India and soon set about organising peasants, farmers, and urban labourers to protest against excessive land-tax and discrimination.
Assuming leadership of the Indian National Congress in 1921, Gandhi led nationwide campaigns for easing poverty, expanding women's rights, building religious and ethnic amity, ending untouchability, and, above all, achieving swaraj or self-rule. Gandhi adopted the short dhoti woven with hand-spun yarn as a mark of identification with India's rural poor. He began to live in a self-sufficient residential community, to eat simple food, and undertake long fasts as a means of both introspection and political protest. Bringing anti-colonial nationalism to the common Indians, Gandhi led them in challenging the British-imposed salt tax with the 400 km (250 mi) Dandi Salt March in 1930 and in calling for the British to quit India in 1942. He was imprisoned many times and for many years in both South Africa and India.
Gandhi's vision of an independent India based on religious pluralism was challenged in the early 1940s by a Muslim nationalism which demanded a separate homeland for Muslims within British India. In August 1947, Britain granted independence, but the British Indian Empire was partitioned into two dominions, a Hindu-majority India and a Muslim-majority Pakistan. As many displaced Hindus, Muslims, and Sikhs made their way to their new lands, religious violence broke out, especially in the Punjab and Bengal. Abstaining from the official celebration of independence, Gandhi visited the affected areas, attempting to alleviate distress. In the months following, he undertook several hunger strikes to stop the religious violence. The last of these was begun in Delhi on 12 January 1948 when he was 78. The belief that Gandhi had been too resolute in his defense of both Pakistan and Indian Muslims spread among some Hindus in India. Among these was Nathuram Godse, a militant Hindu nationalist from Pune, western India, who assassinated Gandhi by firing three bullets into his chest at an interfaith prayer meeting in Delhi on 30 January 1948.
Gandhi's birthday, 2 October, is commemorated in India as Gandhi Jayanti, a national holiday, and worldwide as the International Day of Nonviolence. Gandhi is considered to be the Father of the Nation in post-colonial India. During India's nationalist movement and in several decades immediately after, he was also commonly called Bapu (Gujarati endearment for "father", roughly "papa", "daddy").


== Early life and background ==


=== Parents ===
Gandhi's father, Karamchand Uttamchand Gandhi (1822–1885), served as the dewan (chief minister) of Porbandar state. His family originated from the then village of Kutiana in what was then Junagadh State. Although he only had been a clerk in the state administration and had an elementary education, Karamchand proved a capable chief minister.During his tenure, Karamchand married four times. His first two wives died young, after each had given birth to a daughter, and his third marriage was childless. In 1857, he sought his third wife's permission to remarry; that year, he married Putlibai (1844–1891), who also came from Junagadh, and was from a Pranami Vaishnava family. Karamchand and Putlibai had four children: a son, Laxmidas (c. 1860–1914); a daughter, Raliatbehn (1862–1960); a second son, Karsandas (c. 1866–1913). and a third son, Mohandas Karamchand Gandhi who was born on 2 October 1869 in Porbandar (also known as Sudamapuri), a coastal town on the Kathiawar Peninsula and then part of the small princely state of Porbandar in the Kathiawar Agency of the British Raj.
In 1874, Gandhi's father Karamchand left Porbandar for the smaller state of Rajkot, where he became a counsellor to its ruler, the Thakur Sahib; though Rajkot was a less prestigious state than Porbandar, the British regional political agency was located there, which gave the state's diwan a measure of security. In 1876, Karamchand became diwan of Rajkot and was succeeded as diwan of Porbandar by his brother Tulsidas. His family then rejoined him in Rajkot.


=== Childhood ===
As a child, Gandhi was described by his sister Raliat as "restless as mercury, either playing or roaming about. One of his favourite pastimes was twisting dogs' ears." The Indian classics, especially the stories of Shravana and king Harishchandra, had a great impact on Gandhi in his childhood. In his autobiography, he states that they left an indelible impression on his mind. He writes: "It haunted me and I must have acted Harishchandra to myself times without number." Gandhi's early self-identification with truth and love as supreme values is traceable to these epic characters.The family's religious background was eclectic. Mohandas was born into a Gujarati Hindu Modh Bania family. Gandhi's father Karamchand was Hindu and his mother Putlibai was from a Pranami Vaishnava Hindu family. Gandhi's father was of Modh Baniya caste in the varna of Vaishya. His mother came from the medieval Krishna bhakti-based Pranami tradition, whose religious texts include the Bhagavad Gita, the Bhagavata Purana, and a collection of 14 texts with teachings that the tradition believes to include the essence of the Vedas, the Quran and the Bible. Gandhi was deeply influenced by his mother, an extremely pious lady who "would not think of taking her meals without her daily prayers... she would take the hardest vows and keep them without flinching. To keep two or three consecutive fasts was nothing to her."
At age 9, Gandhi entered the local school in Rajkot, near his home. There, he studied the rudiments of arithmetic, history, the Gujarati language and geography. At the age of 11, he joined the High School in Rajkot, Alfred High School. He was an average student, won some prizes, but was a shy and tongue tied student, with no interest in games; his only companions were books and school lessons.


=== Marriage ===
In May 1883, the 13-year-old Mohandas was married to 14-year-old Kasturbai Gokuldas Kapadia (her first name was usually shortened to "Kasturba", and affectionately to "Ba") in an arranged marriage, according to the custom of the region at that time. In the process, he lost a year at school but was later allowed to make up by accelerating his studies. His wedding was a joint event, where his brother and cousin were also married. Recalling the day of their marriage, he once said, "As we didn't know much about marriage, for us it meant only wearing new clothes, eating sweets and playing with relatives." As was the prevailing tradition, the adolescent bride was to spend much time at her parents' house, and away from her husband.Writing many years later, Mohandas described with regret the lustful feelings he felt for his young bride: "even at school I used to think of her, and the thought of nightfall and our subsequent meeting was ever haunting me." He later recalled feeling jealous and possessive of her, such as when she would visit a temple with her girlfriends, and being sexually lustful in his feelings for her.In late 1885, Gandhi's father Karamchand died. Gandhi, then 16 years old, and his wife of age 17 had their first baby, who survived only a few days. The two deaths anguished Gandhi. The Gandhi couple had four more children, all sons: Harilal, born in 1888; Manilal, born in 1892; Ramdas, born in 1897; and Devdas, born in 1900.In November 1887, the 18-year-old Gandhi graduated from high school in Ahmedabad. In January 1888, he enrolled at Samaldas College in Bhavnagar State, then the sole degree-granting institution of higher education in the region. However, he dropped out, and returned to his family in Porbandar.


== Three years in London ==


=== Student of law ===
Gandhi had dropped out of the cheapest college he could afford in Bombay. Mavji Dave Joshiji, a Brahmin priest and family friend, advised Gandhi and his family that he should consider law studies in London. In July 1888, his wife Kasturba gave birth to their first surviving son, Harilal. His mother was not comfortable about Gandhi leaving his wife and family, and going so far from home. Gandhi's uncle Tulsidas also tried to dissuade his nephew. Gandhi wanted to go. To persuade his wife and mother, Gandhi made a vow in front of his mother that he would abstain from meat, alcohol and women. Gandhi's brother Laxmidas, who was already a lawyer, cheered Gandhi's London studies plan and offered to support him. Putlibai gave Gandhi her permission and blessing.
On 10 August 1888, Gandhi aged 18, left Porbandar for Mumbai, then known as Bombay. Upon arrival, he stayed with the local Modh Bania community whose elders warned him that England would tempt him to compromise his religion, and eat and drink in Western ways. Despite Gandhi informing them of his promise to his mother and her blessings, he was excommunicated from his caste. Gandhi ignored this, and on 4 September, he sailed from Bombay to London, with his brother seeing him off. Gandhi attended University College, London, where he took classes in English literature with Henry Morley in 1888–1889.He also enrolled at the Inns of Court School of Law in Inner Temple with the intention of becoming a barrister. His childhood shyness and self-withdrawal had continued through his teens. He retained these traits when he arrived in London, but joined a public speaking practice group and overcame his shyness sufficiently to practise law.He demonstrated a keen interest in the welfare of London's impoverished dockland communities. In 1889, a bitter trade dispute broke out in London, with dockers striking for better pay and conditions, and seamen, shipbuilders, factory girls and other joining the strike in solidarity. The strikers were successful, in part due to the mediation of Cardinal Manning, leading Gandhi and an Indian friend to make a point of visiting the cardinal and thanking him for his work.


=== Vegetarianism and committee work ===
Gandhi's time in London was influenced by the vow he had made to his mother. He tried to adopt "English" customs, including taking dancing lessons. However, he did not appreciate the bland vegetarian food offered by his landlady and was frequently hungry until he found one of London's few vegetarian restaurants. Influenced by Henry Salt's writing, he joined the London Vegetarian Society, and was elected to its executive committee under the aegis of its president and benefactor Arnold Hills. An achievement while on the committee was the establishment of a Bayswater chapter. Some of the vegetarians he met were members of the Theosophical Society, which had been founded in 1875 to further universal brotherhood, and which was devoted to the study of Buddhist and Hindu literature. They encouraged Gandhi to join them in reading the Bhagavad Gita both in translation as well as in the original.Gandhi had a friendly and productive relationship with Hills, but the two men took a different view on the continued LVS membership of fellow committee member Thomas Allinson. Their disagreement is the first known example of Gandhi challenging authority, despite his shyness and temperamental disinclination towards confrontation.
Allinson had been promoting newly available birth control methods, but Hills disapproved of these, believing they undermined public morality. He believed vegetarianism to be a moral movement and that Allinson should therefore no longer remain a member of the LVS. Gandhi shared Hills' views on the dangers of birth control, but defended Allinson's right to differ. It would have been hard for Gandhi to challenge Hills; Hills was 12 years his senior and unlike Gandhi, highly eloquent. He bankrolled the LVS and was a captain of industry with his Thames Ironworks company employing more than 6,000 people in the East End of London. He was also a highly accomplished sportsman who later founded the football club West Ham United. In his 1927 An Autobiography, Vol. I, Gandhi wrote:

 The question deeply interested me...I had a high regard for Mr. Hills and his generosity. But I thought it was quite improper to exclude a man from a vegetarian society simply because he refused to regard puritan morals as one of the objects of the society
A motion to remove Allinson was raised, and was debated and voted on by the committee. Gandhi's shyness was an obstacle to his defence of Allinson at the committee meeting. He wrote his views down on paper but shyness prevented him from reading out his arguments, so Hills, the President, asked another committee member to read them out for him. Although some other members of the committee agreed with Gandhi, the vote was lost and Allinson excluded. There were no hard feelings, with Hills proposing the toast at the LVS farewell dinner in honour of Gandhi's return to India.


=== Called to the bar ===
Gandhi, at age 22, was called to the bar in June 1891 and then left London for India, where he learned that his mother had died while he was in London and that his family had kept the news from him. His attempts at establishing a law practice in Bombay failed because he was psychologically unable to cross-examine witnesses. He returned to Rajkot to make a modest living drafting petitions for litigants, but he was forced to stop when he ran afoul of British officer Sam Sunny.In 1893, a Muslim merchant in Kathiawar named Dada Abdullah contacted Gandhi. Abdullah owned a large successful shipping business in South Africa. His distant cousin in Johannesburg needed a lawyer, and they preferred someone with Kathiawari heritage. Gandhi inquired about his pay for the work. They offered a total salary of £105 (~$4,143.31 2023 money) plus travel expenses. He accepted it, knowing that it would be at least a one-year commitment in the Colony of Natal, South Africa, also a part of the British Empire.


== Civil rights activist in South Africa (1893–1914) ==
In April 1893, Gandhi aged 23, set sail for South Africa to be the lawyer for Abdullah's cousin. He spent 21 years in South Africa, where he developed his political views, ethics and politics.Immediately upon arriving in South Africa, Gandhi faced discrimination because of his skin colour and heritage. He was not allowed to sit with European passengers in the stagecoach and told to sit on the floor near the driver, then beaten when he refused; elsewhere he was kicked into a gutter for daring to walk near a house, in another instance thrown off a train at Pietermaritzburg after refusing to leave the first-class. He sat in the train station, shivering all night and pondering if he should return to India or protest for his rights. He chose to protest and was allowed to board the train the next day. In another incident, the magistrate of a Durban court ordered Gandhi to remove his turban, which he refused to do. Indians were not allowed to walk on public footpaths in South Africa. Gandhi was kicked by a police officer out of the footpath onto the street without warning.When Gandhi arrived in South Africa, according to Herman, he thought of himself as "a Briton first, and an Indian second". However, the prejudice against him and his fellow Indians from British people that Gandhi experienced and observed deeply bothered him. He found it humiliating, struggling to understand how some people can feel honour or superiority or pleasure in such inhumane practices. Gandhi began to question his people's standing in the British Empire.The Abdullah case that had brought him to South Africa concluded in May 1894, and the Indian community organised a farewell party for Gandhi as he prepared to return to India. However, a new Natal government discriminatory proposal led to Gandhi extending his original period of stay in South Africa. He planned to assist Indians in opposing a bill to deny them the right to vote, a right then proposed to be an exclusive European right. He asked Joseph Chamberlain, the British Colonial Secretary, to reconsider his position on this bill. Though unable to halt the bill's passage, his campaign was successful in drawing attention to the grievances of Indians in South Africa. He helped found the Natal Indian Congress in 1894, and through this organisation, he moulded the Indian community of South Africa into a unified political force. In January 1897, when Gandhi landed in Durban, a mob of white settlers attacked him and he escaped only through the efforts of the wife of the police superintendent. However, he refused to press charges against any member of the mob.
During the Boer War, Gandhi volunteered in 1900 to form a group of stretcher-bearers as the Natal Indian Ambulance Corps. According to Arthur Herman, Gandhi wanted to disprove the British colonial stereotype that Hindus were not fit for "manly" activities involving danger and exertion, unlike the Muslim "martial races". Gandhi raised eleven hundred Indian volunteers, to support British combat troops against the Boers. They were trained and medically certified to serve on the front lines. They were auxiliaries at the Battle of Colenso to a White volunteer ambulance corps. At the battle of Spion Kop Gandhi and his bearers moved to the front line and had to carry wounded soldiers for miles to a field hospital because the terrain was too rough for the ambulances. Gandhi and thirty-seven other Indians received the Queen's South Africa Medal.
In 1906, the Transvaal government promulgated a new Act compelling registration of the colony's Indian and Chinese populations. At a mass protest meeting held in Johannesburg on 11 September that year, Gandhi adopted his still evolving methodology of Satyagraha (devotion to the truth), or nonviolent protest, for the first time. According to Anthony Parel, Gandhi was also influenced by the Tamil moral text Tirukkuṛaḷ after Leo Tolstoy mentioned it in their correspondence that began with "A Letter to a Hindu". Gandhi urged Indians to defy the new law and to suffer the punishments for doing so. Gandhi's ideas of protests, persuasion skills and public relations had emerged. He took these back to India in 1915.


=== Europeans, Indians and Africans ===
Gandhi focused his attention on Indians and Africans while he was in South Africa. He initially was not interested in politics. This changed, however, after he was discriminated against and bullied, such as by being thrown out of a train coach because of his skin colour by a white train official. After several such incidents with Whites in South Africa, Gandhi's thinking and focus changed, and he felt he must resist this and fight for rights. He entered politics by forming the Natal Indian Congress. According to Ashwin Desai and Goolam Vahed, Gandhi's views on racism are contentious in some cases, but that changed afterward. Gandhi suffered persecution from the beginning in South Africa. Like with other coloured people, white officials denied him his rights, and the press and those in the streets bullied and called him a "parasite", "semi-barbarous", "canker", "squalid coolie", "yellow man", and other epithets. People would spit on him as an expression of racial hate.
While in South Africa, Gandhi focused on the racial persecution of Indians before he started to focus on racism against Africans. In some cases, state Desai and Vahed, his behaviour was one of being a willing part of racial stereotyping and African exploitation. During a speech in September 1896, Gandhi complained that the whites in the British colony of South Africa were "degrading the Indian to the level of a raw Kaffir". Scholars cite it as an example of evidence that Gandhi at that time thought of Indians and black South Africans differently. As another example given by Herman, Gandhi, at age 24, prepared a legal brief for the Natal Assembly in 1895, seeking voting rights for Indians. Gandhi cited race history and European Orientalists' opinions that "Anglo-Saxons and Indians are sprung from the same Aryan stock or rather the Indo-European peoples", and argued that Indians should not be grouped with the Africans.Years later, Gandhi and his colleagues served and helped Africans as nurses and by opposing racism. The Nobel Peace Prize winner Nelson Mandela is among admirers of Gandhi's efforts to fight against racism in Africa. The general image of Gandhi, state Desai and Vahed, has been reinvented since his assassination as if he was always a saint when in reality his life was more complex, contained inconvenient truths and was one that changed over time. Scholars have also pointed the evidence to a rich history of co-operation and efforts by Gandhi and Indian people with nonwhite South Africans against persecution of Africans and the Apartheid.In 1906, when the Bambatha Rebellion broke out in the colony of Natal, then 36-year-old Gandhi, despite sympathising with the Zulu rebels encouraged Indian South Africans to form a volunteer stretcher-bearer unit. Writing in the Indian Opinion, Gandhi argued that military service would be beneficial to the Indian community and claimed it would give them "health and happiness." Gandhi eventually led a volunteer mixed unit of Indian and African stretcher-bearers to treat wounded combatants during the suppression of the rebellion.
The medical unit commanded by Gandhi operated for less than two months before being disbanded. After the suppression of the rebellion, the colonial establishment showed no interest in extending to the Indian community the civil rights granted to white South Africans. This led Gandhi to becoming disillusioned with the Empire and aroused a spiritual awakening with him; historian Arthur L. Herman wrote that his African experience was a part of his great disillusionment with the West, transforming him into an "uncompromising non-cooperator".By 1910, Gandhi's newspaper, Indian Opinion, was covering reports on discrimination against Africans by the colonial regime. Gandhi remarked that the Africans are "alone are the original inhabitants of the land. … The whites, on the other hand, have occupied the land forcibly and appropriated it to themselves."In 1910, Gandhi established, with the help of his friend Hermann Kallenbach, an idealistic community they named Tolstoy Farm near Johannesburg. There he nurtured his policy of peaceful resistance.In the years after black South Africans gained the right to vote in South Africa (1994), Gandhi was proclaimed a national hero with numerous monuments.


== Struggle for Indian independence (1915–1947) ==

At the request of Gopal Krishna Gokhale, conveyed to him by C. F. Andrews, Gandhi returned to India in 1915. He brought an international reputation as a leading Indian nationalist, theorist and community organiser.
Gandhi joined the Indian National Congress and was introduced to Indian issues, politics and the Indian people primarily by Gokhale. Gokhale was a key leader of the Congress Party best known for his restraint and moderation, and his insistence on working inside the system. Gandhi took Gokhale's liberal approach based on British Whiggish traditions and transformed it to make it look Indian.Gandhi took leadership of the Congress in 1920 and began escalating demands until on 26 January 1930 the Indian National Congress declared the independence of India. The British did not recognise the declaration but negotiations ensued, with the Congress taking a role in provincial government in the late 1930s. Gandhi and the Congress withdrew their support of the Raj when the Viceroy declared war on Germany in September 1939 without consultation. Tensions escalated until Gandhi demanded immediate independence in 1942 and the British responded by imprisoning him and tens of thousands of Congress leaders. Meanwhile, the Muslim League did co-operate with Britain and moved, against Gandhi's strong opposition, to demands for a totally separate Muslim state of Pakistan. In August 1947 the British partitioned the land with India and Pakistan each achieving independence on terms that Gandhi disapproved.


=== Role in World War I ===

In April 1918, during the latter part of World War I, the Viceroy invited Gandhi to a War Conference in Delhi. Gandhi agreed to actively recruit Indians for the war effort. In contrast to the Zulu War of 1906 and the outbreak of World War I in 1914, when he recruited volunteers for the Ambulance Corps, this time Gandhi attempted to recruit combatants. In a June 1918 leaflet entitled "Appeal for Enlistment", Gandhi wrote "To bring about such a state of things we should have the ability to defend ourselves, that is, the ability to bear arms and to use them... If we want to learn the use of arms with the greatest possible despatch, it is our duty to enlist ourselves in the army." He did, however, stipulate in a letter to the Viceroy's private secretary that he "personally will not kill or injure anybody, friend or foe."Gandhi's war recruitment campaign brought into question his consistency on nonviolence. Gandhi's private secretary noted that "The question of the consistency between his creed of 'Ahimsa' (nonviolence) and his recruiting campaign was raised not only then but has been discussed ever since."In July 1918, Gandhi admitted that he couldn't persuade even one individual to enlist for the world war. "So far I have not a single recruit to my credit apart," Gandhi wrote. He added: "They object because they fear to die."


=== Champaran agitations ===

Gandhi's first major achievement came in 1917 with the Champaran agitation in Bihar. The Champaran agitation pitted the local peasantry against largely Anglo-Indian plantation owners who were backed by the local administration. The peasants were forced to grow indigo (Indigofera sp.), a cash crop for Indigo dye whose demand had been declining over two decades, and were forced to sell their crops to the planters at a fixed price. Unhappy with this, the peasantry appealed to Gandhi at his ashram in Ahmedabad. Pursuing a strategy of nonviolent protest, Gandhi took the administration by surprise and won concessions from the authorities.


=== Kheda agitations ===

In 1918, Kheda was hit by floods and famine and the peasantry was demanding relief from taxes. Gandhi moved his headquarters to Nadiad, organising scores of supporters and fresh volunteers from the region, the most notable being Vallabhbhai Patel. Using non-co-operation as a technique, Gandhi initiated a signature campaign where peasants pledged non-payment of revenue even under the threat of confiscation of land. A social boycott of mamlatdars and talatdars (revenue officials within the district) accompanied the agitation. Gandhi worked hard to win public support for the agitation across the country. For five months, the administration refused, but by the end of May 1918, the Government gave way on important provisions and relaxed the conditions of payment of revenue tax until the famine ended. In Kheda, Vallabhbhai Patel represented the farmers in negotiations with the British, who suspended revenue collection and released all the prisoners.


=== Khilafat movement ===

In 1919, following World War I, Gandhi (aged 49) sought political co-operation from Muslims in his fight against British imperialism by supporting the Ottoman Empire that had been defeated in the World War. Before this initiative of Gandhi, communal disputes and religious riots between Hindus and Muslims were common in British India, such as the riots of 1917–18. Gandhi had already supported the British crown with resources and by recruiting Indian soldiers to fight the war in Europe on the British side. This effort of Gandhi was in part motivated by the British promise to reciprocate the help with swaraj (self-government) to Indians after the end of World War I. The British government, instead of self government, had offered minor reforms instead, disappointing Gandhi. Gandhi announced his satyagraha (civil disobedience) intentions. The British colonial officials made their counter move by passing the Rowlatt Act, to block Gandhi's movement. The Act allowed the British government to treat civil disobedience participants as criminals and gave it the legal basis to arrest anyone for "preventive indefinite detention, incarceration without judicial review or any need for a trial".Gandhi felt that Hindu-Muslim co-operation was necessary for political progress against the British. He leveraged the Khilafat movement, wherein Sunni Muslims in India, their leaders such as the sultans of princely states in India and Ali brothers championed the Turkish Caliph as a solidarity symbol of Sunni Islamic community (ummah). They saw the Caliph as their means to support Islam and the Islamic law after the defeat of Ottoman Empire in World War I. Gandhi's support to the Khilafat movement led to mixed results. It initially led to a strong Muslim support for Gandhi. However, the Hindu leaders including Rabindranath Tagore questioned Gandhi's leadership because they were largely against recognising or supporting the Sunni Islamic Caliph in Turkey.The increasing Muslim support for Gandhi, after he championed the Caliph's cause, temporarily stopped the Hindu-Muslim communal violence. It offered evidence of inter-communal harmony in joint Rowlatt satyagraha demonstration rallies, raising Gandhi's stature as the political leader to the British. His support for the Khilafat movement also helped him sideline Muhammad Ali Jinnah, who had announced his opposition to the satyagraha non-co-operation movement approach of Gandhi. Jinnah began creating his independent support, and later went on to lead the demand for West and East Pakistan. Though they agreed in general terms on Indian independence, they disagreed on the means of achieving this. Jinnah was mainly interested in dealing with the British via constitutional negotiation, rather than attempting to agitate the masses.In 1922 the Khilafat movement gradually collapsed following the end of the non-cooperation movement with the arrest of Gandhi. A number of Muslim leaders and delegates abandoned Gandhi and Congress. Hindu-Muslim communal conflicts reignited. Deadly religious riots re-appeared in numerous cities, with 91 in United Provinces of Agra and Oudh alone.


=== Non-co-operation ===

With his book Hind Swaraj (1909) Gandhi, aged 40, declared that British rule was established in India with the co-operation of Indians and had survived only because of this co-operation. If Indians refused to co-operate, British rule would collapse and swaraj (Indian independence) would come.
In February 1919, Gandhi cautioned the Viceroy of India with a cable communication that if the British were to pass the Rowlatt Act, he would appeal to Indians to start civil disobedience. The British government ignored him and passed the law, stating it would not yield to threats. The satyagraha civil disobedience followed, with people assembling to protest the Rowlatt Act. On 30 March 1919, British law officers opened fire on an assembly of unarmed people, peacefully gathered, participating in satyagraha in Delhi.People rioted in retaliation. On 6 April 1919, a Hindu festival day, he asked a crowd to remember not to injure or kill British people, but to express their frustration with peace, to boycott British goods and burn any British clothing they owned. He emphasised the use of non-violence to the British and towards each other, even if the other side used violence. Communities across India announced plans to gather in greater numbers to protest. Government warned him to not enter Delhi. Gandhi defied the order. On 9 April, Gandhi was arrested.On 13 April 1919, people including women with children gathered in an Amritsar park, and British Indian Army officer Reginald Dyer surrounded them and ordered troops under his command to fire on them. The resulting Jallianwala Bagh massacre (or Amritsar massacre) of hundreds of Sikh and Hindu civilians enraged the subcontinent, but was supported by some Britons and parts of the British media as a necessary response. Gandhi in Ahmedabad, on the day after the massacre in Amritsar, did not criticise the British and instead criticised his fellow countrymen for not exclusively using 'love' to deal with the 'hate' of the British government. Gandhi demanded that the Indian people stop all violence, stop all property destruction, and went on fast-to-death to pressure Indians to stop their rioting.The massacre and Gandhi's non-violent response to it moved many, but also made some Sikhs and Hindus upset that Dyer was getting away with murder. Investigation committees were formed by the British, which Gandhi asked Indians to boycott. The unfolding events, the massacre and the British response, led Gandhi to the belief that Indians will never get a fair equal treatment under British rulers, and he shifted his attention to swaraj and political independence for India. In 1921, Gandhi was the leader of the Indian National Congress. He reorganised the Congress. With Congress now behind him, and Muslim support triggered by his backing the Khilafat movement to restore the Caliph in Turkey, Gandhi had the political support and the attention of the British Raj.
Gandhi expanded his nonviolent non-co-operation platform to include the swadeshi policy – the boycott of foreign-made goods, especially British goods. Linked to this was his advocacy that khadi (homespun cloth) be worn by all Indians instead of British-made textiles. Gandhi exhorted Indian men and women, rich or poor, to spend time each day spinning khadi in support of the independence movement. In addition to boycotting British products, Gandhi urged the people to boycott British institutions and law courts, to resign from government employment, and to forsake British titles and honours. Gandhi thus began his journey aimed at crippling the British India government economically, politically and administratively.The appeal of "Non-cooperation" grew, its social popularity drew participation from all strata of Indian society. Gandhi was arrested on 10 March 1922, tried for sedition, and sentenced to six years' imprisonment. He began his sentence on 18 March 1922. With Gandhi isolated in prison, the Indian National Congress split into two factions, one led by Chitta Ranjan Das and Motilal Nehru favouring party participation in the legislatures, and the other led by Chakravarti Rajagopalachari and Sardar Vallabhbhai Patel, opposing this move. Furthermore, co-operation among Hindus and Muslims ended as Khilafat movement collapsed with the rise of Atatürk in Turkey. Muslim leaders left the Congress and began forming Muslim organisations. The political base behind Gandhi had broken into factions. Gandhi was released in February 1924 for an appendicitis operation, having served only two years.


=== Salt Satyagraha (Salt March) ===

After his early release from prison for political crimes in 1924, over the second half of the 1920s Gandhi continued to pursue swaraj. He pushed through a resolution at the Calcutta Congress in December 1928 calling on the British government to grant India dominion status or face a new campaign of non-cooperation with complete independence for the country as its goal. After his support for World War I with Indian combat troops, and the failure of Khilafat movement in preserving the rule of Caliph in Turkey, followed by a collapse in Muslim support for his leadership, some such as Subhas Chandra Bose and Bhagat Singh questioned his values and non-violent approach. While many Hindu leaders championed a demand for immediate independence, Gandhi revised his own call to a one-year wait, instead of two.The British did not respond favourably to Gandhi's proposal. British political leaders such as Lord Birkenhead and Winston Churchill announced opposition to "the appeasers of Gandhi" in their discussions with European diplomats who sympathised with Indian demands. On 31 December 1929, an Indian flag was unfurled in Lahore. Gandhi led Congress in a celebration on 26 January 1930 of India's Independence Day in Lahore. This day was commemorated by almost every other Indian organisation. Gandhi then launched a new Satyagraha against the British salt tax in March 1930. Gandhi sent an ultimatum in the form of a letter personally addressed to Lord Irwin, the viceroy of India, on 2 March. Gandhi condemned British rule in the letter, describing it as "a curse" that "has impoverished the dumb millions by a system of progressive exploitation and by a ruinously expensive military and civil administration...It has reduced us politically to serfdom." Gandhi also mentioned in the letter that the viceroy received a salary "over five thousand times India's average income." In the letter, Gandhi also stressed his continued adherence to non-violent forms of protest.This was highlighted by the Salt March to Dandi from 12 March to 6 April, where, together with 78 volunteers, he marched 388 kilometres (241 mi) from Ahmedabad to Dandi, Gujarat to make salt himself, with the declared intention of breaking the salt laws. The march took 25 days to cover 240 miles with Gandhi speaking to often huge crowds along the way. Thousands of Indians joined him in Dandi. On 5 May he was interned under a regulation dating from 1827 in anticipation of a protest that he had planned. The protest at Dharasana salt works on 21 May went ahead without him see. A horrified American journalist, Webb Miller, described the British response thus:

In complete silence the Gandhi men drew up and halted a hundred yards from the stockade. A picked column advanced from the crowd, waded the ditches and approached the barbed wire stockade... at a word of command, scores of native policemen rushed upon the advancing marchers and rained blows on their heads with their steel-shot lathis [long bamboo sticks]. Not one of the marchers even raised an arm to fend off blows. They went down like ninepins. From where I stood I heard the sickening whack of the clubs on unprotected skulls... Those struck down fell sprawling, unconscious or writhing with fractured skulls or broken shoulders.
This went on for hours until some 300 or more protesters had been beaten, many seriously injured and two killed. At no time did they offer any resistance.
This campaign was one of his most successful at upsetting British hold on India; Britain responded by imprisoning over 60,000 people. Congress estimates, however, put the figure at 90,000. Among them was one of Gandhi's lieutenants, Jawaharlal Nehru.
According to Sarma, Gandhi recruited women to participate in the salt tax campaigns and the boycott of foreign products, which gave many women a new self-confidence and dignity in the mainstream of Indian public life. However, other scholars such as Marilyn French state that Gandhi barred women from joining his civil disobedience movement because he feared he would be accused of using women as a political shield. When women insisted on joining the movement and participating in public demonstrations, Gandhi asked the volunteers to get permissions of their guardians and only those women who can arrange child-care should join him. Regardless of Gandhi's apprehensions and views, Indian women joined the Salt March by the thousands to defy the British salt taxes and monopoly on salt mining. After Gandhi's arrest, the women marched and picketed shops on their own, accepting violence and verbal abuse from British authorities for the cause in the manner Gandhi inspired.


=== Gandhi as folk hero ===
Indian Congress in the 1920s appealed to Andhra Pradesh peasants by creating Telugu language plays that combined Indian mythology and legends, linked them to Gandhi's ideas, and portrayed Gandhi as a messiah, a reincarnation of ancient and medieval Indian nationalist leaders and saints. The plays built support among peasants steeped in traditional Hindu culture, according to Murali, and this effort made Gandhi a folk hero in Telugu speaking villages, a sacred messiah-like figure.According to Dennis Dalton, it was Gandhi's ideas that were responsible for his wide following. Gandhi criticised Western civilisation as one driven by "brute force and immorality", contrasting it with his categorisation of Indian civilisation as one driven by "soul force and morality". Gandhi captured the imagination of the people of his heritage with his ideas about winning "hate with love". These ideas are evidenced in his pamphlets from the 1890s, in South Africa, where too he was popular among the Indian indentured workers. After he returned to India, people flocked to him because he reflected their values.
Gandhi also campaigned hard going from one rural corner of the Indian subcontinent to another. He used terminology and phrases such as Rama-rajya from Ramayana, Prahlada as a paradigmatic icon, and such cultural symbols as another facet of swaraj and satyagraha. During his lifetime, these ideas sounded strange outside India, but they readily and deeply resonated with the culture and historic values of his people.


=== Negotiations ===
The government, represented by Lord Irwin, decided to negotiate with Gandhi. The Gandhi–Irwin Pact was signed in March 1931. The British Government agreed to free all political prisoners, in return for the suspension of the civil disobedience movement. According to the pact, Gandhi was invited to attend the Round Table Conference in London for discussions and as the sole representative of the Indian National Congress. The conference was a disappointment to Gandhi and the nationalists. Gandhi expected to discuss India's independence, while the British side focused on the Indian princes and Indian minorities rather than on a transfer of power. Lord Irwin's successor, Lord Willingdon, took a hard line against India as an independent nation, began a new campaign of controlling and subduing the nationalist movement. Gandhi was again arrested, and the government tried and failed to negate his influence by completely isolating him from his followers.In Britain, Winston Churchill, a prominent Conservative politician who was then out of office but later became its prime minister, became a vigorous and articulate critic of Gandhi and opponent of his long-term plans. Churchill often ridiculed Gandhi, saying in a widely reported 1931 speech:

It is alarming and also nauseating to see Mr Gandhi, a seditious Middle Temple lawyer, now posing as a fakir of a type well known in the East, striding half-naked up the steps of the Vice-regal palace....to parley on equal terms with the representative of the King-Emperor.
Churchill's bitterness against Gandhi grew in the 1930s. He called Gandhi as the one who was "seditious in aim" whose evil genius and multiform menace was attacking the British empire. Churchill called him a dictator, a "Hindu Mussolini", fomenting a race war, trying to replace the Raj with Brahmin cronies, playing on the ignorance of Indian masses, all for selfish gain. Churchill attempted to isolate Gandhi, and his criticism of Gandhi was widely covered by European and American press. It gained Churchill sympathetic support, but it also increased support for Gandhi among Europeans. The developments heightened Churchill's anxiety that the "British themselves would give up out of pacifism and misplaced conscience".


=== Round Table Conferences ===
During the discussions between Gandhi and the British government over 1931–32 at the Round Table Conferences, Gandhi, now aged about 62, sought constitutional reforms as a preparation to the end of colonial British rule, and begin the self-rule by Indians. The British side sought reforms that would keep the Indian subcontinent as a colony. The British negotiators proposed constitutional reforms on a British Dominion model that established separate electorates based on religious and social divisions. The British questioned the Congress party and Gandhi's authority to speak for all of India. They invited Indian religious leaders, such as Muslims and Sikhs, to press their demands along religious lines, as well as B. R. Ambedkar as the representative leader of the untouchables. Gandhi vehemently opposed a constitution that enshrined rights or representations based on communal divisions, because he feared that it would not bring people together but divide them, perpetuate their status, and divert the attention from India's struggle to end the colonial rule.The Second Round Table conference was the only time he left India between 1914 and his death in 1948. He declined the government's offer of accommodation in an expensive West End hotel, preferring to stay in the East End, to live among working-class people, as he did in India. He based himself in a small cell-bedroom at Kingsley Hall for the three-month duration of his stay and was enthusiastically received by East Enders. During this time he renewed his links with the British vegetarian movement.

After Gandhi returned from the Second Round Table conference, he started a new satyagraha. He was arrested and imprisoned at the Yerwada Jail, Pune. While he was in prison, the British government enacted a new law that granted untouchables a separate electorate. It came to be known as the Communal Award. In protest, Gandhi started a fast-unto-death, while he was held in prison. The resulting public outcry forced the government, in consultations with Ambedkar, to replace the Communal Award with a compromise Poona Pact.


=== Congress politics ===
In 1934 Gandhi resigned from Congress party membership. He did not disagree with the party's position but felt that if he resigned, his popularity with Indians would cease to stifle the party's membership, which actually varied, including communists, socialists, trade unionists, students, religious conservatives, and those with pro-business convictions, and that these various voices would get a chance to make themselves heard. Gandhi also wanted to avoid being a target for Raj propaganda by leading a party that had temporarily accepted political accommodation with the Raj.Gandhi returned to active politics again in 1936, with the Nehru presidency and the Lucknow session of the Congress. Although Gandhi wanted a total focus on the task of winning independence and not speculation about India's future, he did not restrain the Congress from adopting socialism as its goal. Gandhi had a clash with Subhas Chandra Bose, who had been elected president in 1938, and who had previously expressed a lack of faith in nonviolence as a means of protest. Despite Gandhi's opposition, Bose won a second term as Congress President, against Gandhi's nominee, Bhogaraju Pattabhi Sitaramayya. Gandhi declared that Sitaramayya's defeat was his defeat. Bose later left the Congress when the All-India leaders resigned en masse in protest of his abandonment of the principles introduced by Gandhi.


=== World War II and Quit India movement ===

Gandhi opposed providing any help to the British war effort and he campaigned against any Indian participation in World War II. The British government responded with the arrests of Gandhi and many other Congress leaders and killed over 1,000 Indians who participated in this movement. A number of violent attacks were also carried out by the nationalists against the British government. While Gandhi's campaign did not enjoy the support of a number of Indian leaders, and over 2.5 million Indians volunteered and joined the British military to fight on various fronts of the Allied Forces, the movement played a role in weakening the control over the South Asian region by the British regime and it ultimately paved the way for Indian independence.Gandhi's opposition to the Indian participation in World War II was motivated by his belief that India could not be party to a war ostensibly being fought for democratic freedom while that freedom was denied to India itself. He also condemned Nazism and Fascism, a view which won endorsement of other Indian leaders. As the war progressed, Gandhi intensified his demand for independence, calling for the British to Quit India in a 1942 speech in Mumbai. This was Gandhi's and the Congress Party's most definitive revolt aimed at securing the British exit from India. The British government responded quickly to the Quit India speech, and within hours after Gandhi's speech arrested Gandhi and all the members of the Congress Working Committee. His countrymen retaliated the arrests by damaging or burning down hundreds of government owned railway stations, police stations, and cutting down telegraph wires.In 1942, Gandhi now nearing age 73, urged his people to completely stop co-operating with the imperial government. In this effort, he urged that they neither kill nor injure British people, but be willing to suffer and die if violence is initiated by the British officials. He clarified that the movement would not be stopped because of any individual acts of violence, saying that the "ordered anarchy" of "the present system of administration" was "worse than real anarchy." He urged Indians to karo ya maro ("do or die") in the cause of their rights and freedoms.
Gandhi's arrest lasted two years, as he was held in the Aga Khan Palace in Pune. During this period, his long time secretary Mahadev Desai died of a heart attack, his wife Kasturba died after 18 months' imprisonment on 22 February 1944; and Gandhi suffered a severe malaria attack. While in jail, he agreed to an interview with Stuart Gelder, a British journalist. Gelder then composed and released an interview summary, cabled it to the mainstream press, that announced sudden concessions Gandhi was willing to make, comments that shocked his countrymen, the Congress workers and even Gandhi. The latter two claimed that it distorted what Gandhi actually said on a range of topics and falsely repudiated the Quit India movement.Gandhi was released before the end of the war on 6 May 1944 because of his failing health and necessary surgery; the Raj did not want him to die in prison and enrage the nation. He came out of detention to an altered political scene – the Muslim League for example, which a few years earlier had appeared marginal, "now occupied the centre of the political stage" and the topic of Jinnah's campaign for Pakistan was a major talking point. Gandhi and Jinnah had extensive correspondence and the two men met several times over a period of two weeks in September 1944 at Jinnah's house in Bombay, where Gandhi insisted on a united religiously plural and independent India which included Muslims and non-Muslims of the Indian subcontinent coexisting. Jinnah rejected this proposal and insisted instead for partitioning the subcontinent on religious lines to create a separate Muslim homeland (later Pakistan). These discussions continued through 1947.While the leaders of Congress languished in jail, the other parties supported the war and gained organisational strength. Underground publications flailed at the ruthless suppression of Congress, but it had little control over events. At the end of the war, the British gave clear indications that power would be transferred to Indian hands. At this point Gandhi called off the struggle, and around 100,000 political prisoners were released, including the Congress's leadership.


=== Partition and independence ===

Gandhi opposed the partition of the Indian subcontinent along religious lines. The Indian National Congress and Gandhi called for the British to Quit India. However, the All-India Muslim League demanded "Divide and Quit India". Gandhi suggested an agreement which required the Congress and the Muslim League to co-operate and attain independence under a provisional government, thereafter, the question of partition could be resolved by a plebiscite in the districts with a Muslim majority.Jinnah rejected Gandhi's proposal and called for Direct Action Day, on 16 August 1946, to press Muslims to publicly gather in cities and support his proposal for the partition of the Indian subcontinent into a Muslim state and non-Muslim state. Huseyn Shaheed Suhrawardy, the Muslim League Chief Minister of Bengal – now Bangladesh and West Bengal, gave Calcutta's police special holiday to celebrate the Direct Action Day. The Direct Action Day triggered a mass murder of Calcutta Hindus and the torching of their property, and holidaying police were missing to contain or stop the conflict. The British government did not order its army to move in to contain the violence. The violence on Direct Action Day led to retaliatory violence against Muslims across India. Thousands of Hindus and Muslims were murdered, and tens of thousands were injured in the cycle of violence in the days that followed. Gandhi visited the most riot-prone areas to appeal a stop to the massacres.
Archibald Wavell, the Viceroy and Governor-General of British India for three years through February 1947, had worked with Gandhi and Jinnah to find a common ground, before and after accepting Indian independence in principle. Wavell condemned Gandhi's character and motives as well as his ideas. Wavell accused Gandhi of harbouring the single minded idea to "overthrow British rule and influence and to establish a Hindu raj", and called Gandhi a "malignant, malevolent, exceedingly shrewd" politician. Wavell feared a civil war on the Indian subcontinent, and doubted Gandhi would be able to stop it.The British reluctantly agreed to grant independence to the people of the Indian subcontinent, but accepted Jinnah's proposal of partitioning the land into Pakistan and India. Gandhi was involved in the final negotiations, but Stanley Wolpert states the "plan to carve up British India was never approved of or accepted by Gandhi".The partition was controversial and violently disputed. More than half a million were killed in religious riots as 10 million to 12 million non-Muslims (Hindus and Sikhs mostly) migrated from Pakistan into India, and Muslims migrated from India into Pakistan, across the newly created borders of India, West Pakistan and East Pakistan.Gandhi spent the day of independence not celebrating the end of the British rule but appealing for peace among his countrymen by fasting and spinning in Calcutta on 15 August 1947. The partition had gripped the Indian subcontinent with religious violence and the streets were filled with corpses. Gandhi's fasting and protests are credited for stopping the religious riots and communal violence.


== Death ==

At 5:17 pm on 30 January 1948, Gandhi was with his grandnieces in the garden of Birla House (now Gandhi Smriti), on his way to address a prayer meeting, when Nathuram Godse, a Hindu nationalist, fired three bullets into his chest from a pistol at close range. According to some accounts, Gandhi died instantly. In other accounts, such as one prepared by an eyewitness journalist, Gandhi was carried into the Birla House, into a bedroom. There he died about 30 minutes later as one of Gandhi's family members read verses from Hindu scriptures.Prime Minister Jawaharlal Nehru addressed his countrymen over the All-India Radio saying:
Friends and comrades, the light has gone out of our lives, and there is darkness everywhere, and I do not quite know what to tell you or how to say it. Our beloved leader, Bapu as we called him, the father of the nation, is no more. Perhaps I am wrong to say that; nevertheless, we will not see him again, as we have seen him for these many years, we will not run to him for advice or seek solace from him, and that is a terrible blow, not only for me, but for millions and millions in this country.
Godse, a Hindu nationalist, with links to the Hindu Mahasabha and the Rashtriya Swayamsevak Sangh, made no attempt to escape; several other conspirators were soon arrested as well. The accused were Nathuram Vinayak Godse, Narayan Apte, Vinayak Damodar Savarkar, Shankar Kistayya, Dattatraya Parchure, Vishnu Karkare, Madanlal Pahwa, and Gopal Godse.: 38 The trial began on 27 May 1948 and ran for eight months before Justice Atma Charan passed his final order on 10 February 1949. The prosecution called 149 witnesses, the defense none. The court found all of the defendants except one guilty as charged. Eight men were convicted for the murder conspiracy, and others were convicted for violation of the Explosive Substances Act. Savarkar was acquitted and set free. Nathuram Godse and Narayan Apte were sentenced to death by hanging and the remaining six (including Godse's brother, Gopal) were sentenced to life imprisonment.


=== Funeral and memorials ===
Gandhi's death was mourned nationwide. Over a million people joined the five-mile-long funeral procession that took over five hours to reach Raj Ghat from Birla house, where he was assassinated, and another million watched the procession pass by. Gandhi's body was transported on a weapons carrier, whose chassis was dismantled overnight to allow a high-floor to be installed so that people could catch a glimpse of his body. The engine of the vehicle was not used; instead four drag-ropes held by 50 people each pulled the vehicle. All Indian-owned establishments in London remained closed in mourning as thousands of people from all faiths and denominations and Indians from all over Britain converged at India House in London.
Gandhi was cremated in accordance with Hindu tradition. His ashes were poured into urns which were sent across India for memorial services. Most of the ashes were immersed at the Sangam at Allahabad on 12 February 1948, but some were secretly taken away. In 1997, Tushar Gandhi immersed the contents of one urn, found in a bank vault and reclaimed through the courts, at the Sangam at Allahabad. Some of Gandhi's ashes were scattered at the source of the Nile River near Jinja, Uganda, and a memorial plaque marks the event. On 30 January 2008, the contents of another urn were immersed at Girgaum Chowpatty. Another urn is at the palace of the Aga Khan in Pune (where Gandhi was held as a political prisoner from 1942 to 1944) and another in the Self-Realization Fellowship Lake Shrine in Los Angeles.The Birla House site where Gandhi was assassinated is now a memorial called Gandhi Smriti. The place near Yamuna river where he was cremated is the Rāj Ghāt memorial in New Delhi. A black marble platform, it bears the epigraph "Hē Rāma" (Devanagari: हे ! राम or, Hey Raam). These are said to be Gandhi's last words after he was shot.


== Principles, practices, and beliefs ==

Gandhi's statements, letters and life have attracted much political and scholarly analysis of his principles, practices and beliefs, including what influenced him. Some writers present him as a paragon of ethical living and pacifism, while others present him as a more complex, contradictory and evolving character influenced by his culture and circumstances.


=== Truth and Satyagraha ===
Gandhi dedicated his life to discovering and pursuing truth, or Satya, and called his movement satyagraha, which means "appeal to, insistence on, or reliance on the Truth". The first formulation of the satyagraha as a political movement and principle occurred in 1920, which he tabled as "Resolution on Non-cooperation" in September that year before a session of the Indian Congress. It was the satyagraha formulation and step, states Dennis Dalton, that deeply resonated with beliefs and culture of his people, embedded him into the popular consciousness, transforming him quickly into Mahatma.
Gandhi based Satyagraha on the Vedantic ideal of self-realisation, ahimsa (nonviolence), vegetarianism, and universal love. William Borman states that the key to his satyagraha is rooted in the Hindu Upanishadic texts. According to Indira Carr, Gandhi's ideas on ahimsa and satyagraha were founded on the philosophical foundations of Advaita Vedanta. I. Bruce Watson states that some of these ideas are found not only in traditions within Hinduism, but also in Jainism or Buddhism, particularly those about non-violence, vegetarianism and universal love, but Gandhi's synthesis was to politicise these ideas. Gandhi's concept of satya as a civil movement, states Glyn Richards, are best understood in the context of the Hindu terminology of Dharma and Ṛta.Gandhi stated that the most important battle to fight was overcoming his own demons, fears, and insecurities. Gandhi summarised his beliefs first when he said "God is Truth". He would later change this statement to "Truth is God". Thus, satya (truth) in Gandhi's philosophy is "God". Gandhi, states Richards, described the term "God" not as a separate power, but as the Being (Brahman, Atman) of the Advaita Vedanta tradition, a nondual universal that pervades in all things, in each person and all life. According to Nicholas Gier, this to Gandhi meant the unity of God and humans, that all beings have the same one soul and therefore equality, that atman exists and is same as everything in the universe, ahimsa (non-violence) is the very nature of this atman.
The essence of Satyagraha is "soul force" as a political means, refusing to use brute force against the oppressor, seeking to eliminate antagonisms between the oppressor and the oppressed, aiming to transform or "purify" the oppressor. It is not inaction but determined passive resistance and non-co-operation where, states Arthur Herman, "love conquers hate". A euphemism sometimes used for Satyagraha is that it is a "silent force" or a "soul force" (a term also used by Martin Luther King Jr. during his "I Have a Dream" speech). It arms the individual with moral power rather than physical power. Satyagraha is also termed a "universal force", as it essentially "makes no distinction between kinsmen and strangers, young and old, man and woman, friend and foe."Gandhi wrote: "There must be no impatience, no barbarity, no insolence, no undue pressure. If we want to cultivate a true spirit of democracy, we cannot afford to be intolerant. Intolerance betrays want of faith in one's cause." Civil disobedience and non-co-operation as practised under Satyagraha are based on the "law of suffering", a doctrine that the endurance of suffering is a means to an end. This end usually implies a moral upliftment or progress of an individual or society. Therefore, non-co-operation in Satyagraha is in fact a means to secure the co-operation of the opponent consistently with truth and justice.While Gandhi's idea of satyagraha as a political means attracted a widespread following among Indians, the support was not universal. For example, Muslim leaders such as Jinnah opposed the satyagraha idea, accused Gandhi to be reviving Hinduism through political activism, and began effort to counter Gandhi with Muslim nationalism and a demand for Muslim homeland. The untouchability leader Ambedkar, in June 1945, after his decision to convert to Buddhism and the first Law and Justice minister of modern India, dismissed Gandhi's ideas as loved by "blind Hindu devotees", primitive, influenced by spurious brew of Tolstoy and Ruskin, and "there is always some simpleton to preach them". Winston Churchill caricatured Gandhi as a "cunning huckster" seeking selfish gain, an "aspiring dictator", and an "atavistic spokesman of a pagan Hinduism". Churchill stated that the civil disobedience movement spectacle of Gandhi only increased "the danger to which white people there [British India] are exposed".


=== Nonviolence ===
Although Gandhi was not the originator of the principle of nonviolence, he was the first to apply it in the political field on a large scale. The concept of nonviolence (ahimsa) has a long history in Indian religious thought, and is considered the highest dharma (ethical value virtue), a precept to be observed towards all living beings (sarvbhuta), at all times (sarvada), in all respects (sarvatha), in action, words and thought. Gandhi explains his philosophy and ideas about ahimsa as a political means in his autobiography The Story of My Experiments with Truth.Even though Gandhi considered non-violence to be "infinitely superior to violence", he preferred violence to cowardice. He added that he "would rather have India resort to arms in order to defend her honor than that she should in a cowardly manner become or remain a helpless witness to her own dishonor".


== Literary works ==
Gandhi was a prolific writer. His signature style was simple, precise, clear and as devoid of artificialities. One of Gandhi's earliest publications, Hind Swaraj, published in Gujarati in 1909, became "the intellectual blueprint" for India's independence movement. The book was translated into English the next year, with a copyright legend that read "No Rights Reserved". For decades he edited several newspapers including Harijan in Gujarati, in Hindi and in the English language; Indian Opinion while in South Africa and, Young India, in English, and Navajivan, a Gujarati monthly, on his return to India. Later, Navajivan was also published in Hindi. In addition, he wrote letters almost every day to individuals and newspapers.Gandhi also wrote several books including his autobiography, The Story of My Experiments with Truth (Gujarātī "સત્યના પ્રયોગો અથવા આત્મકથા"), of which he bought the entire first edition to make sure it was reprinted. His other autobiographies included: Satyagraha in South Africa about his struggle there, Hind Swaraj or Indian Home Rule, a political pamphlet, and a paraphrase in Gujarati of John Ruskin's Unto This Last which was an early critique of political economy. This last essay can be considered his programme on economics. He also wrote extensively on vegetarianism, diet and health, religion, social reforms, etc. Gandhi usually wrote in Gujarati, though he also revised the Hindi and English translations of his books. In 1934, he wrote Songs from Prison while prisoned in Yerawada jail in Maharashtra.Gandhi's complete works were published by the Indian government under the name The Collected Works of Mahatma Gandhi in the 1960s. The writings comprise about 50,000 pages published in about a hundred volumes. In 2000, a revised edition of the complete works sparked a controversy, as it contained a large number of errors and omissions. The Indian government later withdrew the revised edition.


== Legacy ==

Gandhi is noted as the greatest figure of the successful Indian independence movement against the British rule. He is also hailed as the greatest figure of modern India. American historian Stanley Wolpert described Gandhi as "India's greatest revolutionary nationalist leader" and the greatest Indian since the Buddha. In 1999, Gandhi was named "Asian of the century" by Asiaweek. In a 2000 BBC poll, he was voted as the greatest man of the millennium.The word Mahatma, while often mistaken for Gandhi's given name in the West, is taken from the Sanskrit words maha (meaning Great) and atma (meaning Soul). He was publicly bestowed with the honorific title "Mahatma" in July 1914 at farewell meeting in Town Hall, Durban. Rabindranath Tagore is said to have accorded the title to Gandhi by 1915. In his autobiography, Gandhi nevertheless explains that he never valued the title, and was often pained by it.Innumerable streets, roads and localities in India are named after Gandhi. These include M.G.Road (the main street of a number of Indian cities including Mumbai, Bangalore, Kolkata, Lucknow, Kanpur, Gangtok and Indore), Gandhi Market (near Sion, Mumbai) and Gandhinagar (the capital of the state of Gujarat, Gandhi's birthplace).
As of 2008, over 150 countries have released stamps on Gandhi. In October 2019, about 87 countries including Turkey, the United States, Russia, Iran, Uzbekistan, and Palestine released commemorative Gandhi stamps on the 150th anniversary of his birth.Florian asteroid 120461 Gandhi was named in his honour in September 2020. In October 2022, a statue of Gandhi was installed in Astana on the embankment of the rowing canal, opposite the cult monument to the defenders of Kazakhstan.On 15 December 2022, the United Nations headquarters in New York unveiled the statue of Gandhi. UN Secretary-General António Guterres called Gandhi an "uncompromising advocate for peaceful co-existence".


=== Followers and international influence ===
Gandhi influenced important leaders and political movements. Leaders of the civil rights movement in the United States, including Martin Luther King Jr., James Lawson, and James Bevel, drew from the writings of Gandhi in the development of their own theories about nonviolence. King said "Christ gave us the goals and Mahatma Gandhi the tactics." King sometimes referred to Gandhi as "the little brown saint." Anti-apartheid activist and former President of South Africa, Nelson Mandela, was inspired by Gandhi. Others include Steve Biko, Václav Havel, and Aung San Suu Kyi.
In his early years, the former President of South Africa Nelson Mandela was a follower of the nonviolent resistance philosophy of Gandhi. Bhana and Vahed commented on these events as "Gandhi inspired succeeding generations of South African activists seeking to end White rule. This legacy connects him to Nelson Mandela...in a sense, Mandela completed what Gandhi started."Gandhi's life and teachings inspired many who specifically referred to Gandhi as their mentor or who dedicated their lives to spreading Gandhi's ideas. In Europe, Romain Rolland was the first to discuss Gandhi in his 1924 book Mahatma Gandhi, and Brazilian anarchist and feminist Maria Lacerda de Moura wrote about Gandhi in her work on pacifism. In 1931, physicist Albert Einstein exchanged letters with Gandhi, and called him "a role model for the generations to come" in a letter writing about him. Einstein said of Gandhi:

Mahatma Gandhi's life achievement stands unique in political history. He has invented a completely new and humane means for the liberation war of an oppressed country, and practised it with greatest energy and devotion. The moral influence he had on the consciously thinking human being of the entire civilised world will probably be much more lasting than it seems in our time with its overestimation of brutal violent forces. Because lasting will only be the work of such statesmen who wake up and strengthen the moral power of their people through their example and educational works. We may all be happy and grateful that destiny gifted us with such an enlightened contemporary, a role model for the generations to come.
Generations to come will scarce believe that such a one as this walked the earth in flesh and blood.
Farah Omar, a political activist from Somaliland visited India in 1930, where he met Gandhi and was influenced by Gandhi's non-violent philosophy which he adopted in his campaign in British Somaliland.Lanza del Vasto went to India in 1936 intending to live with Gandhi; he later returned to Europe to spread Gandhi's philosophy and founded the Community of the Ark in 1948 (modelled after Gandhi's ashrams). Madeleine Slade (known as "Mirabehn") was the daughter of a British admiral who spent much of her adult life in India as a devotee of Gandhi.In addition, the British musician John Lennon referred to Gandhi when discussing his views on nonviolence. In 2007, former US Vice President and environmentalist Al Gore drew upon Gandhi's idea of satyagraha in a speech on climate change. 44th President of the United States Barack Obama said in September 2009 that his biggest inspiration came from Gandhi. His reply was in response to the question "Who was the one person, dead or live, that you would choose to dine with?". He continued that "He's somebody I find a lot of inspiration in. He inspired Dr. King with his message of nonviolence. He ended up doing so much and changed the world just by the power of his ethics."Time magazine named The 14th Dalai Lama, Lech Wałęsa, Martin Luther King Jr., Cesar Chavez, Aung San Suu Kyi, Benigno Aquino Jr., Desmond Tutu, and Nelson Mandela as Children of Gandhi and his spiritual heirs to nonviolence. The Mahatma Gandhi District in Houston, Texas, United States, an ethnic Indian enclave, is officially named after Gandhi.Gandhi's ideas had a significant influence on 20th-century philosophy. It began with his engagement with Romain Rolland and Martin Buber. Jean-Luc Nancy said that the French philosopher Maurice Blanchot engaged critically with Gandhi from the point of view of "European spirituality". Since then philosophers including Hannah Arendt, Etienne Balibar and Slavoj Žižek found that Gandhi was a necessary reference to discuss morality in politics. Recently in the light of climate change Gandhi's views on technology are gaining importance in the fields of environmental philosophy and philosophy of technology.


=== Global days that celebrate Gandhi ===
In 2007, the United Nations General Assembly declared Gandhi's birthday 2 October as "the International Day of Nonviolence." First proposed by UNESCO in 1948, as the School Day of Nonviolence and Peace (DENIP in Spanish), 30 January is observed as the School Day of Nonviolence and Peace in schools of many countries In countries with a Southern Hemisphere school calendar, it is observed on 30 March.


=== Awards ===
Time magazine named Gandhi the Man of the Year in 1930. In the same magazine's 1999 list of The Most Important People of the Century, Gandhi was second only to Albert Einstein, who had called Gandhi "the greatest man of our age". The University of Nagpur awarded him an LL.D. in 1937. The Government of India awarded the annual Gandhi Peace Prize to distinguished social workers, world leaders and citizens. Nelson Mandela, the leader of South Africa's struggle to eradicate racial discrimination and segregation, was a prominent non-Indian recipient. In 2011, Gandhi topped the TIME's list of top 25 political icons of all time.Gandhi did not receive the Nobel Peace Prize, although he was nominated five times between 1937 and 1948, including the first-ever nomination by the American Friends Service Committee, though he made the short list only twice, in 1937 and 1947. Decades later, the Nobel Committee publicly declared its regret for the omission, and admitted to deeply divided nationalistic opinion denying the award. Gandhi was nominated in 1948 but was assassinated before nominations closed. That year, the committee chose not to award the peace prize stating that "there was no suitable living candidate" and later research shows that the possibility of awarding the prize posthumously to Gandhi was discussed and that the reference to no suitable living candidate was to Gandhi. Geir Lundestad, Secretary of Norwegian Nobel Committee in 2006 said, "The greatest omission in our 106-year history is undoubtedly that Mahatma Gandhi never received the Nobel Peace prize. Gandhi could do without the Nobel Peace prize, whether Nobel committee can do without Gandhi is the question". When the 14th Dalai Lama was awarded the Prize in 1989, the chairman of the committee said that this was "in part a tribute to the memory of Mahatma Gandhi". In the summer of 1995, the North American Vegetarian Society inducted him posthumously into the Vegetarian Hall of Fame.


==== Father of the Nation ====
Indians widely describe Gandhi as the Father of the Nation. Origin of this title is traced back to a radio address (on Singapore radio) on 6 July 1944 by Subhash Chandra Bose where Bose addressed Gandhi as "The Father of the Nation". On 28 April 1947, Sarojini Naidu during a conference also referred Gandhi as "Father of the Nation". He is also conferred the title "Bapu" (Gujarati: endearment for father, papa).


=== Film, theatre and literature ===
A five-hour nine-minute long biographical documentary film, Mahatma: Life of Gandhi, 1869–1948, made by Vithalbhai Jhaveri in 1968, quoting Gandhi's words and using black and white archival footage and photographs, captures the history of those times. Ben Kingsley portrayed him in Richard Attenborough's 1982 film Gandhi, which won the Academy Award for Best Picture. It was based on the biography by Louis Fischer. The 1996 film The Making of the Mahatma documented Gandhi's time in South Africa and his transformation from an inexperienced barrister to recognised political leader.
Gandhi was a central figure in the 2006 comedy film Lage Raho Munna Bhai. Jahnu Barua's Maine Gandhi Ko Nahin Mara (I did not kill Gandhi), places contemporary society as a backdrop with its vanishing memory of Gandhi's values as a metaphor for the senile forgetfulness of the protagonist of his 2005 film, writes Vinay Lal.
In the tale Le Jour du Jugement Dernier, in the collection Les Mémoires de Satan et autres contes loufoques, by Pierre Cormon, God tries to judge Gandhi at the Last Judgement but realises that the character is more complex than he appears.
In 1967, Gandhi was set to be featured on the album cover of one of the best selling albums of The Beatles, Sgt. Pepper's Lonely Hearts Club Band, however this idea was later cancelled due to respect for Gandhi.
The 1979 opera Satyagraha by American composer Philip Glass is loosely based on Gandhi's life. The opera's libretto, taken from the Bhagavad Gita, is sung in the original Sanskrit.
The 1995 Marathi play Gandhi Virudh Gandhi explored the relationship between Gandhi and his son Harilal. The 2007 film, Gandhi, My Father was inspired on the same theme. The 1989 Marathi play Me Nathuram Godse Boltoy and the 1997 Hindi play Gandhi Ambedkar criticised Gandhi and his principles.
Several biographers have undertaken the task of describing Gandhi's life. Among them are D. G. Tendulkar with his Mahatma. Life of Mohandas Karamchand Gandhi in eight volumes, Chaman Nahal's Gandhi Quartet, and Pyarelal and Sushila Nayyar with their Mahatma Gandhi in 10 volumes. The 2010 biography, Great Soul: Mahatma Gandhi and His Struggle With India by Joseph Lelyveld contained controversial material speculating about Gandhi's sexual life. Lelyveld, however, stated that the press coverage "grossly distort[s]" the overall message of the book. The 2014 film Welcome Back Gandhi takes a fictionalised look at how Gandhi might react to modern day India. The 2019 play Bharat Bhagya Vidhata, inspired by Pujya Gurudevshri Rakeshbhai and produced by Sangeet Natak Akademi and Shrimad Rajchandra Mission Dharampur takes a look at how Gandhi cultivated the values of truth and non-violence.
"Mahatma Gandhi" is used by Cole Porter in his lyrics for the song "You're the Top" which is included in the 1934 musical Anything Goes. In the song, Porter rhymes 'Mahatma Gandhi' with 'Napoleon Brandy.'
Gandhi is mentioned in the Kris Kristofferson song "They Killed Him."


=== Current impact within India ===
India, with its rapid economic modernisation and urbanisation, has rejected Gandhi's economics but accepted much of his politics and continues to revere his memory. Reporter Jim Yardley notes that, "modern India is hardly a Gandhian nation, if it ever was one. His vision of a village-dominated economy was shunted aside during his lifetime as rural romanticism, and his call for a national ethos of personal austerity and nonviolence has proved antithetical to the goals of an aspiring economic and military power." By contrast, Gandhi is "given full credit for India's political identity as a tolerant, secular democracy."Gandhi's birthday, 2 October, is a national holiday in India, Gandhi Jayanti. Gandhi's image also appears on paper currency of all denominations issued by Reserve Bank of India, except for the one rupee note. Gandhi's date of death, 30 January, is commemorated as a Martyrs' Day in India.There are three temples in India dedicated to Gandhi. One is located at Sambalpur in Odisha and the second at Nidaghatta village near Kadur in Chikmagalur district of Karnataka and the third one at Chityal in the district of Nalgonda, Telangana. The Gandhi Memorial in Kanyakumari resembles central Indian Hindu temples and the Tamukkam or Summer Palace in Madurai now houses the Mahatma Gandhi Museum.


=== Descendants ===
Gandhi's children and grandchildren live in India and other countries. Grandson Rajmohan Gandhi is a professor in Illinois and an author of Gandhi's biography titled Mohandas, while another, Tarun Gandhi, has authored several authoritative books on his grandfather. Another grandson, Kanu Ramdas Gandhi (the son of Gandhi's third son Ramdas), was found living in an old age home in Delhi despite having taught earlier in the United States.


'''

### hard

In [7]:
# record start time
start = time.time()

# We will generate all possible mcq from single page in json format
prompt = "Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text:"+" "+item+'''\n Give me output in this JSON array format:[{"question": string, "options":List[string], "answer":string}]'''
prompt_template= f'''[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> {prompt}[/INST]'''

# print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
# output = model.generate(inputs=input_ids, do_sample=True,max_length=32768)
# output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
# print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@",tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    do_sample=True,
    max_new_tokens=4096
)

# print("######",pipe(prompt_template)[0]['generated_text'])
print(pipe(prompt_template)[0]['generated_text'])


# record end time
end = time.time()

temp = end-start
print(temp)
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes

total_time_for_generation = "%dh:%dm:%ds" %(hours,minutes,seconds)

if hours == 0:
    print('The time of execution of downloading and loading the model : %dm:%ds' %(minutes,seconds))

else:
    print('The time of execution of downloading and loading the model : %dh:%dm:%ds' %(hours,minutes,seconds))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


*** Pipeline:
[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text: 
Mohandas Karamchand Gandhi (2 October 1869 – 30 January 1948) was an Indian lawyer, anti-colonial nationalist and political ethicist who employed nonviolent resistance to lead the successful campaign for India's independence from British rule. He inspired movements for civil rights and freedom across the world. The honorific Mahātmā (from Sanskrit 'great-souled, venerable'), first applied to him in South Africa in 1914, is now used throughout the world.
Born and raised in a Hindu family in coastal Gujarat, Gandhi trained in the law at the Inner Temple, London, and was called to the bar at age 22 in June 1891. After two uncertain years in India, where he was unable to start a successful law practice, he moved to South Africa in 1

### medium

In [8]:
# record start time
start = time.time()

# We will generate all possible mcq from single page in json format
prompt = "Generate as many as possible difficulty level medium Multiple choice questions with four options and answer using this text:"+" "+item+'''\n Give me output in this JSON array format:[{"question": string, "options":List[string], "answer":string}]'''
prompt_template= f'''[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> {prompt}[/INST]'''


input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
# output = model.generate(inputs=input_ids, do_sample=True,max_length=32768)
# output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
# print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@",tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    do_sample=True,
    max_new_tokens= 4096
)

print("######",pipe(prompt_template)[0]['generated_text'])
# record end time
end = time.time()

temp = end-start
print(temp)
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes

total_time_for_generation = "%dh:%dm:%ds" %(hours,minutes,seconds)

if hours == 0:
    print('The time of execution of downloading and loading the model : %dm:%ds' %(minutes,seconds))

else:
    print('The time of execution of downloading and loading the model : %dh:%dm:%ds' %(hours,minutes,seconds))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


*** Pipeline:
###### [INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> Generate as many as possible difficulty level medium Multiple choice questions with four options and answer using this text: 
Mohandas Karamchand Gandhi (2 October 1869 – 30 January 1948) was an Indian lawyer, anti-colonial nationalist and political ethicist who employed nonviolent resistance to lead the successful campaign for India's independence from British rule. He inspired movements for civil rights and freedom across the world. The honorific Mahātmā (from Sanskrit 'great-souled, venerable'), first applied to him in South Africa in 1914, is now used throughout the world.
Born and raised in a Hindu family in coastal Gujarat, Gandhi trained in the law at the Inner Temple, London, and was called to the bar at age 22 in June 1891. After two uncertain years in India, where he was unable to start a successful law practice, he moved to South Af

### easy

In [9]:
# record start time
start = time.time()

# We will generate all possible mcq from single page in json format
prompt = "Generate as many as possible difficulty level easy Multiple choice questions with four options and answer using this text:"+" "+item+'''\n Give me output in this JSON array format:[{"question": string, "options":List[string], "answer":string}]'''
prompt_template= f'''[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> {prompt}[/INST]'''


input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    do_sample=True,
    max_new_tokens=4096,
    temperature= 0.6,
    top_p= 0.95,
    top_k= 50,
    repetition_penalty= 1.2,
    return_full_text= False

)

# print("######",pipe(prompt_template)[0]['generated_text'][len(prompt):])
print(pipe(prompt_template)[0]['generated_text'])
# record end time
end = time.time()

temp = end-start
print(temp)
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes

total_time_for_generation = "%dh:%dm:%ds" %(hours,minutes,seconds)

if hours == 0:
    print('The time of execution of downloading and loading the model : %dm:%ds' %(minutes,seconds))

else:
    print('The time of execution of downloading and loading the model : %dh:%dm:%ds' %(hours,minutes,seconds))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


*** Pipeline:
 Here are some multiple choice questions about Mohandas Karamchand Gandhi in the requested JSON format:
```json
[
    {
        "question": "What is the honorific Mahātmā derived from?",
        "options": ["Sanskrit 'great-souled, venerable'", "Arabic 'great teacher, guide'", "Persian 'great warrior, hero'", "Urdu 'great scientist, scholar'"],
        "answer": "'great-souled, venerable'"
    },
    {
        "question": "Which of the following was NOT a part of Gandhi's childhood?",
        "options": ["Twisting dogs' ears", "Playing cricket", "Reading the Indian classics", "Being shy and tongue tied"],
        "answer": "Playing cricket"
    },
    {
        "question": "Where did Gandhi study law?",
        "options": ["University College, London", "Inner Temple in London", "Samaldas College in Bhavnagar State", "None of the above"],
        "answer": "University College, London and Inner Temple in London"
    },
    {
        "question": "What did Gandhi promote thro

In [10]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Wed Mar  6 06:55:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:3D:00.0 Off |                  Off |
| 30%   42C    P2              82W / 300W |  40865MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## ML-Text

In [11]:
item = '''
Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Recently, generative artificial neural networks have been able to surpass many previous approaches in performance.Machine learning approaches have been applied to many fields including large language models, computer vision, speech recognition, email filtering, agriculture, and medicine, where it is too costly to develop algorithms to perform the needed tasks. ML is known in its application across business problems under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.
The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis through unsupervised learning. From a theoretical point of view Probably approximately correct learning provides a framework for describing machine learning.


== History and relationships to other fields ==

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence. The synonym self-teaching computers was also used in this time period.Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes. In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells. Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data. Other researchers who have studied human cognitive systems contributed to the modern machine learning technologies as well, including logician Walter Pitts and Warren McCulloch, who proposed the early mathematical models of neural networks to come up with algorithms that mirror human thought processes.By the early 1960s an experimental "learning machine" with punched tape memory, called Cybertron, had been developed by Raytheon Company to analyze sonar signals, electrocardiograms, and speech patterns using rudimentary reinforcement learning. It was repetitively "trained" by a human operator/teacher to recognize patterns and equipped with a "goof" button to cause it to re-evaluate incorrect decisions. A representative book on research into machine learning during the 1960s was Nilsson's book on Learning Machines, dealing mostly with machine learning for pattern classification. Interest related to pattern recognition continued into the 1970s, as described by Duda and Hart in 1973. In 1981 a report was given on using teaching strategies so that a neural network learns to recognize 40 characters (26 letters, 10 digits, and 4 special symbols) from a computer terminal.Tom M. Mitchell provided a widely quoted, more formal definition of the algorithms studied in the machine learning field: "A computer program is said to learn from experience E with respect to some class of tasks T and performance measure P if its performance at tasks in T, as measured by P,  improves with experience E." This definition of the tasks in which machine learning is concerned offers a fundamentally operational definition rather than defining the field in cognitive terms. This follows Alan Turing's proposal in his paper "Computing Machinery and Intelligence", in which the question "Can machines think?" is replaced with the question "Can machines do what we (as thinking entities) can do?".Modern-day machine learning has two objectives, one is to classify data based on models which have been developed, the other purpose is to make predictions for future outcomes based on these models. A hypothetical algorithm specific to classifying data may use computer vision of moles coupled with supervised learning in order to train it to classify the cancerous moles. A machine learning algorithm for stock trading may inform the trader of future potential predictions.


=== Artificial intelligence ===
As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; these were mostly perceptrons and other models that were later found to be reinventions of the generalized linear models of statistics. Probabilistic reasoning was also employed, especially in automated medical diagnosis.: 488 However, an increasing emphasis on the logical, knowledge-based approach caused a rift between AI and machine learning. Probabilistic systems were plagued by theoretical and practical problems of data acquisition and representation.: 488  By 1980, expert systems had come to dominate AI, and statistics was out of favor. Work on symbolic/knowledge-based learning did continue within AI, leading to inductive logic programming, but the more statistical line of research was now outside the field of AI proper, in pattern recognition and information retrieval.: 708–710, 755  Neural networks research had been abandoned by AI and computer science around the same time. This line, too, was continued outside the AI/CS field, as "connectionism", by researchers from other disciplines including Hopfield, Rumelhart, and Hinton. Their main success came in the mid-1980s with the reinvention of backpropagation.: 25 Machine learning (ML), reorganized and recognized as its own field, started to flourish in the 1990s. The field changed its goal from achieving artificial intelligence to tackling solvable problems of a practical nature. It shifted focus away from the symbolic approaches it had inherited from AI, and toward methods and models borrowed from statistics, fuzzy logic, and probability theory.


=== Data mining ===
Machine learning and data mining often employ the same methods and overlap significantly, but while machine learning focuses on prediction, based on known properties learned from the training data, data mining focuses on the discovery of (previously) unknown properties in the data (this is the analysis step of knowledge discovery in databases). Data mining uses many machine learning methods, but with different goals; on the other hand, machine learning also employs data mining methods as "unsupervised learning" or as a preprocessing step to improve learner accuracy. Much of the confusion between these two research communities (which do often have separate conferences and separate journals, ECML PKDD being a major exception) comes from the basic assumptions they work with: in machine learning, performance is usually evaluated with respect to the ability to reproduce known knowledge, while in knowledge discovery and data mining (KDD) the key task is the discovery of previously unknown knowledge. Evaluated with respect to known knowledge, an uninformed (unsupervised) method will easily be outperformed by other supervised methods, while in a typical KDD task, supervised methods cannot be used due to the unavailability of training data.
Machine learning also has intimate ties to optimization: many learning problems are formulated as minimization of some loss function on a training set of examples. Loss functions express the discrepancy between the predictions of the model being trained and the actual problem instances (for example, in classification, one wants to assign a label to instances, and models are trained to correctly predict the pre-assigned labels of a set of examples).


=== Generalization ===
The difference between optimization and machine learning arises from the goal of generalization: while optimization algorithms can minimize the loss on a training set, machine learning is concerned with minimizing the loss on unseen samples. Characterizing the generalization of various learning algorithms is an active topic of current research, especially for deep learning algorithms.


=== Statistics ===
Machine learning and statistics are closely related fields in terms of methods, but distinct in their principal goal: statistics draws population inferences from a sample, while machine learning finds generalizable predictive patterns. According to Michael I. Jordan, the ideas of machine learning, from methodological principles to theoretical tools, have had a long pre-history in statistics. He also suggested the term data science as a placeholder to call the overall field.Conventional statistical analyses require the a priori selection of a model most suitable for the study data set. In addition, only significant or theoretically relevant variables based on previous experience are included for analysis. In contrast, machine learning is not built on a pre-structured model; rather, the data shape the model by detecting underlying patterns. The more variables (input) used to train the model, the more accurate the ultimate model will be.Leo Breiman distinguished two statistical modeling paradigms: data model and algorithmic model, wherein "algorithmic model" means more or less the machine learning algorithms like Random Forest.
Some statisticians have adopted methods from machine learning, leading to a combined field that they call statistical learning.


=== Statistical Physics ===
Analytical and computational techniques derived from deep-rooted physics of disordered systems can be extended to large-scale problems, including machine learning, e.g., to analyze the weight space of deep neural networks. Statistical physics is thus finding applications in the area of medical diagnostics.


== Theory ==

A core objective of a learner is to generalize from its experience. Generalization in this context is the ability of a learning machine to perform accurately on new, unseen examples/tasks after having experienced a learning data set. The training examples come from some generally unknown probability distribution (considered representative of the space of occurrences) and the learner has to build a general model about this space that enables it to produce sufficiently accurate predictions in new cases.
The computational analysis of machine learning algorithms and their performance is a branch of theoretical computer science known as computational learning theory via the Probably Approximately Correct Learning (PAC) model. Because training sets are finite and the future is uncertain, learning theory usually does not yield guarantees of the performance of algorithms. Instead, probabilistic bounds on the performance are quite common. The bias–variance decomposition is one way to quantify generalization error.
For the best performance in the context of generalization, the complexity of the hypothesis should match the complexity of the function underlying the data. If the hypothesis is less complex than the function, then the model has under fitted the data. If the complexity of the model is increased in response, then the training error decreases. But if the hypothesis is too complex, then the model is subject to overfitting and generalization will be poorer.In addition to performance bounds, learning theorists study the time complexity and feasibility of learning. In computational learning theory, a computation is considered feasible if it can be done in polynomial time. There are two kinds of time complexity results: Positive results show that a certain class of functions can be learned in polynomial time. Negative results show that certain classes cannot be learned in polynomial time.


== Approaches ==

Machine learning approaches are traditionally divided into three broad categories, which correspond to learning paradigms, depending on the nature of the "signal" or "feedback" available to the learning system:

Supervised learning: The computer is presented with example inputs and their desired outputs, given by a "teacher", and the goal is to learn a general rule that maps inputs to outputs.
Unsupervised learning: No labels are given to the learning algorithm, leaving it on its own to find structure in its input. Unsupervised learning can be a goal in itself (discovering hidden patterns in data) or a means towards an end (feature learning).
Reinforcement learning: A computer program interacts with a dynamic environment in which it must perform a certain goal (such as driving a vehicle or playing a game against an opponent). As it navigates its problem space, the program is provided feedback that's analogous to rewards, which it tries to maximize.Although each algorithm has advantages and limitations, no single algorithm works for all problems.


=== Supervised learning ===

Supervised learning algorithms build a mathematical model of a set of data that contains both the inputs and the desired outputs. The data is known as training data, and consists of a set of training examples. Each training example has one or more inputs and the desired output, also known as a supervisory signal.  In the mathematical model, each training example is represented by an array or vector, sometimes called a feature vector, and the training data is represented by a matrix. Through iterative optimization of an objective function, supervised learning algorithms learn a function that can be used to predict the output associated with new inputs. An optimal function allows the algorithm to correctly determine the output for inputs that were not a part of the training data. An algorithm that improves the accuracy of its outputs or predictions over time is said to have learned to perform that task.Types of supervised-learning algorithms include active learning, classification and regression. Classification algorithms are used when the outputs are restricted to a limited set of values, and regression algorithms are used when the outputs may have any numerical value within a range. As an example, for a classification algorithm that filters emails, the input would be an incoming email, and the output would be the name of the folder in which to file the email.
Similarity learning is an area of supervised machine learning closely related to regression and classification, but the goal is to learn from examples using a similarity function that measures how similar or related two objects are. It has applications in ranking, recommendation systems, visual identity tracking, face verification, and speaker verification.


=== Unsupervised learning ===

Unsupervised learning algorithms find structures in data that has not been labeled, classified or categorized. Instead of responding to feedback, unsupervised learning algorithms identify commonalities in the data and react based on the presence or absence of such commonalities in each new piece of data. Central applications of unsupervised machine learning include clustering, dimensionality reduction, and density estimation. Unsupervised learning algorithms also streamlined the process of identifying large indel based haplotypes of a gene of interest from pan-genome.
Cluster analysis is the assignment of a set of observations into subsets (called clusters) so that observations within the same cluster are similar according to one or more predesignated criteria, while observations drawn from different clusters are dissimilar. Different clustering techniques make different assumptions on the structure of the data, often defined by some similarity metric and evaluated, for example, by internal compactness, or the similarity between members of the same cluster, and separation, the difference between clusters. Other methods are based on estimated density and graph connectivity.


=== Semi-supervised learning ===

Semi-supervised learning falls between unsupervised learning (without any labeled training data) and supervised learning (with completely labeled training data). Some of the training examples are missing training labels, yet many machine-learning researchers have found that unlabeled data, when used in conjunction with a small amount of labeled data, can produce a considerable improvement in learning accuracy.
In weakly supervised learning, the training labels are noisy, limited, or imprecise; however, these labels are often cheaper to obtain, resulting in larger effective training sets.


=== Reinforcement learning ===

Reinforcement learning is an area of machine learning concerned with how software agents ought to take actions in an environment so as to maximize some notion of cumulative reward. Due to its generality, the field is studied in many other disciplines, such as game theory, control theory, operations research, information theory, simulation-based optimization, multi-agent systems, swarm intelligence, statistics and genetic algorithms. In reinforcement learning, the environment is typically represented as a Markov decision process (MDP). Many reinforcements learning algorithms use dynamic programming techniques. Reinforcement learning algorithms do not assume knowledge of an exact mathematical model of the MDP and are used when exact models are infeasible. Reinforcement learning algorithms are used in autonomous vehicles or in learning to play a game against a human opponent.


=== Dimensionality reduction ===
Dimensionality reduction is a process of reducing the number of random variables under consideration by obtaining a set of principal variables. In other words, it is a process of reducing the dimension of the feature set, also called the "number of features". Most of the dimensionality reduction techniques can be considered as either feature elimination or extraction. One of the popular methods of dimensionality reduction is principal component analysis (PCA). PCA involves changing higher-dimensional data (e.g., 3D) to a smaller space (e.g., 2D). This results in a smaller dimension of data (2D instead of 3D), while keeping all original variables in the model without changing the data.
The manifold hypothesis proposes that high-dimensional data sets lie along low-dimensional manifolds, and many dimensionality reduction techniques make this assumption, leading to the area of manifold learning and manifold regularization.


=== Other types ===
Other approaches have been developed which do not fit neatly into this three-fold categorization, and sometimes more than one is used by the same machine learning system. For example, topic modeling, meta-learning.


==== Self-learning ====
Self-learning, as a machine learning paradigm was introduced in 1982 along with a neural network capable of self-learning, named crossbar adaptive array (CAA). It is learning with no external rewards and no external teacher advice. The CAA self-learning algorithm computes, in a crossbar fashion, both decisions about actions and emotions (feelings) about consequence situations. The system is driven by the interaction between cognition and emotion.
The self-learning algorithm updates a memory matrix W =||w(a,s)|| such that in each iteration executes the following machine learning routine:

in situation s perform action a
receive a consequence situation s'
compute emotion of being in the consequence situation v(s')
update crossbar memory  w'(a,s) = w(a,s) + v(s')It is a system with only one input, situation, and only one output, action (or behavior) a. There is neither a separate reinforcement input nor an advice input from the environment. The backpropagated value (secondary reinforcement) is the emotion toward the consequence situation. The CAA exists in two environments, one is the behavioral environment where it behaves, and the other is the genetic environment, wherefrom it initially and only once receives initial emotions about situations to be encountered in the behavioral environment. After receiving the genome (species) vector from the genetic environment, the CAA learns a goal-seeking behavior, in an environment that contains both desirable and undesirable situations.


==== Feature learning ====

Several learning algorithms aim at discovering better representations of the inputs provided during training. Classic examples include principal component analysis and cluster analysis. Feature learning algorithms, also called representation learning algorithms, often attempt to preserve the information in their input but also transform it in a way that makes it useful, often as a pre-processing step before performing classification or predictions. This technique allows reconstruction of the inputs coming from the unknown data-generating distribution, while not being necessarily faithful to configurations that are implausible under that distribution. This replaces manual feature engineering, and allows a machine to both learn the features and use them to perform a specific task.
Feature learning can be either supervised or unsupervised. In supervised feature learning, features are learned using labeled input data. Examples include artificial neural networks, multilayer perceptrons, and supervised dictionary learning. In unsupervised feature learning, features are learned with unlabeled input data.  Examples include dictionary learning, independent component analysis, autoencoders, matrix factorization and various forms of clustering.Manifold learning algorithms attempt to do so under the constraint that the learned representation is low-dimensional. Sparse coding algorithms attempt to do so under the constraint that the learned representation is sparse, meaning that the mathematical model has many zeros. Multilinear subspace learning algorithms aim to learn low-dimensional representations directly from tensor representations for multidimensional data, without reshaping them into higher-dimensional vectors. Deep learning algorithms discover multiple levels of representation, or a hierarchy of features, with higher-level, more abstract features defined in terms of (or generating) lower-level features. It has been argued that an intelligent machine is one that learns a representation that disentangles the underlying factors of variation that explain the observed data.Feature learning is motivated by the fact that machine learning tasks such as classification often require input that is mathematically and computationally convenient to process. However, real-world data such as images, video, and sensory data has not yielded attempts to algorithmically define specific features. An alternative is to discover such features or representations through examination, without relying on explicit algorithms.


==== Sparse dictionary learning ====

Sparse dictionary learning is a feature learning method where a training example is represented as a linear combination of basis functions, and is assumed to be a sparse matrix. The method is strongly NP-hard and difficult to solve approximately. A popular heuristic method for sparse dictionary learning is the K-SVD algorithm. Sparse dictionary learning has been applied in several contexts. In classification, the problem is to determine the class to which a previously unseen training example belongs. For a dictionary where each class has already been built, a new training example is associated with the class that is best sparsely represented by the corresponding dictionary. Sparse dictionary learning has also been applied in image de-noising. The key idea is that a clean image patch can be sparsely represented by an image dictionary, but the noise cannot.


==== Anomaly detection ====

In data mining, anomaly detection, also known as outlier detection, is the identification of rare items, events or observations which raise suspicions by differing significantly from the majority of the data. Typically, the anomalous items represent an issue such as bank fraud, a structural defect, medical problems or errors in a text. Anomalies are referred to as outliers, novelties, noise, deviations and exceptions.In particular, in the context of abuse and network intrusion detection, the interesting objects are often not rare objects, but unexpected bursts of inactivity. This pattern does not adhere to the common statistical definition of an outlier as a rare object. Many outlier detection methods (in particular, unsupervised algorithms) will fail on such data unless aggregated appropriately. Instead, a cluster analysis algorithm may be able to detect the micro-clusters formed by these patterns.Three broad categories of anomaly detection techniques exist. Unsupervised anomaly detection techniques detect anomalies in an unlabeled test data set under the assumption that the majority of the instances in the data set are normal, by looking for instances that seem to fit the least to the remainder of the data set. Supervised anomaly detection techniques require a data set that has been labeled as "normal" and "abnormal" and involves training a classifier (the key difference to many other statistical classification problems is the inherently unbalanced nature of outlier detection). Semi-supervised anomaly detection techniques construct a model representing normal behavior from a given normal training data set and then test the likelihood of a test instance to be generated by the model.


==== Robot learning ====
Robot learning is inspired by a multitude of machine learning methods, starting from supervised learning, reinforcement learning, and finally meta-learning (e.g. MAML).


==== Association rules ====

Association rule learning is a rule-based machine learning method for discovering relationships between variables in large databases. It is intended to identify strong rules discovered in databases using some measure of "interestingness".Rule-based machine learning is a general term for any machine learning method that identifies, learns, or evolves "rules" to store, manipulate or apply knowledge. The defining characteristic of a rule-based machine learning algorithm is the identification and utilization of a set of relational rules that collectively represent the knowledge captured by the system. This is in contrast to other machine learning algorithms that commonly identify a singular model that can be universally applied to any instance in order to make a prediction. Rule-based machine learning approaches include learning classifier systems, association rule learning, and artificial immune systems.
Based on the concept of strong rules, Rakesh Agrawal, Tomasz Imieliński and Arun Swami introduced association rules for discovering regularities between products in large-scale transaction data recorded by point-of-sale (POS) systems in supermarkets. For example, the rule

{onions,potatoes} => {burger}
{\displaystyle \{\mathrm {onions,potatoes} \}\Rightarrow \{\mathrm {burger} \}}
found in the sales data of a supermarket would indicate that if a customer buys onions and potatoes together, they are likely to also buy hamburger meat. Such information can be used as the basis for decisions about marketing activities such as promotional pricing or product placements. In addition to market basket analysis, association rules are employed today in application areas including Web usage mining, intrusion detection, continuous production, and bioinformatics. In contrast with sequence mining, association rule learning typically does not consider the order of items either within a transaction or across transactions.
Learning classifier systems (LCS) are a family of rule-based machine learning algorithms that combine a discovery component, typically a genetic algorithm, with a learning component, performing either supervised learning, reinforcement learning, or unsupervised learning. They seek to identify a set of context-dependent rules that collectively store and apply knowledge in a piecewise manner in order to make predictions.Inductive logic programming (ILP) is an approach to rule learning using logic programming as a uniform representation for input examples, background knowledge, and hypotheses. Given an encoding of the known background knowledge and a set of examples represented as a logical database of facts, an ILP system will derive a hypothesized logic program that entails all positive and no negative examples. Inductive programming is a related field that considers any kind of programming language for representing hypotheses (and not only logic programming), such as functional programs.
Inductive logic programming is particularly useful in bioinformatics and natural language processing. Gordon Plotkin and Ehud Shapiro laid the initial theoretical foundation for inductive machine learning in a logical setting. Shapiro built their first implementation (Model Inference System) in 1981: a Prolog program that inductively inferred logic programs from positive and negative examples. The term inductive here refers to philosophical induction, suggesting a theory to explain observed facts, rather than mathematical induction, proving a property for all members of a well-ordered set.


=== Models ===
Performing machine learning can involve creating a model, which is trained on some training data and then can process additional data to make predictions. Various types of models have been used and researched for machine learning systems.


==== Artificial neural networks ====

Artificial neural networks (ANNs), or connectionist systems, are computing systems vaguely inspired by the biological neural networks that constitute animal brains. Such systems "learn" to perform tasks by considering examples, generally without being programmed with any task-specific rules.
An ANN is a model based on a collection of connected units or nodes called "artificial neurons", which loosely model the neurons in a biological brain. Each connection, like the synapses in a biological brain, can transmit information, a "signal", from one artificial neuron to another. An artificial neuron that receives a signal can process it and then signal additional artificial neurons connected to it. In common ANN implementations, the signal at a connection between artificial neurons is a real number, and the output of each artificial neuron is computed by some non-linear function of the sum of its inputs. The connections between artificial neurons are called "edges". Artificial neurons and edges typically have a weight that adjusts as learning proceeds. The weight increases or decreases the strength of the signal at a connection. Artificial neurons may have a threshold such that the signal is only sent if the aggregate signal crosses that threshold. Typically, artificial neurons are aggregated into layers. Different layers may perform different kinds of transformations on their inputs. Signals travel from the first layer (the input layer) to the last layer (the output layer), possibly after traversing the layers multiple times.
The original goal of the ANN approach was to solve problems in the same way that a human brain would. However, over time, attention moved to performing specific tasks, leading to deviations from biology. Artificial neural networks have been used on a variety of tasks, including computer vision, speech recognition, machine translation, social network filtering, playing board and video games and medical diagnosis.
Deep learning consists of multiple hidden layers in an artificial neural network. This approach tries to model the way the human brain processes light and sound into vision and hearing. Some successful applications of deep learning are computer vision and speech recognition.


==== Decision trees ====

Decision tree learning uses a decision tree as a predictive model to go from observations about an item (represented in the branches) to conclusions about the item's target value (represented in the leaves). It is one of the predictive modeling approaches used in statistics, data mining, and machine learning. Tree models where the target variable can take a discrete set of values are called classification trees; in these tree structures, leaves represent class labels, and branches represent conjunctions of features that lead to those class labels. Decision trees where the target variable can take continuous values (typically real numbers) are called regression trees. In decision analysis, a decision tree can be used to visually and explicitly represent decisions and decision making. In data mining, a decision tree describes data, but the resulting classification tree can be an input for decision-making.


==== Support-vector machines ====

Support-vector machines (SVMs), also known as support-vector networks, are a set of related supervised learning methods used for classification and regression. Given a set of training examples, each marked as belonging to one of two categories, an SVM training algorithm builds a model that predicts whether a new example falls into one category.  An SVM training algorithm is a non-probabilistic, binary, linear classifier, although methods such as Platt scaling exist to use SVM in a probabilistic classification setting. In addition to performing linear classification, SVMs can efficiently perform a non-linear classification using what is called the kernel trick, implicitly mapping their inputs into high-dimensional feature spaces.


==== Regression analysis ====

Regression analysis encompasses a large variety of statistical methods to estimate the relationship between input variables and their associated features. Its most common form is linear regression, where a single line is drawn to best fit the given data according to a mathematical criterion such as ordinary least squares. The latter is often extended by regularization methods to mitigate overfitting and bias, as in ridge regression. When dealing with non-linear problems, go-to models include polynomial regression (for example, used for trendline fitting in Microsoft Excel), logistic regression (often used in statistical classification) or even kernel regression, which introduces non-linearity by taking advantage of the kernel trick to implicitly map input variables to higher-dimensional space.


==== Bayesian networks ====

A Bayesian network, belief network, or directed acyclic graphical model is a probabilistic graphical model that represents a set of random variables and their conditional independence with a directed acyclic graph (DAG). For example, a Bayesian network could represent the probabilistic relationships between diseases and symptoms. Given symptoms, the network can be used to compute the probabilities of the presence of various diseases. Efficient algorithms exist that perform inference and learning. Bayesian networks that model sequences of variables, like speech signals or protein sequences, are called dynamic Bayesian networks. Generalizations of Bayesian networks that can represent and solve decision problems under uncertainty are called influence diagrams.


==== Gaussian processes ====

A Gaussian process is a stochastic process in which every finite collection of the random variables in the process has a multivariate normal distribution, and it relies on a pre-defined covariance function, or kernel, that models how pairs of points relate to each other depending on their locations.
Given a set of observed points, or input–output examples, the distribution of the (unobserved) output of a new point as function of its input data can be directly computed by looking like the observed points and the covariances between those points and the new, unobserved point.
Gaussian processes are popular surrogate models in Bayesian optimization used to do hyperparameter optimization.


==== Genetic algorithms ====

A genetic algorithm (GA) is a search algorithm and heuristic technique that mimics the process of natural selection, using methods such as mutation and crossover to generate new genotypes in the hope of finding good solutions to a given problem. In machine learning, genetic algorithms were used in the 1980s and 1990s. Conversely, machine learning techniques have been used to improve the performance of genetic and evolutionary algorithms.


==== Belief functions ====

The theory of belief functions, also referred to as evidence theory or Dempster–Shafer theory, is a general framework for reasoning with uncertainty, with understood connections to other frameworks such as probability, possibility and  imprecise probability theories. These theoretical frameworks can be thought of as a kind of learner and have some analogous properties of how evidence is combined (e.g.,  Dempster's rule of combination), just like how in a pmf-based Bayesian approach would combine probabilities. However, there are many caveats to these beliefs functions when compared to Bayesian approaches in order to incorporate ignorance and Uncertainty quantification. These belief function approaches that are implemented within the machine learning domain typically leverage a fusion approach of various ensemble methods to better handle the learner's decision boundary, low samples, and ambiguous class issues that standard machine learning approach tend to have difficulty resolving. However, the computational complexity of these algorithms are dependent on the number of propositions (classes), and can lead a much higher computation time when compared to other machine learning approaches.


=== Training models ===
Typically, machine learning models require a high quantity of reliable data in order for the models to perform accurate predictions. When training a machine learning model, machine learning engineers need to target and collect a large and representative sample of data. Data from the training set can be as varied as a corpus of text, a collection of images, sensor data, and data collected from individual users of a service. Overfitting is something to watch out for when training a machine learning model. Trained models derived from biased or non-evaluated data can result in skewed or undesired predictions. Bias models may result in detrimental outcomes thereby furthering the negative impacts on society or objectives. Algorithmic bias is a potential result of data not being fully prepared for training. Machine learning ethics is becoming a field of study and notably be integrated within machine learning engineering teams.


==== Federated learning ====

Federated learning is an adapted form of distributed artificial intelligence to training machine learning models that decentralizes the training process, allowing for users' privacy to be maintained by not needing to send their data to a centralized server. This also increases efficiency by decentralizing the training process to many devices. For example, Gboard uses federated machine learning to train search query prediction models on users' mobile phones without having to send individual searches back to Google.


== Applications ==
There are many applications for machine learning, including:

In 2006, the media-services provider Netflix held the first "Netflix Prize" competition to find a program to better predict user preferences and improve the accuracy of its existing Cinematch movie recommendation algorithm by at least 10%. A joint team made up of researchers from AT&T Labs-Research in collaboration with the teams Big Chaos and Pragmatic Theory built an ensemble model to win the Grand Prize in 2009 for $1 million. Shortly after the prize was awarded, Netflix realized that viewers' ratings were not the best indicators of their viewing patterns ("everything is a recommendation") and they changed their recommendation engine accordingly. In 2010 The Wall Street Journal wrote about the firm Rebellion Research and their use of machine learning to predict the financial crisis. In 2012, co-founder of Sun Microsystems, Vinod Khosla, predicted that 80% of medical doctors jobs would be lost in the next two decades to automated machine learning medical diagnostic software. In 2014, it was reported that a machine learning algorithm had been applied in the field of art history to study fine art paintings and that it may have revealed previously unrecognized influences among artists. In 2019 Springer Nature published the first research book created using machine learning. In 2020, machine learning technology was used to help make diagnoses and aid researchers in developing a cure for COVID-19. Machine learning was recently applied to predict the pro-environmental behavior of travelers. Recently, machine learning technology was also applied to optimize smartphone's performance and thermal behavior based on the user's interaction with the phone. When applied correctly, machine learning algorithms (MLAs) can utilize a wide range of company characteristics to predict stock returns without overfitting. By employing effective feature engineering and combining forecasts, MLAs can generate results that far surpass those obtained from basic linear techniques like OLS.


== Limitations ==
Although machine learning has been transformative in some fields, machine-learning programs often fail to deliver expected results. Reasons for this are numerous: lack of (suitable) data, lack of access to the data, data bias, privacy problems, badly chosen tasks and algorithms, wrong tools and people, lack of resources, and evaluation problems.The "black box theory" poses another yet significant challenge. Black box refers to a situation where the algorithm or the process of producing an output is entirely opaque, meaning that even the coders of the algorithm cannot audit the pattern that the machine extracted out of the data. The House of Lords Select Committee, which claimed that such an “intelligence system” that could have a “substantial impact on an individual’s life” would not be considered acceptable unless it provided “a full and satisfactory explanation for the decisions” it makes.In 2018, a self-driving car from Uber failed to detect a pedestrian, who was killed after a collision. Attempts to use machine learning in healthcare with the IBM Watson system failed to deliver even after years of time and billions of dollars invested. Microsoft's chatbot has been reported to produce hostile and offensive response against its users.Machine learning has been used as a strategy to update the evidence related to a systematic review and increased reviewer burden related to the growth of biomedical literature. While it has improved with training sets, it has not yet developed sufficiently to reduce the workload burden without limiting the necessary sensitivity for the findings research themselves.


=== Bias ===
Machine learning approaches in particular can suffer from different data biases. A machine learning system trained specifically on current customers may not be able to predict the needs of new customer groups that are not represented in the training data. When trained on human-made data, machine learning is likely to pick up the constitutional and unconscious biases already present in society.Language models learned from data have been shown to contain human-like biases. In an experiment carried out by ProPublica, an investigative journalism organization, a machine learning algorithm’s insight towards the recidivism rates among prisoners falsely flagged “black defendants high risk twice as often as white defendants.” In 2015, Google photos would often tag black people as gorillas, and in 2018 this still was not well resolved, but Google reportedly was still using the workaround to remove all gorillas from the training data, and thus was not able to recognize real gorillas at all. Similar issues with recognizing non-white people have been found in many other systems. In 2016, Microsoft tested a chatbot that learned from Twitter, and it quickly picked up racist and sexist language.Because of such challenges, the effective use of machine learning may take longer to be adopted in other domains. Concern for fairness in machine learning, that is, reducing bias in machine learning and propelling its use for human good is increasingly expressed by artificial intelligence scientists, including Fei-Fei Li, who reminds engineers that "There's nothing artificial about AI...It's inspired by people, it's created by people, and—most importantly—it impacts people. It is a powerful tool we are only just beginning to understand, and that is a profound responsibility."


=== Explainability ===

Explainable AI (XAI), or Interpretable AI, or Explainable Machine Learning (XML), is artificial intelligence (AI) in which humans can understand the decisions or predictions made by the AI. It contrasts with the "black box" concept in machine learning where even its designers cannot explain why an AI arrived at a specific decision. By refining the mental models of users of AI-powered systems and dismantling their misconceptions, XAI promises to help users perform more effectively. XAI may be an implementation of the social right to explanation.


=== Overfitting ===

Settling on a bad, overly complex theory gerrymandered to fit all the past training data is known as overfitting. Many systems attempt to reduce overfitting by rewarding a theory in accordance with how well it fits the data but penalizing the theory in accordance with how complex the theory is.


=== Other limitations and vulnerabilities ===
Learners can also disappoint by "learning the wrong lesson". A toy example is that an image classifier trained only on pictures of brown horses and black cats might conclude that all brown patches are likely to be horses. A real-world example is that, unlike humans, current image classifiers often do not primarily make judgments from the spatial relationship between components of the picture, and they learn relationships between pixels that humans are oblivious to, but that still correlate with images of certain types of real objects. Modifying these patterns on a legitimate image can result in "adversarial" images that the system misclassifies.Adversarial vulnerabilities can also result in nonlinear systems, or from non-pattern perturbations. For some systems, it is possible to change the output by only changing a single adversarially chosen pixel. Machine learning models are often vulnerable to manipulation and/or evasion via adversarial machine learning.Researchers have demonstrated how backdoors can be placed undetectably into classifying (e.g., for categories "spam" and well-visible "not spam" of posts) machine learning models which are often developed and/or trained by third parties. Parties can change the classification of any input, including in cases for which a type of data/software transparency is provided, possibly including white-box access.


== Model assessments ==
Classification of machine learning models can be validated by accuracy estimation techniques like the holdout method, which splits the data in a training and test set (conventionally 2/3 training set and 1/3 test set designation) and evaluates the performance of the training model on the test set. In comparison, the K-fold-cross-validation method randomly partitions the data into K subsets and then K experiments are performed each respectively considering 1 subset for evaluation and the remaining K-1 subsets for training the model. In addition to the holdout and cross-validation methods, bootstrap, which samples n instances with replacement from the dataset, can be used to assess model accuracy.In addition to overall accuracy, investigators frequently report sensitivity and specificity meaning True Positive Rate (TPR) and True Negative Rate (TNR) respectively. Similarly, investigators sometimes report the false positive rate (FPR) as well as the false negative rate (FNR). However, these rates are ratios that fail to reveal their numerators and denominators. The total operating characteristic (TOC) is an effective method to express a model's diagnostic ability. TOC shows the numerators and denominators of the previously mentioned rates, thus TOC provides more information than the commonly used receiver operating characteristic (ROC) and ROC's associated area under the curve (AUC).


== Ethics ==

Machine learning poses a host of ethical questions. Systems that are trained on datasets collected with biases may exhibit these biases upon use (algorithmic bias), thus digitizing cultural prejudices. For example, in 1988, the UK's Commission for Racial Equality found that St. George's Medical School had been using a computer program trained from data of previous admissions staff and this program had denied nearly 60 candidates who were found to be either women or had non-European sounding names. Using job hiring data from a firm with racist hiring policies may lead to a machine learning system duplicating the bias by scoring job applicants by similarity to previous successful applicants. Another example includes predictive policing company Geolitica's predictive algorithm that resulted in “disproportionately high levels of over-policing in low-income and minority communities” after being trained with historical crime data.While responsible collection of data and documentation of algorithmic rules used by a system is considered a critical part of machine learning, some researchers blame lack of participation and representation of minority population in the field of AI for machine learning's vulnerability to biases. In fact, according to research carried out by the Computing Research Association (CRA) in 2021, “female faculty merely make up 16.1%” of all faculty members who focus on AI among several universities around the world. Furthermore, among the group of “new U.S. resident AI PhD graduates,” 45% identified as white, 22.4% as Asian, 3.2% as Hispanic, and 2.4% as African American, which further demonstrates a lack of diversity in the field of AI.AI can be well-equipped to make decisions in technical fields, which rely heavily on data and historical information. These decisions rely on objectivity and logical reasoning. Because human languages contain biases, machines trained on language corpora will necessarily also learn these biases.Other forms of ethical challenges, not related to personal biases, are seen in health care. There are concerns among health care professionals that these systems might not be designed in the public's interest but as income-generating machines. This is especially true in the United States where there is a long-standing ethical dilemma of improving health care, but also increasing profits. For example, the algorithms could be designed to provide patients with unnecessary tests or medication in which the algorithm's proprietary owners hold stakes. There is potential for machine learning in health care to provide professionals an additional tool to diagnose, medicate, and plan recovery paths for patients, but this requires these biases to be mitigated.


== Hardware ==
Since the 2010s, advances in both machine learning algorithms and computer hardware have led to more efficient methods for training deep neural networks (a particular narrow subdomain of machine learning) that contain many layers of non-linear hidden units. By 2019, graphic processing units (GPUs), often with AI-specific enhancements, had displaced CPUs as the dominant method of training large-scale commercial cloud AI. OpenAI estimated the hardware computing used in the largest deep learning projects from AlexNet (2012) to AlphaZero (2017), and found a 300,000-fold increase in the amount of compute required, with a doubling-time trendline of 3.4 months.


=== Neuromorphic/Physical Neural Networks ===
A physical neural network or Neuromorphic computer  is a type of artificial neural network in which an electrically adjustable material is used to emulate the function of a neural synapse. "Physical" neural network is used to emphasize the reliance on physical hardware used to emulate neurons as opposed to software-based approaches. More generally the term is applicable to other artificial neural networks in which a memristor or other electrically adjustable resistance material is used to emulate a neural synapse.


=== Embedded Machine Learning ===
Embedded Machine Learning is a sub-field of machine learning, where the machine learning model is run on embedded systems with limited computing resources such as wearable computers, edge devices and microcontrollers. Running machine learning model in embedded devices removes the need for transferring and storing data on cloud servers for further processing, henceforth, reducing data breaches and privacy leaks happening because of transferring data, and also minimizes theft of intellectual properties, personal data and business secrets. Embedded Machine Learning could be applied through several techniques including hardware acceleration, using approximate computing, optimization of machine learning models and many more.


'''

## code

### hard1

In [12]:
# record start time
start = time.time()

# We will generate all possible mcq from single page in json format
prompt = "Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text:"+" "+item+'''\n Give me output in this JSON array format:[{"question": string, "options":List[string], "answer":string}]'''
prompt_template= f'''[INST] <<SYS>>You are a Ph.D. field expert in Machine learning and best in generating Multiple choice questions with demanded JSON format. <</SYS>> {prompt}[/INST]'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, do_sample=True,max_new_tokens= 4096)
print(tokenizer.decode(output[0]))
# record end time
end = time.time()

temp = end-start
print(temp)
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes

total_time_for_generation = "%dh:%dm:%ds" %(hours,minutes,seconds)

if hours == 0:
    print('The time of execution of downloading and loading the model : %dm:%ds' %(minutes,seconds))

else:
    print('The time of execution of downloading and loading the model : %dh:%dm:%ds' %(hours,minutes,seconds))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




*** Generate:
<s> [INST] <<SYS>>You are a Ph.D. field expert in Machine learning and best in generating Multiple choice questions with demanded JSON format. <</SYS>> Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text: 
Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Recently, generative artificial neural networks have been able to surpass many previous approaches in performance.Machine learning approaches have been applied to many fields including large language models, computer vision, speech recognition, email filtering, agriculture, and medicine, where it is too costly to develop algorithms to perform the needed tasks. ML is known in its application across business problems under the name predictive analytics. Althou

In [13]:
# record start time
start = time.time()

# We will generate all possible mcq from single page in json format
prompt = "Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text:"+" "+item+'''\n Give me output in this JSON array format:[{"question": string, "options":List[string], "answer":string}]'''
prompt_template= f'''[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> {prompt}[/INST]'''


input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
# output = model.generate(inputs=input_ids, do_sample=True,max_length=32768)
# output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
# print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@",tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=4096,
    do_sample=True,
)

# print("######",pipe(prompt_template)[0]['generated_text'])
print(pipe(prompt_template)[0]['generated_text'])
# record end time
end = time.time()

temp = end-start
print(temp)
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes

total_time_for_generation = "%dh:%dm:%ds" %(hours,minutes,seconds)

if hours == 0:
    print('The time of execution of downloading and loading the model : %dm:%ds' %(minutes,seconds))

else:
    print('The time of execution of downloading and loading the model : %dh:%dm:%ds' %(hours,minutes,seconds))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


*** Pipeline:
[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text: 
Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Recently, generative artificial neural networks have been able to surpass many previous approaches in performance.Machine learning approaches have been applied to many fields including large language models, computer vision, speech recognition, email filtering, agriculture, and medicine, where it is too costly to develop algorithms to perform the needed tasks. ML is known in its application across business problems under the name predictive analytics. Although not all machine learning is

### medium

In [14]:
# record start time
start = time.time()

# We will generate all possible mcq from single page in json format
prompt = "Generate as many as possible difficulty level medium Multiple choice questions with four options and answer using this text:"+" "+item+'''\n Give me output in this JSON array format:[{"question": string, "options":List[string], "answer":string}]'''
prompt_template= f'''[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> {prompt}[/INST]'''


input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
# output = model.generate(inputs=input_ids, do_sample=True,max_length=32768)
# output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
# print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@",tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=4096,
    do_sample=True,
)


print(pipe(prompt_template)[0]['generated_text'])
# record end time
end = time.time()

temp = end-start
print(temp)
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes

total_time_for_generation = "%dh:%dm:%ds" %(hours,minutes,seconds)

if hours == 0:
    print('The time of execution of downloading and loading the model : %dm:%ds' %(minutes,seconds))

else:
    print('The time of execution of downloading and loading the model : %dh:%dm:%ds' %(hours,minutes,seconds))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


*** Pipeline:
[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> Generate as many as possible difficulty level medium Multiple choice questions with four options and answer using this text: 
Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Recently, generative artificial neural networks have been able to surpass many previous approaches in performance.Machine learning approaches have been applied to many fields including large language models, computer vision, speech recognition, email filtering, agriculture, and medicine, where it is too costly to develop algorithms to perform the needed tasks. ML is known in its application across business problems under the name predictive analytics. Although not all machine learning 

### easy

In [15]:
# record start time
start = time.time()

# We will generate all possible mcq from single page in json format
prompt = "Generate as many as possible difficulty level easy Multiple choice questions with four options and answer using this text:"+" "+item+'''\n Give me output in this JSON array format:[{"question": string, "options":List[string], "answer":string}]'''
prompt_template= f'''[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> {prompt}[/INST]'''


input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
# output = model.generate(inputs=input_ids, do_sample=True,max_length=32768)
# output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
# print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@",tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=4096,
    do_sample=True,
)


print(pipe(prompt_template)[0]['generated_text'])
# record end time
end = time.time()

temp = end-start
print(temp)
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes

total_time_for_generation = "%dh:%dm:%ds" %(hours,minutes,seconds)

if hours == 0:
    print('The time of execution of downloading and loading the model : %dm:%ds' %(minutes,seconds))

else:
    print('The time of execution of downloading and loading the model : %dh:%dm:%ds' %(hours,minutes,seconds))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


*** Pipeline:
[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> Generate as many as possible difficulty level easy Multiple choice questions with four options and answer using this text: 
Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Recently, generative artificial neural networks have been able to surpass many previous approaches in performance.Machine learning approaches have been applied to many fields including large language models, computer vision, speech recognition, email filtering, agriculture, and medicine, where it is too costly to develop algorithms to perform the needed tasks. ML is known in its application across business problems under the name predictive analytics. Although not all machine learning is

In [16]:
# del pipe
# del model

# import gc
# import torch

# def flush():
#   gc.collect()
#   torch.cuda.empty_cache()
#   torch.cuda.reset_peak_memory_stats()

In [17]:
# flush()

In [18]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Wed Mar  6 07:04:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:3D:00.0 Off |                  Off |
| 30%   41C    P2              82W / 300W |  40865MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [19]:
# print("######",pipe(prompt_template)[0]['generated_text'][len(prompt)])


In [20]:
# record start time
start = time.time()

# We will generate all possible mcq from single page in json format
prompt = "Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text:"+" "+item+'''\n Give me output in this JSON array format:[{"question": string, "options":List[string], "answer":string}]'''
prompt_template= f'''[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> {prompt}[/INST]'''


input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
# output = model.generate(inputs=input_ids, do_sample=True,max_length=32768)
# output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
# print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@",tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=8192,
    do_sample=True,
)


print(pipe(prompt_template)[0]['generated_text'])
# record end time
end = time.time()

temp = end-start
print(temp)
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes

total_time_for_generation = "%dh:%dm:%ds" %(hours,minutes,seconds)

if hours == 0:
    print('The time of execution of downloading and loading the model : %dm:%ds' %(minutes,seconds))

else:
    print('The time of execution of downloading and loading the model : %dh:%dm:%ds' %(hours,minutes,seconds))

*** Pipeline:


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text: 
Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Recently, generative artificial neural networks have been able to surpass many previous approaches in performance.Machine learning approaches have been applied to many fields including large language models, computer vision, speech recognition, email filtering, agriculture, and medicine, where it is too costly to develop algorithms to perform the needed tasks. ML is known in its application across business problems under the name predictive analytics. Although not all machine learning is statistically

In [21]:
# record start time
start = time.time()

# We will generate all possible mcq from single page in json format
prompt = "Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text:"+" "+item+'''\n Give me output in this JSON array format:[{"question": string, "options":List[string], "answer":string}]'''
prompt_template= f'''[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> {prompt}[/INST]'''


input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
# output = model.generate(inputs=input_ids, do_sample=True,max_length=32768)
# output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
# print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@",tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=16384,
    do_sample=True,
)


print(pipe(prompt_template)[0]['generated_text'])
# record end time
end = time.time()

temp = end-start
print(temp)
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes

total_time_for_generation = "%dh:%dm:%ds" %(hours,minutes,seconds)

if hours == 0:
    print('The time of execution of downloading and loading the model : %dm:%ds' %(minutes,seconds))

else:
    print('The time of execution of downloading and loading the model : %dh:%dm:%ds' %(hours,minutes,seconds))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


*** Pipeline:
[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text: 
Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Recently, generative artificial neural networks have been able to surpass many previous approaches in performance.Machine learning approaches have been applied to many fields including large language models, computer vision, speech recognition, email filtering, agriculture, and medicine, where it is too costly to develop algorithms to perform the needed tasks. ML is known in its application across business problems under the name predictive analytics. Although not all machine learning is

In [22]:
# record start time
start = time.time()

# We will generate all possible mcq from single page in json format
prompt = "Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text:"+" "+item+'''\n Give me output in this JSON array format:[{"question": string, "options":List[string], "answer":string}]'''
prompt_template= f'''[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> {prompt}[/INST]'''


input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
# output = model.generate(inputs=input_ids, do_sample=True,max_length=32768)
# output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
# print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@",tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=32768,
    do_sample=True,
)


print(pipe(prompt_template)[0]['generated_text'])
# record end time
end = time.time()

temp = end-start
print(temp)
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes

total_time_for_generation = "%dh:%dm:%ds" %(hours,minutes,seconds)

if hours == 0:
    print('The time of execution of downloading and loading the model : %dm:%ds' %(minutes,seconds))

else:
    print('The time of execution of downloading and loading the model : %dh:%dm:%ds' %(hours,minutes,seconds))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


*** Pipeline:
[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text: 
Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions. Recently, generative artificial neural networks have been able to surpass many previous approaches in performance.Machine learning approaches have been applied to many fields including large language models, computer vision, speech recognition, email filtering, agriculture, and medicine, where it is too costly to develop algorithms to perform the needed tasks. ML is known in its application across business problems under the name predictive analytics. Although not all machine learning is

In [23]:
item = '''
Mohandas Karamchand Gandhi (2 October 1869 – 30 January 1948) was an Indian lawyer, anti-colonial nationalist and political ethicist who employed nonviolent resistance to lead the successful campaign for India's independence from British rule. He inspired movements for civil rights and freedom across the world. The honorific Mahātmā (from Sanskrit 'great-souled, venerable'), first applied to him in South Africa in 1914, is now used throughout the world.
Born and raised in a Hindu family in coastal Gujarat, Gandhi trained in the law at the Inner Temple, London, and was called to the bar at age 22 in June 1891. After two uncertain years in India, where he was unable to start a successful law practice, he moved to South Africa in 1893 to represent an Indian merchant in a lawsuit. He went on to live in South Africa for 21 years. It was here that Gandhi raised a family and first employed nonviolent resistance in a campaign for civil rights. In 1915, aged 45, he returned to India and soon set about organising peasants, farmers, and urban labourers to protest against excessive land-tax and discrimination.
Assuming leadership of the Indian National Congress in 1921, Gandhi led nationwide campaigns for easing poverty, expanding women's rights, building religious and ethnic amity, ending untouchability, and, above all, achieving swaraj or self-rule. Gandhi adopted the short dhoti woven with hand-spun yarn as a mark of identification with India's rural poor. He began to live in a self-sufficient residential community, to eat simple food, and undertake long fasts as a means of both introspection and political protest. Bringing anti-colonial nationalism to the common Indians, Gandhi led them in challenging the British-imposed salt tax with the 400 km (250 mi) Dandi Salt March in 1930 and in calling for the British to quit India in 1942. He was imprisoned many times and for many years in both South Africa and India.
Gandhi's vision of an independent India based on religious pluralism was challenged in the early 1940s by a Muslim nationalism which demanded a separate homeland for Muslims within British India. In August 1947, Britain granted independence, but the British Indian Empire was partitioned into two dominions, a Hindu-majority India and a Muslim-majority Pakistan. As many displaced Hindus, Muslims, and Sikhs made their way to their new lands, religious violence broke out, especially in the Punjab and Bengal. Abstaining from the official celebration of independence, Gandhi visited the affected areas, attempting to alleviate distress. In the months following, he undertook several hunger strikes to stop the religious violence. The last of these was begun in Delhi on 12 January 1948 when he was 78. The belief that Gandhi had been too resolute in his defense of both Pakistan and Indian Muslims spread among some Hindus in India. Among these was Nathuram Godse, a militant Hindu nationalist from Pune, western India, who assassinated Gandhi by firing three bullets into his chest at an interfaith prayer meeting in Delhi on 30 January 1948.
Gandhi's birthday, 2 October, is commemorated in India as Gandhi Jayanti, a national holiday, and worldwide as the International Day of Nonviolence. Gandhi is considered to be the Father of the Nation in post-colonial India. During India's nationalist movement and in several decades immediately after, he was also commonly called Bapu (Gujarati endearment for "father", roughly "papa", "daddy").


== Early life and background ==


=== Parents ===
Gandhi's father, Karamchand Uttamchand Gandhi (1822–1885), served as the dewan (chief minister) of Porbandar state. His family originated from the then village of Kutiana in what was then Junagadh State. Although he only had been a clerk in the state administration and had an elementary education, Karamchand proved a capable chief minister.During his tenure, Karamchand married four times. His first two wives died young, after each had given birth to a daughter, and his third marriage was childless. In 1857, he sought his third wife's permission to remarry; that year, he married Putlibai (1844–1891), who also came from Junagadh, and was from a Pranami Vaishnava family. Karamchand and Putlibai had four children: a son, Laxmidas (c. 1860–1914); a daughter, Raliatbehn (1862–1960); a second son, Karsandas (c. 1866–1913). and a third son, Mohandas Karamchand Gandhi who was born on 2 October 1869 in Porbandar (also known as Sudamapuri), a coastal town on the Kathiawar Peninsula and then part of the small princely state of Porbandar in the Kathiawar Agency of the British Raj.
In 1874, Gandhi's father Karamchand left Porbandar for the smaller state of Rajkot, where he became a counsellor to its ruler, the Thakur Sahib; though Rajkot was a less prestigious state than Porbandar, the British regional political agency was located there, which gave the state's diwan a measure of security. In 1876, Karamchand became diwan of Rajkot and was succeeded as diwan of Porbandar by his brother Tulsidas. His family then rejoined him in Rajkot.

'''

In [25]:
# record start time
start = time.time()

# We will generate all possible mcq from single page in json format
prompt = "Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text:"+" "+item+'''\n Give me output in this JSON array format:[{"question": string, "options":List[string], "answer":string}]'''
prompt_template= f'''[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> {prompt}[/INST]'''

# print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, do_sample=True,max_length=32768)
# output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@",tokenizer.decode(output[0]))

end_time = time.time()

# Calculate tokens per second
num_tokens_generated = output.shape[1]
time_taken = end_time - start
tokens_per_second = num_tokens_generated / time_taken

print(f"Tokens per second: {tokens_per_second}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ <s> [INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text: 
Mohandas Karamchand Gandhi (2 October 1869 – 30 January 1948) was an Indian lawyer, anti-colonial nationalist and political ethicist who employed nonviolent resistance to lead the successful campaign for India's independence from British rule. He inspired movements for civil rights and freedom across the world. The honorific Mahātmā (from Sanskrit 'great-souled, venerable'), first applied to him in South Africa in 1914, is now used throughout the world.
Born and raised in a Hindu family in coastal Gujarat, Gandhi trained in the law at the Inner Temple, London, and was called to the bar at age 22 in June 1891. After two uncertain years in India, where he was unable to start a successful la

In [26]:
# record start time
start = time.time()

# We will generate all possible mcq from single page in json format
prompt = "Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text:"+" "+item+'''\n Give me output in this JSON array format:[{"question": string, "options":List[string], "answer":string}]'''
prompt_template= f'''[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> {prompt}[/INST]'''

# print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, do_sample=True,max_length=4096)
# output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@",tokenizer.decode(output[0]))

end_time = time.time()

# Calculate tokens per second
num_tokens_generated = output.shape[1]
time_taken = end_time - start
tokens_per_second = num_tokens_generated / time_taken

print(f"Tokens per second: {tokens_per_second}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ <s> [INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text: 
Mohandas Karamchand Gandhi (2 October 1869 – 30 January 1948) was an Indian lawyer, anti-colonial nationalist and political ethicist who employed nonviolent resistance to lead the successful campaign for India's independence from British rule. He inspired movements for civil rights and freedom across the world. The honorific Mahātmā (from Sanskrit 'great-souled, venerable'), first applied to him in South Africa in 1914, is now used throughout the world.
Born and raised in a Hindu family in coastal Gujarat, Gandhi trained in the law at the Inner Temple, London, and was called to the bar at age 22 in June 1891. After two uncertain years in India, where he was unable to start a successful la

In [27]:
# record start time
start = time.time()

# We will generate all possible mcq from single page in json format
prompt = "Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text:"+" "+item+'''\n Give me output in this JSON array format:[{"question": string, "options":List[string], "answer":string}]'''
prompt_template= f'''[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> {prompt}[/INST]'''

# print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, do_sample=True,max_length=8192)
# output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@",tokenizer.decode(output[0]))

end_time = time.time()

# Calculate tokens per second
num_tokens_generated = output.shape[1]
time_taken = end_time - start
tokens_per_second = num_tokens_generated / time_taken

print(f"Tokens per second: {tokens_per_second}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ <s> [INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text: 
Mohandas Karamchand Gandhi (2 October 1869 – 30 January 1948) was an Indian lawyer, anti-colonial nationalist and political ethicist who employed nonviolent resistance to lead the successful campaign for India's independence from British rule. He inspired movements for civil rights and freedom across the world. The honorific Mahātmā (from Sanskrit 'great-souled, venerable'), first applied to him in South Africa in 1914, is now used throughout the world.
Born and raised in a Hindu family in coastal Gujarat, Gandhi trained in the law at the Inner Temple, London, and was called to the bar at age 22 in June 1891. After two uncertain years in India, where he was unable to start a successful la

In [28]:
# record start time
start = time.time()

# We will generate all possible mcq from single page in json format
prompt = "Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text:"+" "+item+'''\n Give me output in this JSON array format:[{"question": string, "options":List[string], "answer":string}]'''
prompt_template= f'''[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> {prompt}[/INST]'''

# print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, do_sample=True,max_length=16384)
# output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@",tokenizer.decode(output[0]))

end_time = time.time()

# Calculate tokens per second
num_tokens_generated = output.shape[1]
time_taken = end_time - start
tokens_per_second = num_tokens_generated / time_taken

print(f"Tokens per second: {tokens_per_second}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ <s> [INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text: 
Mohandas Karamchand Gandhi (2 October 1869 – 30 January 1948) was an Indian lawyer, anti-colonial nationalist and political ethicist who employed nonviolent resistance to lead the successful campaign for India's independence from British rule. He inspired movements for civil rights and freedom across the world. The honorific Mahātmā (from Sanskrit 'great-souled, venerable'), first applied to him in South Africa in 1914, is now used throughout the world.
Born and raised in a Hindu family in coastal Gujarat, Gandhi trained in the law at the Inner Temple, London, and was called to the bar at age 22 in June 1891. After two uncertain years in India, where he was unable to start a successful la

In [29]:
# record start time
start = time.time()

# We will generate all possible mcq from single page in json format
prompt = "Generate as many as possible difficulty level easy Multiple choice questions with four options and answer using this text:"+" "+item+'''\n Give me output in this JSON array format:[{"question": string, "options":List[string], "answer":string}]'''
prompt_template= f'''[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> {prompt}[/INST]'''


input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    do_sample=True,
    max_new_tokens=4096,
    temperature= 0.6,
    top_p= 0.95,
    top_k= 50,
    repetition_penalty= 1.2,
    return_full_text= False

)

# print("######",pipe(prompt_template)[0]['generated_text'][len(prompt):])
output_text = pipe(prompt_template)[0]['generated_text']
print(pipe(prompt_template)[0]['generated_text'])
# record end time
end = time.time()

temp = end-start
print(temp)
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes

total_time_for_generation = "%dh:%dm:%ds" %(hours,minutes,seconds)

if hours == 0:
    print('The time of execution of downloading and loading the model : %dm:%ds' %(minutes,seconds))

else:
    print('The time of execution of downloading and loading the model : %dh:%dm:%ds' %(hours,minutes,seconds))

generation_time = end - start
generated_tokens = len(tokenizer.encode(output_text))

tokens_per_second = generated_tokens / generation_time

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


*** Pipeline:


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Sure! Here are some multiple choice questions about Mohandas Karamchand Gandhi in the requested JSON format:

[
{"question": "When and where was Gandhi born?", "options": ["2 October 1869 in Porbandar", "1 October 1868 in Mumbai", "3 October 1869 in Delhi", "2 October 1870 in Ahmedabad"], "answer": "2 October 1869 in Porbandar"},
{"question": "What did Gandhi study in London?", "options": ["Medicine", "Law", "Engineering", "Political Science"], "answer": "Law"},
{"question": "In which country did Gandhi first employ nonviolent resistance?", "options": ["India", "South Africa", "England", "United States"], "answer": "South Africa"},
{"question": "What party did Gandhi lead during the Indian Independence Movement?", "options": ["Congress Party", "Muslim League", "Communist Party", "National Socialist Party"], "answer": "Congress Party"},
{"question": "Which march did Gandhi lead to challenge the British-imposed salt tax?", "options": ["The Dharasana Satyagraha", "The Quit India Movement

In [37]:
tokens_per_second

3.2200662027586158

In [30]:
generated_tokens

413

In [35]:
lst2 = '''
{"question": "When and where was Gandhi born?", "options": ["2 October 1869 in Porbandar", "1 October 1868 in Mumbai", "3 October 1869 in Delhi", "2 October 1870 in Ahmedabad"], "answer": "2 October 1869 in Porbandar"},
{"question": "What did Gandhi study in London?", "options": ["Medicine", "Law", "Engineering", "Political Science"], "answer": "Law"},
{"question": "In which country did Gandhi first employ nonviolent resistance?", "options": ["India", "South Africa", "England", "United States"], "answer": "South Africa"},
{"question": "What party did Gandhi lead during the Indian Independence Movement?", "options": ["Congress Party", "Muslim League", "Communist Party", "National Socialist Party"], "answer": "Congress Party"},
{"question": "Which march did Gandhi lead to challenge the British-imposed salt tax?", "options": ["The Dharasana Satyagraha", "The Quit India Movement", "The Civil Disobedience Movement", "The Dandi Salt March"], "answer": "The Dandi Salt March"},
{"question": "Who assassinated Gandhi?", "options": ["Nathuram Godse", "Bal Gangadhar Tilak", "Subhas Chandra Bose", "Jawaharlal Nehru"], "answer": "Nathuram Godse"}
'''

In [36]:
len(lst2)

1152

In [38]:

# record start time
start = time.time()

# We will generate all possible mcq from single page in json format
prompt = "Generate as many as possible difficulty level easy Multiple choice questions with four options and answer using this text:"+" "+item+'''\n Give me output in this JSON array format:[{"question": string, "options":List[string], "answer":string}]'''
prompt_template= f'''[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> {prompt}[/INST]'''


input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    do_sample=True,
    max_new_tokens=4096,
    temperature= 0.6,
    top_p= 0.95,
    top_k= 50,
    repetition_penalty= 1.2,
    return_full_text= False

)

# print("######",pipe(prompt_template)[0]['generated_text'][len(prompt):])
output_text = pipe(prompt_template)[0]['generated_text']
print(pipe(prompt_template)[0]['generated_text'])
# record end time
end = time.time()

temp = end-start
print(temp)
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes

total_time_for_generation = "%dh:%dm:%ds" %(hours,minutes,seconds)

if hours == 0:
    print('The time of execution of downloading and loading the model : %dm:%ds' %(minutes,seconds))

else:
    print('The time of execution of downloading and loading the model : %dh:%dm:%ds' %(hours,minutes,seconds))

generation_time = end - start
generated_tokens = len(tokenizer.encode(output_text))

tokens_per_second = generated_tokens / generation_time

print(f"Generated {generated_tokens} tokens in {generation_time:.2f} seconds.")
print(f"Tokens per second: {tokens_per_second:.2f}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


*** Pipeline:


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Sure! Here are some multiple choice questions about Mohandas Karamchand Gandhi:
```json
[
    {
        "question": "When and where was Mohandas Karamchand Gandhi born?",
        "options": ["2 October 1869 in Porbandar", "1 January 1870 in Mumbai", "3 September 1868 in Delhi", "5 May 1871 in Kolkata"],
        "answer": "2 October 1869 in Porbandar"
    },
    {
        "question": "What did Gandhi study at the Inner Temple in London?",
        "options": ["Medicine", "Law", "Political Science", "Engineering"],
        "answer": "Law"
    },
    {
        "question": "In what country did Gandhi first employ nonviolent resistance in a campaign for civil rights?",
        "options": ["India", "South Africa", "United States", "England"],
        "answer": "South Africa"
    },
    {
        "question": "What was the name of the march that Gandhi led to challenge the British-imposed salt tax?",
        "options": ["The Dandi March", "The Salt Satyagraha", "The Quit India Movement", "The 

## hindi

In [8]:
item = '''
मोहनदास करमचन्द गांधी (जन्म: 2 अक्टूबर 1869 -  निधन: 30 जनवरी 1948) जिन्हें महात्मा गांधी के नाम से भी जाना जाता है, भारत एवं भारतीय स्वतन्त्रता आन्दोलन के एक प्रमुख राजनैतिक एवं आध्यात्मिक नेता थे। वे सत्याग्रह (व्यापक सविनय अवज्ञा) के माध्यम से अत्याचार के प्रतिकार करने के समर्थक अग्रणी नेता थे, उनकी इस अवधारणा की नींव सम्पूर्ण अहिंसा के सिद्धान्त पर रखी गयी थी जिसने भारत सहित पूरे विश्व में जनता के नागरिक अधिकारों एवं स्वतन्त्रता के प्रति आन्दोलन के लिये प्रेरित किया। उन्हें संसार में साधारण जनता महात्मा गांधी के नाम से जानती है। संस्कृत भाषा में महात्मा अथवा 'महान आत्मा' एक सम्मान सूचक शब्द है। गांधी को महात्मा के नाम से सबसे पहले 1915 में राजवैद्य जीवराम कालिदास ने संबोधित किया था। एक अन्य मत के अनुसार स्वामी श्रद्धानन्द ने 1915 मे महात्मा की उपाधि दी थी। तीसरा मत ये है कि रवीन्द्रनाथ ठाकुर ने 12 अप्रैल 1919 को अपने एक लेख मे उन्हें महात्मा कहकर सम्बोधित किया था। । उन्हें बापू (गुजराती भाषा में બાપુ बापू अर्थात् पिता) के नाम से भी स्मरण किया जाता है। एक मत के अनुसार गांधीजी को बापू सम्बोधित करने वाले प्रथम व्यक्ति उनके साबरमती आश्रम के शिष्य थे सुभाष चन्द्र बोस ने 6 जुलाई 1944 को रंगून रेडियो से गांधी जी के नाम जारी प्रसारण में उन्हें राष्ट्रपिता कहकर सम्बोधित करते हुए आज़ाद हिन्द फौज के सैनिकों के लिये उनका आशीर्वाद और शुभकामनाएँ माँगीं थीं। प्रति वर्ष 2 अक्टूबर को उनका जन्म दिन भारत में गांधी जयन्ती के रूप में और पूरे विश्व में अन्तरराष्ट्रीय अहिंसा दिवस के रूप में मनाया जाता है।
सबसे पहले गान्धी जी ने प्रवासी वकील के रूप में दक्षिण अफ्रीका में भारतीय समुदाय के लोगों के नागरिक अधिकारों के लिये संघर्ष हेतु सत्याग्रह करना आरम्भ किया। 1915 में उनकी भारत वापसी हुई। उसके बाद उन्होंने यहाँ के किसानों, श्रमिकों और नगरीय श्रमिकों को अत्यधिक भूमि कर और भेदभाव के विरुद्ध आवाज उठाने के लिये एकजुट किया। 1921 में भारतीय राष्ट्रीय कांग्रेस की बागडोर संभालने के बाद उन्होंने देशभर में दरिद्रता से मुक्ति दिलाने, महिलाओं के अधिकारों का विस्तार, धार्मिक एवं जातीय एकता का निर्माण व आत्मनिर्भरता के लिये अस्पृश्‍यता के विरोध में अनेकों कार्यक्रम चलाये। इन सबमें विदेशी राज से मुक्ति दिलाने वाला स्वराज की प्राप्ति वाला कार्यक्रम ही प्रमुख था। गाँधी जी ने ब्रिटिश सरकार द्वारा भारतीयों पर लगाये गये लवण कर के विरोध में 1930 में नमक सत्याग्रह और इसके बाद 1942 में अंग्रेजो भारत छोड़ो आन्दोलन से विशेष विख्याति प्राप्त की। दक्षिण अफ्रीका और भारत में विभिन्न अवसरों पर कई वर्षों तक उन्हें कारागृह में भी रहना पड़ा। गांधी जी ने सभी परिस्थितियों में अहिंसा और सत्य का पालन किया और सभी को इनका पालन करने के लिये वकालत भी की। उन्होंने साबरमती आश्रम में अपना जीवन बिताया और परम्परागत भारतीय पोशाक धोती व सूत से बनी शाल पहनी जिसे वे स्वयं चरखे पर सूत कातकर हाथ से बनाते थे। उन्होंने सादा शाकाहारी भोजन खाया और आत्मशुद्धि के लिये लम्बे-लम्बे उपवास रखे। 12 फरवरी वर्ष 1948 में महात्मा गांधी के अस्थि कलश जिन 12 तटों पर विसर्जित किए गए थे, त्रिमोहिनी संगम भी उनमें से एक है |


== प्रारम्भिक जीवन ==
मोहनदास करमचन्द गान्धी का जन्म पश्चिमी भारत में वर्तमान गुजरात के एक तटीय नगर पोरबंदर नामक स्थान पर २ अक्टूबर सन् १८६९ को हुआ था। उनके पिता करमचन्द गान्धी सनातन धर्म की पंसारी जाति से सम्बन्ध रखते थे और ब्रिटिश राज के समय काठियावाड़ की एक छोटी सी रियासत (पोरबंदर) के दीवान अर्थात् प्रधान मन्त्री थे। गुजराती भाषा में गान्धी का अर्थ है पंसारी जबकि हिन्दी भाषा में गन्धी का अर्थ है इत्र फुलेल बेचने वाला जिसे अंग्रेजी में परफ्यूमर कहा जाता है। उनकी माता पुतलीबाई परनामी वैश्य समुदाय की थीं। पुतलीबाई करमचन्द की चौथी पत्नी थी। उनकी पहली तीन पत्नियाँ प्रसव के समय मर गयीं थीं। भक्ति करने वाली माता की देखरेख और उस क्षेत्र की जैन परम्पराओं के कारण युवा मोहनदास पर वे प्रभाव प्रारम्भ में ही पड़ गये थे जिसने आगे चलकर महात्मा गांधी के जीवन में महत्वपूर्ण भूमिका निभायी। इन प्रभावों में सम्मिलित थे दुर्बलों में उत्साह की भावना, शाकाहारी जीवन, आत्मशुद्धि के लिये उपवास तथा विभिन्न जातियों के लोगों के बीच सहिष्णुता। 


=== कम आयु में विवाह ===
मई १८८३ में साढे १३ वर्ष की आयु पूर्ण करते ही उनका विवाह १४ वर्ष की कस्तूर बाई मकनजी से कर दिया गया। पत्नी का पहला नाम छोटा करके कस्तूरबा कर दिया गया और उसे लोग प्यार से बा कहते थे। यह विवाह उनके माता पिता द्वारा तय किया गया व्यवस्थित बाल विवाह था जो उस समय उस क्षेत्र में प्रचलित था। परन्तु उस क्षेत्र में यही रीति थी कि किशोर दुल्हन को अपने माता पिता के घर और अपने पति से अलग अधिक समय तक रहना पड़ता था। १८८५ में जब गान्धी जी १५ वर्ष के थे तब इनकी पहली सन्तान ने जन्म लिया। किन्तु वह केवल कुछ दिन ही जीवित रही। और इसी वर्ष उनके पिता करमचन्द गांधी भी चल बसे। मोहनदास और कस्तूरबा के चार सन्तान हुईं जो सभी पुत्र थे। हरीलाल गान्धी १८८८ में, मणिलाल गान्धी १८९२ में, रामदास गान्धी १८९७ में और देवदास गांधी १९०० में जन्मे। पोरबंदर से उन्होंने मिडिल और राजकोट से हाई स्कूल किया। दोनों परीक्षाओं में शैक्षणिक स्तर वह एक साधारण छात्र रहे। मैट्रिक के बाद की परीक्षा उन्होंने भावनगर के शामलदास कॉलेज से कुछ समस्या के साथ उत्तीर्ण की। जब तक वे वहाँ रहे अप्रसन्न ही रहे क्योंकि उनका परिवार उन्हें बैरिस्टर बनाना चाहता था।


=== विदेश में शिक्षा व विदेश में ही वकालत ===
अपने १९वें जन्मदिन से लगभग एक महीने पहले ही ४ सितम्बर १८८८ को गांधी यूनिवर्सिटी कॉलेज लन्दन में कानून की पढाई करने और बैरिस्टर बनने के लिये इंग्लैंड चले गये। भारत छोड़ते समय जैन भिक्षु बेचारजी के समक्ष हिन्दुओं को मांस, शराब तथा संकीर्ण विचारधारा को त्यागने के लिए अपनी अपनी माता जी को दिए गये एक वचन ने उनके शाही राजधानी लंदन में बिताये गये समय को काफी प्रभावित किया। हालांकि गांधी जी ने अंग्रेजी रीति रिवाजों का अनुभव भी किया जैसे उदाहरण के तौर पर नृत्य कक्षाओं में जाने आदि का। फिर भी वह अपनी मकान मालकिन द्वारा मांस एवं पत्ता गोभी को हजम.नहीं कर सके। उन्होंने कुछ शाकाहारी भोजनालयों की ओर इशारा किया। अपनी माता की इच्छाओं के बारे में जो कुछ उन्होंने पढा था उसे सीधे अपनाने की बजाय उन्होंने बौद्धिकता से शाकाहारी भोजन का अपना भोजन स्वीकार किया। उन्होंने शाकाहारी समाज की सदस्यता ग्रहण की और इसकी कार्यकारी समिति के लिये उनका चयन भी हो गया जहाँ उन्होंने एक स्थानीय अध्याय की नींव रखी। बाद में उन्होने संस्थाएँ गठित करने में महत्वपूर्ण अनुभव का परिचय देते हुए इसे श्रेय दिया। वे जिन शाकाहारी लोगों से मिले उनमें से कुछ थियोसोफिकल सोसायटी के सदस्य भी थे। इस सोसाइटी की स्थापना १८७५ में विश्व बन्धुत्व को प्रबल करने के लिये की गयी थी और इसे बौद्ध धर्म एवं सनातन धर्म के साहित्य के अध्ययन के लिये समर्पित किया गया था।
उन्हों लोगों ने गांधी जी को श्रीमद्भगवद्गीता पढ़ने के लिये प्रेरित किया। हिन्दू, ईसाई, बौद्ध, इस्लाम और अन्य धर्मों .के बारे में पढ़ने से पहले गांधी ने धर्म में विशेष रुचि नहीं दिखायी। इंग्लैंड और वेल्स बार एसोसिएशन में वापस बुलावे पर वे भारत लौट आये किन्तु बम्बई में वकालत करने में उन्हें कोई खास सफलता नहीं मिली। बाद में एक हाई स्कूल शिक्षक के रूप में अंशकालिक नौकरी का प्रार्थना पत्र अस्वीकार कर दिये जाने पर उन्होंने जरूरतमन्दों के लिये मुकदमे की अर्जियाँ लिखने के लिये राजकोट को ही अपना स्थायी मुकाम बना लिया। परन्तु एक अंग्रेज अधिकारी की मूर्खता के कारण उन्हें यह कारोबार भी छोड़ना पड़ा। अपनी आत्मकथा में उन्होंने इस घटना का वर्णन अपने बड़े भाई की ओर से परोपकार की असफल कोशिश के रूप में किया है। यही वह कारण था जिस वजह से उन्होंने सन् १८९३ में एक भारतीय फर्म से नेटाल दक्षिण अफ्रीका में, जो उन दिनों ब्रिटिश साम्राज्य का भाग होता था, एक वर्ष के करार पर वकालत का कारोवार स्वीकार कर लिया।


== दक्षिण अफ्रीका (1893-1914) में नागरिक अधिकारों के आन्दोलन ==
दक्षिण अफ्रीका में गान्धी को भारतीयों पर भेदभाव का सामना करना पड़ा। आरम्भ में उन्हें प्रथम श्रेणी कोच की वैध टिकट होने के बाद तीसरी श्रेणी के डिब्बे में जाने से इन्कार करने के लिए ट्रेन से बाहर फेंक दिया गया था। इतना ही नहीं पायदान पर शेष यात्रा करते हुए एक यूरोपियन यात्री के अन्दर आने पर चालक की मार भी झेलनी पड़ी। उन्होंने अपनी इस यात्रा में अन्य भी कई कठिनाइयों का सामना किया। अफ्रीका में कई होटलों को उनके लिए वर्जित कर दिया गया। इसी तरह ही बहुत सी घटनाओं में से एक यह भी थी जिसमें अदालत के न्यायाधीश ने उन्हें अपनी पगड़ी उतारने का आदेश दिया था जिसे उन्होंने नहीं माना। ये सारी घटनाएँ गान्धी के जीवन में एक मोड़ बन गईं और विद्यमान सामाजिक अन्याय के प्रति जागरुकता का कारण बनीं तथा सामाजिक सक्रियता की व्याख्या करने में मददगार सिद्ध हुईं। दक्षिण अफ्रीका में भारतीयों पर हो रहे अन्याय को देखते हुए गान्धी ने अंग्रेजी साम्राज्य के अन्तर्गत अपने देशवासियों के सम्मान तथा देश में स्वयं अपनी स्थिति के लिए प्रश्न उठाये।


== १९०६ के ज़ुलु युद्ध में भूमिका ==
१९०६ में, ज़ुलु (Zulu) दक्षिण अफ्रीका में नए चुनाव कर के लागू करने के बाद दो अंग्रेज अधिकारियों को मार डाला गया। बदले में अंग्रेजों ने जूलू के खिलाफ युद्ध छेड़ दिया। गांधी जी ने भारतीयों को भर्ती करने के लिए ब्रिटिश अधिकारियों को सक्रिय रूप से प्रेरित किया। उनका तर्क था अपनी नागरिकता के दावों को कानूनी जामा पहनाने के लिए भारतीयों को युद्ध प्रयासों में सहयोग देना चाहिए। तथापि, अंग्रेजों ने अपनी सेना में भारतीयों को पद देने से इंकार कर दिया था। इसके बावजूद उन्होने गांधी जी के इस प्रस्ताव को मान लिया कि भारतीय घायल अंग्रेज सैनिकों को उपचार के लिए स्टेचर पर लाने के लिए स्वैच्छा पूर्वक कार्य कर सकते हैं। इस कोर की बागडोर गांधी ने थामी।२१ जुलाई (July 21), १९०६ को गांधी जी ने भारतीय जनमत इंडियन ओपिनिय (Indian Opinion) में लिखा कि २३ भारतीय निवासियों के विरूद्ध चलाए गए आप्रेशन के संबंध में प्रयोग द्वारा नेटाल सरकार के कहने पर एक कोर का गठन किया गया है।दक्षिण अफ्रीका में भारतीय लोगों से इंडियन ओपिनियन में अपने कॉलमों के माध्‍यम से इस युद्ध में शामिल होने के लिए आग्रह किया और कहा, यदि सरकार केवल यही महसूस करती हे कि आरक्षित बल बेकार हो रहे हैं तब वे इसका उपयोग करेंगे और असली लड़ाई के लिए भारतीयों का प्रशिक्षण देकर इसका अवसर देंगे।गांधी की राय में, १९०६ का मसौदा अध्यादेश भारतीयों की स्थिति में किसी निवासी के नीचे वाले स्तर के समान लाने जैसा था। इसलिए उन्होंने सत्याग्रह (Satyagraha), की तर्ज पर "काफिर (Kaffir)s " .का उदाहरण देते हुए भारतीयों से अध्यादेश का विरोध करने का आग्रह किया। उनके शब्दों में, " यहाँ तक कि आधी जातियां और काफिर जो हमसे कम आधुनिक हैं ने भी सरकार का विरोध किया है। पास का नियम उन पर भी लागू होता है किंतु वे पास नहीं दिखाते हैं।


== भारतीय स्वतंत्रता संग्राम के लिए संघर्ष (१९१६ -१९४५) ==
गांधी १९१५ में दक्षिण अफ्रीका से भारत में रहने के लिए लौट आए। उन्होंने भारतीय राष्ट्रीय कांग्रेस के अधिवेशनों पर अपने विचार व्य‍क्त किए, लेकिन उनके विचार भारत के मुख्य मुद्दों, राजनीति तथा उस समय के कांग्रेस दल के प्रमुख भारतीय नेता गोपाल कृष्ण गोखले  पर ही आधारित थे जो एक सम्मानित नेता थे।


=== चंपारण और खेड़ा ===
गांधी की पहली बड़ी उपलब्धि १९१८ में चम्पारन सत्याग्रह और खेड़ा सत्याग्रह  में मिली हालांकि अपने निर्वाह के लिए जरूरी खाद्य फसलों की बजाए नील (indigo) नकद पैसा देने वाली खाद्य फसलों की खेती वाले आंदोलन भी महत्वपूर्ण रहे। जमींदारों (अधिकांश अंग्रेज) की ताकत से दमन हुए भारतीयों को नाममात्र भरपाई भत्ता दिया गया जिससे वे अत्यधिक गरीबी से घिर गए। गांवों को बुरी तरह गंदा और अस्वास्थ्यकर (unhygienic); और शराब, अस्पृश्यता और पर्दा से बांध दिया गया। अब एक विनाशकारी अकाल के कारण शाही कोष की भरपाई के लिए अंग्रेजों ने दमनकारी कर लगा दिए जिनका बोझ दिन प्रतिदिन बढता ही गया। यह स्थिति निराशजनक थी। खेड़ा (Kheda), गुजरात में भी यही समस्या थी। गांधी जी ने वहां एक आश्रम (ashram) बनाया जहाँ उनके बहुत सारे समर्थकों और नए स्वेच्छिक कार्यकर्ताओं को संगठित किया गया। उन्होंने गांवों का एक विस्तृत अध्ययन और सर्वेक्षण किया जिसमें प्राणियों पर हुए अत्याचार के भयानक कांडों का लेखाजोखा रखा गया और इसमें लोगों की अनुत्पादकीय सामान्य अवस्था को भी शामिल किया गया था। ग्रामीणों में विश्‍वास पैदा करते हुए उन्होंने अपना कार्य गांवों की सफाई करने से आरंभ किया जिसके अंतर्गत स्कूल और अस्पताल बनाए गए और उपरोक्त वर्णित बहुत सी सामाजिक बुराईयों को समाप्त करने के लिए ग्रामीण नेतृत्व प्रेरित किया।
लेकिन इसके प्रमुख प्रभाव उस समय देखने को मिले जब उन्हें अशांति फैलाने के लिए पुलिस ने गिरफ्तार किया और उन्हें प्रांत छोड़ने के लिए आदेश दिया गया। हजारों की तादाद में लोगों ने विरोध प्रदर्शन किए ओर जेल, पुलिस स्टेशन एवं अदालतों के बाहर रैलियां निकालकर गांधी जी को बिना शर्त रिहा करने की मांग की। गांधी जी ने जमींदारों के खिलाफ़ विरोध प्रदर्शन और हड़तालों को का नेतृत्व किया जिन्होंने अंग्रेजी सरकार के मार्गदर्शन में उस क्षेत्र के गरीब किसानों को अधिक क्षतिपूर्ति मंजूर करने तथा खेती पर नियंत्रण, राजस्व में बढोतरी को रद्द करना तथा इसे संग्रहित करने वाले एक समझौते पर हस्ताक्षर किए। इस संघर्ष के दौरान ही, गांधी जी को जनता ने बापू पिता और महात्मा (महान आत्मा) के नाम से संबोधित किया। खेड़ा में सरदार पटेल ने अंग्रेजों के साथ विचार विमर्श के लिए किसानों का नेतृत्व किया जिसमें अंग्रेजों ने राजस्व संग्रहण से मुक्ति देकर सभी कैदियों को रिहा कर दिया गया था। इसके परिणामस्वरूप, गांधी की ख्याति देश भर में फैल गई।


=== असहयोग आन्दोलन ===

गांधी जी ने असहयोग, अहिंसा तथा शांतिपूर्ण प्रतिकार को अंग्रेजों के खिलाफ़ शस्त्र के रूप में उपयोग किया। पंजाब में अंग्रेजी फोजों द्वारा भारतीयों पर जलियावांला नरसंहार जिसे अमृतसर नरसंहार के नाम से भी जाना जाता है ने देश को भारी आघात पहुँचाया जिससे जनता में क्रोध और हिंसा की ज्वाला भड़क उठी। गांधीजी ने ब्रिटिश राज तथा भारतीयों द्वारा ‍प्रतिकारात्मक रवैया दोनों की की। उन्होंने ब्रिटिश नागरिकों तथा दंगों के शिकार लोगों के प्रति संवेदना व्यक्त की तथा पार्टी के आरम्भिक विरोध के बाद दंगों की भंर्त्सना की। गांधी जी के भावनात्मक भाषण के बाद अपने सिद्धांत की वकालत की कि सभी हिंसा और बुराई को न्यायोचित नहीं ठहराया जा सकता है। किंतु ऐसा इस नरसंहार और उसके बाद हुई हिंसा से गांधी जी ने अपना मन संपूर्ण सरकार आर भारतीय सरकार के कब्जे वाली संस्थाओं पर संपूर्ण नियंत्रण लाने पर केंद्रित था जो जल्‍दी ही स्वराज अथवा संपूर्ण व्यक्तिगत, आध्‍यात्मिक एवं राजनैतिक आजादी में बदलने वाला था।

दिसम्बर १९२१ में गांधी जो भारतीय राष्ट्रीय कांग्रेस.का कार्यकारी अधिकारी नियुक्त किया गया। उनके नेतृत्व में कांग्रेस को स्वराज.के नाम वाले एक नए उद्देश्‍य के साथ संगठित किया गया। पार्दी में सदस्यता सांकेतिक शुल्क का भुगताने पर सभी के लिए खुली थी। पार्टी को किसी एक कुलीन संगठन की न बनाकर इसे राष्ट्रीय जनता की पार्टी बनाने के लिए इसके अंदर अनुशासन में सुधार लाने के लिए एक पदसोपान समिति गठित की गई। गांधी जी ने अपने अहिंसात्मक मंच को स्वदेशी नीति — में शामिल करने के लिए विस्तार किया जिसमें विदेशी वस्तुओं विशेषकर अंग्रेजी वस्तुओं का बहिष्कार करना था। इससे जुड़ने वाली उनकी वकालत का कहना था कि सभी भारतीय अंग्रेजों द्वारा बनाए वस्त्रों की अपेक्षा हमारे अपने लोगों द्वारा हाथ से बनाई गई खादी पहनें। गांधी जी ने स्वतंत्रता आंदोलन को सहयोग देने के लिएपुरूषों और महिलाओं को प्रतिदिन खादी के लिए सूत कातने में समय बिताने के लिए कहा। यह अनुशासन और समर्पण लाने की ऐसी नीति थी जिससे अनिच्छा और महत्वाकाक्षा को दूर किया जा सके और इनके स्थान पर उस समय महिलाओं को शामिल किया जाए जब ऐसे बहुत से विचार आने लगे कि इस प्रकार की गतिविधियां महिलाओं के लिए सम्मानजनक नहीं हैं। इसके अलावा गांधी जी ने ब्रिटेन की शैक्षिक संस्थाओं तथा अदालतों का बहिष्कार और सरकारी नौकरियों को छोड़ने का तथा सरकार से प्राप्त तमगों और सम्मान (honours) को वापस लौटाने का भी अनुरोध किया।
असहयोग को दूर-दूर से अपील और सफलता मिली जिससे समाज के सभी वर्गों की जनता में जोश और भागीदारी बढ गई। फिर जैसे ही यह आंदोलन अपने शीर्ष पर पहुँचा वैसे फरवरी १९२२ में इसका अंत चौरी-चोरा, उत्तरप्रदेश में भयानक द्वेष के रूप में अंत हुआ। आंदोलन द्वारा हिंसा का रूख अपनाने के डर को ध्‍यान में रखते हुए और इस पर विचार करते हुए कि इससे उसके सभी कार्यों पर पानी फिर जाएगा, गांधी जी ने व्यापक असहयोग के इस आंदोलन को वापस ले लिया। गांधी पर गिरफ्तार किया गया १० मार्च, १९२२, को राजद्रोह के लिए गांधी जी पर मुकदमा चलाया गया जिसमें उन्हें छह साल कैद की सजा सुनाकर जैल भेद दिया गया। १८ मार्च, १९२२ से लेकर उन्होंने केवल २ साल ही जैल में बिताए थे कि उन्हें फरवरी १९२४ में आंतों के ऑपरेशन के लिए रिहा कर दिया गया।
गांधी जी के एकता वाले व्यक्तित्व के बिना इंडियन नेशनल कांग्रेस उसके जेल में दो साल रहने के दौरान ही दो दलों में बंटने लगी जिसके एक दल का नेतृत्व सदन में पार्टी की भागीदारी के पक्ष वाले चित्त रंजन दास तथा मोतीलाल नेहरू ने किया तो दूसरे दल का नेतृत्व इसके विपरीत चलने वाले चक्रवर्ती राजगोपालाचार्य और सरदार वल्लभ भाई पटेल ने किया। इसके अलावा, हिंदुओं और मुसलमानों के बीच अहिंसा आंदोलन की चरम सीमा पर पहुँचकर सहयोग टूट रहा था। गांधी जी ने इस खाई को बहुत से साधनों से भरने का प्रयास किया जिसमें उन्होंने १९२४ की बसंत में सीमित सफलता दिलाने वाले तीन सप्ताह का उपवास करना भी शामिल था।


=== स्वराज और नमक सत्याग्रह (नमक मार्च) ===
गांधी जी सक्रिय राजनीति से दूर ही रहे और १९२० की अधिकांश अवधि तक वे स्वराज पार्टी और इंडियन नेशनल कांग्रेस के बीच खाई को भरने में लगे रहे और इसके अतिरिक्त वे अस्पृश्यता, शराब, अज्ञानता और गरीबी के खिलाफ आंदोलन छेड़ते भी रहे। उन्होंने पहले १९२८ में लौटे .एक साल पहले अंग्रेजी सरकार ने सर जॉन साइमन के नेतृत्व में एक नया संवेधानिक सुधार आयोग बनाया जिसमें एक भी सदस्य भारतीय नहीं था। इसका परिणाम भारतीय राजनैतिक दलों द्वारा बहिष्कार निकला। दिसम्बर १९२८ में गांधी जी ने कलकत्ता में आयोजित कांग्रेस के एक अधिवेशन में एक प्रस्ताव रखा जिसमें भारतीय साम्राज्य को सत्ता प्रदान करने के लिए कहा गया था अथवा ऐसा न करने के बदले अपने उद्देश्य के रूप में संपूर्ण देश की आजादी के लिए असहयोग आंदोलन का सामना करने के लिए तैयार रहें। गांधी जी ने न केवल युवा वर्ग सुभाष चंद्र बोस तथा जवाहरलाल नेहरू जैसे पुरूषों द्वारा तत्काल आजादी की मांग के विचारों को फलीभूत किया बल्कि अपनी स्वयं की मांग को दो साल की बजाए एक साल के लिए रोक दिया। अंग्रेजों ने कोई जवाब नहीं दिया।.नहीं ३१ दिसम्बर १९२९, भारत का झंडा फहराया गया था लाहौर में है।२६ जनवरी १९३० का दिन लाहौर में भारतीय स्वतंत्रता दिवस के रूप में इंडियन नेशनल कांग्रेस ने मनाया। यह दिन लगभग प्रत्येक भारतीय संगठनों द्वारा भी मनाया गया। इसके बाद गांधी जी ने मार्च १९३० में नमक पर कर लगाए जाने के विरोध में नया सत्याग्रह चलाया जिसे १२ मार्च से ६ अप्रेल तक नमक आंदोलन के याद में ४०० किलोमीटर (२४८ मील) तक का सफर अहमदाबाद से दांडी, गुजरात तक चलाया गया ताकि स्वयं नमक उत्पन्न किया जा सके। समुद्र की ओर इस यात्रा में हजारों की संख्‍या में भारतीयों ने भाग लिया। भारत में अंग्रेजों की पकड़ को विचलित करने वाला यह एक सर्वाधिक सफल आंदोलन था जिसमें अंग्रेजों ने ८०,००० से अधिक लोगों को जेल भेजा।
लार्ड एडवर्ड इरविन द्वारा प्रतिनिधित्व वाली सरकार ने गांधी जी के साथ विचार विमर्श करने का निर्णय लिया। यह इरविन गांधी की सन्धि मार्च १९३१ में हस्ताक्षर किए थे। सविनय अवज्ञा आंदोलन को बंद करने के लिए ब्रिटिश सरकार ने सभी राजनैतिक कैदियों को रिहा करने के लिए अपनी रजामन्दी दे दी। इस समझौते के परिणामस्वरूप गांधी को भारतीय राष्ट्रीय कांग्रेस के एकमात्र प्रतिनिधि के रूप में लंदन में आयोजित होने वाले गोलमेज सम्मेलन में भाग लेने के लिए आमन्त्रित किया गया। यह सम्मेलन गांधी जी और राष्ट्रीयवादी लोगों के लिए घोर निराशाजनक रहा, इसका कारण सत्ता का हस्तांतरण करने की बजाय भारतीय कीमतों एवं भारतीय अल्पसंख्‍यकों पर केन्द्रित होना था। इसके अलावा, लार्ड इरविन के उत्तराधिकारी लार्ड विलिंगटन, ने राष्‍ट्रवादियों के आंदोलन को नियंत्रित एवं कुचलने का एक नया अभियान आरम्भ करदिया। गांधी फिर से गिरफ्तार कर लिए गए और सरकार ने उनके अनुयाईयों को उनसे पूर्णतया दूर रखते हुए गांधी जी द्वारा प्रभावित होने से रोकने की कोशिश की। लेकिन, यह युक्ति सफल नहीं थी।




'''

In [9]:
# record start time
start = time.time()

# We will generate all possible mcq from single page in json format
prompt = "Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text:"+" "+item+'''\n Give me output in this JSON array format:[{"question": string, "options":List[string], "answer":string}]'''
prompt_template= f'''[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> {prompt}[/INST]'''

# print("\n\n*** Generate:")

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=4096,
    do_sample=True,
)


print(pipe(prompt_template)[0]['generated_text'])
# record end time
end = time.time()

temp = end-start
print(temp)
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes

total_time_for_generation = "%dh:%dm:%ds" %(hours,minutes,seconds)

if hours == 0:
    print('The time of execution of downloading and loading the model : %dm:%ds' %(minutes,seconds))

else:
    print('The time of execution of downloading and loading the model : %dh:%dm:%ds' %(hours,minutes,seconds))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


*** Pipeline:
[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text: 
मोहनदास करमचन्द गांधी (जन्म: 2 अक्टूबर 1869 -  निधन: 30 जनवरी 1948) जिन्हें महात्मा गांधी के नाम से भी जाना जाता है, भारत एवं भारतीय स्वतन्त्रता आन्दोलन के एक प्रमुख राजनैतिक एवं आध्यात्मिक नेता थे। वे सत्याग्रह (व्यापक सविनय अवज्ञा) के माध्यम से अत्याचार के प्रतिकार करने के समर्थक अग्रणी नेता थे, उनकी इस अवधारणा की नींव सम्पूर्ण अहिंसा के सिद्धान्त पर रखी गयी थी जिसने भारत सहित पूरे विश्व में जनता के नागरिक अधिकारों एवं स्वतन्त्रता के प्रति आन्दोलन के लिये प्रेरित किया। उन्हें संसार में साधारण जनता महात्मा गांधी के नाम से जानती है। संस्कृत भाषा में महात्मा अथवा 'महान आत्मा' एक सम्मान सूचक शब्द है। गांधी को महात्मा के नाम से सबसे पहले 1915 में राजवैद्य जीवराम कालिदास ने संबोधित किया था। एक अन्य मत के अनुसार स्वामी श्रद्धानन्द ने

In [10]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Mon Apr  1 05:37:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:01:00.0 Off |                  Off |
| 30%   51C    P2              78W / 300W |  45103MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
device_index = PartialState().process_index
inputs = inputs.to(device_index)

NameError: name 'PartialState' is not defined

In [7]:
# record start time
start = time.time()

# We will generate all possible mcq from single page in json format
prompt = "Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text:"+" "+item+'''\n Give me output in this JSON array format:[{"question": string, "options":List[string], "answer":string}]'''
prompt_template= f'''[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> {prompt}[/INST]'''

# print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
# output = model.generate(inputs=input_ids, do_sample=True,max_length=32768)
# output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
# print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@",tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    do_sample=True,
    max_new_tokens=4096
)

# print("######",pipe(prompt_template)[0]['generated_text'])
print(pipe(prompt_template)[0]['generated_text'])


# record end time
end = time.time()

temp = end-start
print(temp)
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes

total_time_for_generation = "%dh:%dm:%ds" %(hours,minutes,seconds)

if hours == 0:
    print('The time of execution of downloading and loading the model : %dm:%ds' %(minutes,seconds))

else:
    print('The time of execution of downloading and loading the model : %dh:%dm:%ds' %(hours,minutes,seconds))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


*** Pipeline:
[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text: 
Mohandas Karamchand Gandhi (2 October 1869 – 30 January 1948) was an Indian lawyer, anti-colonial nationalist and political ethicist who employed nonviolent resistance to lead the successful campaign for India's independence from British rule. He inspired movements for civil rights and freedom across the world. The honorific Mahātmā (from Sanskrit 'great-souled, venerable'), first applied to him in South Africa in 1914, is now used throughout the world.
Born and raised in a Hindu family in coastal Gujarat, Gandhi trained in the law at the Inner Temple, London, and was called to the bar at age 22 in June 1891. After two uncertain years in India, where he was unable to start a successful law practice, he moved to South Africa in 1

In [12]:
# record start time
start = time.time()

# We will generate all possible mcq from single page in json format
prompt = "Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text:"+" "+item+'''\n Give me output in this JSON array format:[{"question": string, "options":List[string], "answer":string}]'''
prompt_template= f'''[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> {prompt}[/INST]'''

# print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
# output = model.generate(inputs=input_ids, do_sample=True,max_length=32768)
# output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
# print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@",tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    do_sample=True,
    max_new_tokens=4096
)

# print("######",pipe(prompt_template)[0]['generated_text'])
print(pipe(prompt_template)[0]['generated_text'])


# record end time
end = time.time()

temp = end-start
print(temp)
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes

total_time_for_generation = "%dh:%dm:%ds" %(hours,minutes,seconds)

if hours == 0:
    print('The time of execution of downloading and loading the model : %dm:%ds' %(minutes,seconds))

else:
    print('The time of execution of downloading and loading the model : %dh:%dm:%ds' %(hours,minutes,seconds))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


*** Pipeline:
[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text: 
Mohandas Karamchand Gandhi (2 October 1869 – 30 January 1948) was an Indian lawyer, anti-colonial nationalist and political ethicist who employed nonviolent resistance to lead the successful campaign for India's independence from British rule. He inspired movements for civil rights and freedom across the world. The honorific Mahātmā (from Sanskrit 'great-souled, venerable'), first applied to him in South Africa in 1914, is now used throughout the world.
Born and raised in a Hindu family in coastal Gujarat, Gandhi trained in the law at the Inner Temple, London, and was called to the bar at age 22 in June 1891. After two uncertain years in India, where he was unable to start a successful law practice, he moved to South Africa in 1

In [13]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Mon Apr  1 05:43:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:01:00.0 Off |                  Off |
| 30%   54C    P2              92W / 300W |  47547MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [14]:
!pip install googletrans

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 425.8 kB/s eta 0:00:000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 197.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 1.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.0 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15716 sha256=411c496d71603dd5a53b8aa5dc1dc306e9525556084aaf0e5ab5c1c377c5dd02
  Stored in directory: /home/.cache/pip/wheels/b3/81/ea/8b030407f8ebfc2f857814e086bb22ca2d4fea1a7be63652ab
Successfully built googletrans
  Attempting uninstall: idna
    Found existing installation: idna 3.6
   

In [15]:
from googletrans import Translator
translator = Translator()

In [24]:
text= '''[{
"question": "When was Mohandas Karamchand Gandhi, also known as Mahatma Gandhi, born?",
"options": ["1 October 1869", "2 October 1869", "3 October 1869", "4 October 1869"],
"answer": "2 October 1869"
},
{
"question": "What was Mahatma Gandhi's main philosophy and method of protest?",
"options": ["Violence and aggression", "Passive resistance and nonviolence", "Legal action and diplomacy", "Military intervention"],
"answer": "Passive resistance and nonviolence"
},
{
"question": "What law did Mahatma Gandhi and his followers break in the Salt March in 1830?",
"options": ["The law against public gatherings", "The law against sedition", "The law against self-rule", "The law against manufacturing salt"],
"answer": "The law against manufacturing salt"
},
{
"question": "Who assassinated Mahatma Gandhi in 1948?",
"options": ["Nathuram Godse", "Jawaharlal Nehru", "Muhammad Ali Jinnah", "Vinayak Damodar Savarkar"],
"answer": "Nathuram Godse"
},
{
"question": "What was Mahatma Gandhi's main goal in his protests and campaigns?",
"options": ["To gain power and influence for himself", "To promote Hinduism and discriminate against other religions", "To achieve independence and self-rule for India", "To enrich himself and his family"],
"answer": "To achieve independence and self-rule for India"
},
{
"question": "What was Mahatma Gandhi's main weapon in his struggles and campaigns?",
"options": ["Violence and weapons", "Legal action and diplomacy", "Nonviolence and civil disobedience", "Military intervention"],
"answer": "Nonviolence and civil disobedience"
},
{
"question": "What was the name of the movement led by Mahatma Gandhi that aimed to achieve independence and self-rule for India?",
"options": ["The Swadeshi Movement", "The Non-Cooperation Movement", "The Quit India Movement", "The Civil Disobedience Movement"],
"answer": "The Non-Cooperation Movement"
},
{
"question": "What was Mahatma Gandhi's main motivation and inspiration for his philosophy and methods of protest?",
"options": ["Personal gain and ambition", "Hate and anger towards the British", "Love and compassion towards all beings", "Fear and cowardice"],
"answer": "Love and compassion towards all beings"
},
{
"question": "What was Mahatma Gandhi's main message and legacy for future generations?",
"options": ["The use of violence and aggression to achieve one's goals", "The importance of power, wealth, and status", "The power of nonviolence, truth, and love to bring about change", "The need to discriminate against and oppress certain groups of people"],
"answer": "The power of nonviolence, truth, and love to bring about change"
},
{
"question": "What was Mahatma Gandhi's main attitude and approach towards the British Raj and the British people?",
"options": ["Hate and hostility", "Indifference and apathy", "Respect and cooperation", "Contempt and ridicule"],
"answer": "Respect and cooperation"
}
]'''

In [20]:
translations = translator.translate(text, dest='hi')
for translation in translations:
    # print(translation.origin, ' -> ', translation.text)

AttributeError: 'NoneType' object has no attribute 'group'

In [18]:
print(translator.LANGUAGES)

AttributeError: 'Translator' object has no attribute 'LANGUAGES'

In [25]:
translator.translate(text)

AttributeError: 'NoneType' object has no attribute 'group'

In [31]:
!pip install googletrans==3.1.0a0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [32]:
translator = Translator(service_urls=[
      'translate.google.com',
      'translate.google.co.in',
    ])
translator.translate('hello')

AttributeError: 'NoneType' object has no attribute 'group'

In [33]:
item = '''
मोहनदास करमचन्द गांधी (जन्म: 2 अक्टूबर 1869 -  निधन: 30 जनवरी 1948) जिन्हें महात्मा गांधी के नाम से भी जाना जाता है, भारत एवं भारतीय स्वतन्त्रता आन्दोलन के एक प्रमुख राजनैतिक एवं आध्यात्मिक नेता थे। वे सत्याग्रह (व्यापक सविनय अवज्ञा) के माध्यम से अत्याचार के प्रतिकार करने के समर्थक अग्रणी नेता थे, उनकी इस अवधारणा की नींव सम्पूर्ण अहिंसा के सिद्धान्त पर रखी गयी थी जिसने भारत सहित पूरे विश्व में जनता के नागरिक अधिकारों एवं स्वतन्त्रता के प्रति आन्दोलन के लिये प्रेरित किया। उन्हें संसार में साधारण जनता महात्मा गांधी के नाम से जानती है। संस्कृत भाषा में महात्मा अथवा 'महान आत्मा' एक सम्मान सूचक शब्द है। गांधी को महात्मा के नाम से सबसे पहले 1915 में राजवैद्य जीवराम कालिदास ने संबोधित किया था। एक अन्य मत के अनुसार स्वामी श्रद्धानन्द ने 1915 मे महात्मा की उपाधि दी थी। तीसरा मत ये है कि रवीन्द्रनाथ ठाकुर ने 12 अप्रैल 1919 को अपने एक लेख मे उन्हें महात्मा कहकर सम्बोधित किया था। । उन्हें बापू (गुजराती भाषा में બાપુ बापू अर्थात् पिता) के नाम से भी स्मरण किया जाता है। एक मत के अनुसार गांधीजी को बापू सम्बोधित करने वाले प्रथम व्यक्ति उनके साबरमती आश्रम के शिष्य थे सुभाष चन्द्र बोस ने 6 जुलाई 1944 को रंगून रेडियो से गांधी जी के नाम जारी प्रसारण में उन्हें राष्ट्रपिता कहकर सम्बोधित करते हुए आज़ाद हिन्द फौज के सैनिकों के लिये उनका आशीर्वाद और शुभकामनाएँ माँगीं थीं। प्रति वर्ष 2 अक्टूबर को उनका जन्म दिन भारत में गांधी जयन्ती के रूप में और पूरे विश्व में अन्तरराष्ट्रीय अहिंसा दिवस के रूप में मनाया जाता है।
सबसे पहले गान्धी जी ने प्रवासी वकील के रूप में दक्षिण अफ्रीका में भारतीय समुदाय के लोगों के नागरिक अधिकारों के लिये संघर्ष हेतु सत्याग्रह करना आरम्भ किया। 1915 में उनकी भारत वापसी हुई। उसके बाद उन्होंने यहाँ के किसानों, श्रमिकों और नगरीय श्रमिकों को अत्यधिक भूमि कर और भेदभाव के विरुद्ध आवाज उठाने के लिये एकजुट किया। 1921 में भारतीय राष्ट्रीय कांग्रेस की बागडोर संभालने के बाद उन्होंने देशभर में दरिद्रता से मुक्ति दिलाने, महिलाओं के अधिकारों का विस्तार, धार्मिक एवं जातीय एकता का निर्माण व आत्मनिर्भरता के लिये अस्पृश्‍यता के विरोध में अनेकों कार्यक्रम चलाये। इन सबमें विदेशी राज से मुक्ति दिलाने वाला स्वराज की प्राप्ति वाला कार्यक्रम ही प्रमुख था। गाँधी जी ने ब्रिटिश सरकार द्वारा भारतीयों पर लगाये गये लवण कर के विरोध में 1930 में नमक सत्याग्रह और इसके बाद 1942 में अंग्रेजो भारत छोड़ो आन्दोलन से विशेष विख्याति प्राप्त की। दक्षिण अफ्रीका और भारत में विभिन्न अवसरों पर कई वर्षों तक उन्हें कारागृह में भी रहना पड़ा। गांधी जी ने सभी परिस्थितियों में अहिंसा और सत्य का पालन किया और सभी को इनका पालन करने के लिये वकालत भी की। उन्होंने साबरमती आश्रम में अपना जीवन बिताया और परम्परागत भारतीय पोशाक धोती व सूत से बनी शाल पहनी जिसे वे स्वयं चरखे पर सूत कातकर हाथ से बनाते थे। उन्होंने सादा शाकाहारी भोजन खाया और आत्मशुद्धि के लिये लम्बे-लम्बे उपवास रखे। 12 फरवरी वर्ष 1948 में महात्मा गांधी के अस्थि कलश जिन 12 तटों पर विसर्जित किए गए थे, त्रिमोहिनी संगम भी उनमें से एक है |


== प्रारम्भिक जीवन ==
मोहनदास करमचन्द गान्धी का जन्म पश्चिमी भारत में वर्तमान गुजरात के एक तटीय नगर पोरबंदर नामक स्थान पर २ अक्टूबर सन् १८६९ को हुआ था। उनके पिता करमचन्द गान्धी सनातन धर्म की पंसारी जाति से सम्बन्ध रखते थे और ब्रिटिश राज के समय काठियावाड़ की एक छोटी सी रियासत (पोरबंदर) के दीवान अर्थात् प्रधान मन्त्री थे। गुजराती भाषा में गान्धी का अर्थ है पंसारी जबकि हिन्दी भाषा में गन्धी का अर्थ है इत्र फुलेल बेचने वाला जिसे अंग्रेजी में परफ्यूमर कहा जाता है। उनकी माता पुतलीबाई परनामी वैश्य समुदाय की थीं। पुतलीबाई करमचन्द की चौथी पत्नी थी। उनकी पहली तीन पत्नियाँ प्रसव के समय मर गयीं थीं। भक्ति करने वाली माता की देखरेख और उस क्षेत्र की जैन परम्पराओं के कारण युवा मोहनदास पर वे प्रभाव प्रारम्भ में ही पड़ गये थे जिसने आगे चलकर महात्मा गांधी के जीवन में महत्वपूर्ण भूमिका निभायी। इन प्रभावों में सम्मिलित थे दुर्बलों में उत्साह की भावना, शाकाहारी जीवन, आत्मशुद्धि के लिये उपवास तथा विभिन्न जातियों के लोगों के बीच सहिष्णुता। 


=== कम आयु में विवाह ===
मई १८८३ में साढे १३ वर्ष की आयु पूर्ण करते ही उनका विवाह १४ वर्ष की कस्तूर बाई मकनजी से कर दिया गया। पत्नी का पहला नाम छोटा करके कस्तूरबा कर दिया गया और उसे लोग प्यार से बा कहते थे। यह विवाह उनके माता पिता द्वारा तय किया गया व्यवस्थित बाल विवाह था जो उस समय उस क्षेत्र में प्रचलित था। परन्तु उस क्षेत्र में यही रीति थी कि किशोर दुल्हन को अपने माता पिता के घर और अपने पति से अलग अधिक समय तक रहना पड़ता था। १८८५ में जब गान्धी जी १५ वर्ष के थे तब इनकी पहली सन्तान ने जन्म लिया। किन्तु वह केवल कुछ दिन ही जीवित रही। और इसी वर्ष उनके पिता करमचन्द गांधी भी चल बसे। मोहनदास और कस्तूरबा के चार सन्तान हुईं जो सभी पुत्र थे। हरीलाल गान्धी १८८८ में, मणिलाल गान्धी १८९२ में, रामदास गान्धी १८९७ में और देवदास गांधी १९०० में जन्मे। पोरबंदर से उन्होंने मिडिल और राजकोट से हाई स्कूल किया। दोनों परीक्षाओं में शैक्षणिक स्तर वह एक साधारण छात्र रहे। मैट्रिक के बाद की परीक्षा उन्होंने भावनगर के शामलदास कॉलेज से कुछ समस्या के साथ उत्तीर्ण की। जब तक वे वहाँ रहे अप्रसन्न ही रहे क्योंकि उनका परिवार उन्हें बैरिस्टर बनाना चाहता था।


=== विदेश में शिक्षा व विदेश में ही वकालत ===
अपने १९वें जन्मदिन से लगभग एक महीने पहले ही ४ सितम्बर १८८८ को गांधी यूनिवर्सिटी कॉलेज लन्दन में कानून की पढाई करने और बैरिस्टर बनने के लिये इंग्लैंड चले गये। भारत छोड़ते समय जैन भिक्षु बेचारजी के समक्ष हिन्दुओं को मांस, शराब तथा संकीर्ण विचारधारा को त्यागने के लिए अपनी अपनी माता जी को दिए गये एक वचन ने उनके शाही राजधानी लंदन में बिताये गये समय को काफी प्रभावित किया। हालांकि गांधी जी ने अंग्रेजी रीति रिवाजों का अनुभव भी किया जैसे उदाहरण के तौर पर नृत्य कक्षाओं में जाने आदि का। फिर भी वह अपनी मकान मालकिन द्वारा मांस एवं पत्ता गोभी को हजम.नहीं कर सके। उन्होंने कुछ शाकाहारी भोजनालयों की ओर इशारा किया। अपनी माता की इच्छाओं के बारे में जो कुछ उन्होंने पढा था उसे सीधे अपनाने की बजाय उन्होंने बौद्धिकता से शाकाहारी भोजन का अपना भोजन स्वीकार किया। उन्होंने शाकाहारी समाज की सदस्यता ग्रहण की और इसकी कार्यकारी समिति के लिये उनका चयन भी हो गया जहाँ उन्होंने एक स्थानीय अध्याय की नींव रखी। बाद में उन्होने संस्थाएँ गठित करने में महत्वपूर्ण अनुभव का परिचय देते हुए इसे श्रेय दिया। वे जिन शाकाहारी लोगों से मिले उनमें से कुछ थियोसोफिकल सोसायटी के सदस्य भी थे। इस सोसाइटी की स्थापना १८७५ में विश्व बन्धुत्व को प्रबल करने के लिये की गयी थी और इसे बौद्ध धर्म एवं सनातन धर्म के साहित्य के अध्ययन के लिये समर्पित किया गया था।
उन्हों लोगों ने गांधी जी को श्रीमद्भगवद्गीता पढ़ने के लिये प्रेरित किया। हिन्दू, ईसाई, बौद्ध, इस्लाम और अन्य धर्मों .के बारे में पढ़ने से पहले गांधी ने धर्म में विशेष रुचि नहीं दिखायी। इंग्लैंड और वेल्स बार एसोसिएशन में वापस बुलावे पर वे भारत लौट आये किन्तु बम्बई में वकालत करने में उन्हें कोई खास सफलता नहीं मिली। बाद में एक हाई स्कूल शिक्षक के रूप में अंशकालिक नौकरी का प्रार्थना पत्र अस्वीकार कर दिये जाने पर उन्होंने जरूरतमन्दों के लिये मुकदमे की अर्जियाँ लिखने के लिये राजकोट को ही अपना स्थायी मुकाम बना लिया। परन्तु एक अंग्रेज अधिकारी की मूर्खता के कारण उन्हें यह कारोबार भी छोड़ना पड़ा। अपनी आत्मकथा में उन्होंने इस घटना का वर्णन अपने बड़े भाई की ओर से परोपकार की असफल कोशिश के रूप में किया है। यही वह कारण था जिस वजह से उन्होंने सन् १८९३ में एक भारतीय फर्म से नेटाल दक्षिण अफ्रीका में, जो उन दिनों ब्रिटिश साम्राज्य का भाग होता था, एक वर्ष के करार पर वकालत का कारोवार स्वीकार कर लिया।


== दक्षिण अफ्रीका (1893-1914) में नागरिक अधिकारों के आन्दोलन ==
दक्षिण अफ्रीका में गान्धी को भारतीयों पर भेदभाव का सामना करना पड़ा। आरम्भ में उन्हें प्रथम श्रेणी कोच की वैध टिकट होने के बाद तीसरी श्रेणी के डिब्बे में जाने से इन्कार करने के लिए ट्रेन से बाहर फेंक दिया गया था। इतना ही नहीं पायदान पर शेष यात्रा करते हुए एक यूरोपियन यात्री के अन्दर आने पर चालक की मार भी झेलनी पड़ी। उन्होंने अपनी इस यात्रा में अन्य भी कई कठिनाइयों का सामना किया। अफ्रीका में कई होटलों को उनके लिए वर्जित कर दिया गया। इसी तरह ही बहुत सी घटनाओं में से एक यह भी थी जिसमें अदालत के न्यायाधीश ने उन्हें अपनी पगड़ी उतारने का आदेश दिया था जिसे उन्होंने नहीं माना। ये सारी घटनाएँ गान्धी के जीवन में एक मोड़ बन गईं और विद्यमान सामाजिक अन्याय के प्रति जागरुकता का कारण बनीं तथा सामाजिक सक्रियता की व्याख्या करने में मददगार सिद्ध हुईं। दक्षिण अफ्रीका में भारतीयों पर हो रहे अन्याय को देखते हुए गान्धी ने अंग्रेजी साम्राज्य के अन्तर्गत अपने देशवासियों के सम्मान तथा देश में स्वयं अपनी स्थिति के लिए प्रश्न उठाये।


== १९०६ के ज़ुलु युद्ध में भूमिका ==
१९०६ में, ज़ुलु (Zulu) दक्षिण अफ्रीका में नए चुनाव कर के लागू करने के बाद दो अंग्रेज अधिकारियों को मार डाला गया। बदले में अंग्रेजों ने जूलू के खिलाफ युद्ध छेड़ दिया। गांधी जी ने भारतीयों को भर्ती करने के लिए ब्रिटिश अधिकारियों को सक्रिय रूप से प्रेरित किया। उनका तर्क था अपनी नागरिकता के दावों को कानूनी जामा पहनाने के लिए भारतीयों को युद्ध प्रयासों में सहयोग देना चाहिए। तथापि, अंग्रेजों ने अपनी सेना में भारतीयों को पद देने से इंकार कर दिया था। इसके बावजूद उन्होने गांधी जी के इस प्रस्ताव को मान लिया कि भारतीय घायल अंग्रेज सैनिकों को उपचार के लिए स्टेचर पर लाने के लिए स्वैच्छा पूर्वक कार्य कर सकते हैं। इस कोर की बागडोर गांधी ने थामी।२१ जुलाई (July 21), १९०६ को गांधी जी ने भारतीय जनमत इंडियन ओपिनिय (Indian Opinion) में लिखा कि २३ भारतीय निवासियों के विरूद्ध चलाए गए आप्रेशन के संबंध में प्रयोग द्वारा नेटाल सरकार के कहने पर एक कोर का गठन किया गया है।दक्षिण अफ्रीका में भारतीय लोगों से इंडियन ओपिनियन में अपने कॉलमों के माध्‍यम से इस युद्ध में शामिल होने के लिए आग्रह किया और कहा, यदि सरकार केवल यही महसूस करती हे कि आरक्षित बल बेकार हो रहे हैं तब वे इसका उपयोग करेंगे और असली लड़ाई के लिए भारतीयों का प्रशिक्षण देकर इसका अवसर देंगे।गांधी की राय में, १९०६ का मसौदा अध्यादेश भारतीयों की स्थिति में किसी निवासी के नीचे वाले स्तर के समान लाने जैसा था। इसलिए उन्होंने सत्याग्रह (Satyagraha), की तर्ज पर "काफिर (Kaffir)s " .का उदाहरण देते हुए भारतीयों से अध्यादेश का विरोध करने का आग्रह किया। उनके शब्दों में, " यहाँ तक कि आधी जातियां और काफिर जो हमसे कम आधुनिक हैं ने भी सरकार का विरोध किया है। पास का नियम उन पर भी लागू होता है किंतु वे पास नहीं दिखाते हैं।


== भारतीय स्वतंत्रता संग्राम के लिए संघर्ष (१९१६ -१९४५) ==
गांधी १९१५ में दक्षिण अफ्रीका से भारत में रहने के लिए लौट आए। उन्होंने भारतीय राष्ट्रीय कांग्रेस के अधिवेशनों पर अपने विचार व्य‍क्त किए, लेकिन उनके विचार भारत के मुख्य मुद्दों, राजनीति तथा उस समय के कांग्रेस दल के प्रमुख भारतीय नेता गोपाल कृष्ण गोखले  पर ही आधारित थे जो एक सम्मानित नेता थे।


=== चंपारण और खेड़ा ===
गांधी की पहली बड़ी उपलब्धि १९१८ में चम्पारन सत्याग्रह और खेड़ा सत्याग्रह  में मिली हालांकि अपने निर्वाह के लिए जरूरी खाद्य फसलों की बजाए नील (indigo) नकद पैसा देने वाली खाद्य फसलों की खेती वाले आंदोलन भी महत्वपूर्ण रहे। जमींदारों (अधिकांश अंग्रेज) की ताकत से दमन हुए भारतीयों को नाममात्र भरपाई भत्ता दिया गया जिससे वे अत्यधिक गरीबी से घिर गए। गांवों को बुरी तरह गंदा और अस्वास्थ्यकर (unhygienic); और शराब, अस्पृश्यता और पर्दा से बांध दिया गया। अब एक विनाशकारी अकाल के कारण शाही कोष की भरपाई के लिए अंग्रेजों ने दमनकारी कर लगा दिए जिनका बोझ दिन प्रतिदिन बढता ही गया। यह स्थिति निराशजनक थी। खेड़ा (Kheda), गुजरात में भी यही समस्या थी। गांधी जी ने वहां एक आश्रम (ashram) बनाया जहाँ उनके बहुत सारे समर्थकों और नए स्वेच्छिक कार्यकर्ताओं को संगठित किया गया। उन्होंने गांवों का एक विस्तृत अध्ययन और सर्वेक्षण किया जिसमें प्राणियों पर हुए अत्याचार के भयानक कांडों का लेखाजोखा रखा गया और इसमें लोगों की अनुत्पादकीय सामान्य अवस्था को भी शामिल किया गया था। ग्रामीणों में विश्‍वास पैदा करते हुए उन्होंने अपना कार्य गांवों की सफाई करने से आरंभ किया जिसके अंतर्गत स्कूल और अस्पताल बनाए गए और उपरोक्त वर्णित बहुत सी सामाजिक बुराईयों को समाप्त करने के लिए ग्रामीण नेतृत्व प्रेरित किया।
लेकिन इसके प्रमुख प्रभाव उस समय देखने को मिले जब उन्हें अशांति फैलाने के लिए पुलिस ने गिरफ्तार किया और उन्हें प्रांत छोड़ने के लिए आदेश दिया गया। हजारों की तादाद में लोगों ने विरोध प्रदर्शन किए ओर जेल, पुलिस स्टेशन एवं अदालतों के बाहर रैलियां निकालकर गांधी जी को बिना शर्त रिहा करने की मांग की। गांधी जी ने जमींदारों के खिलाफ़ विरोध प्रदर्शन और हड़तालों को का नेतृत्व किया जिन्होंने अंग्रेजी सरकार के मार्गदर्शन में उस क्षेत्र के गरीब किसानों को अधिक क्षतिपूर्ति मंजूर करने तथा खेती पर नियंत्रण, राजस्व में बढोतरी को रद्द करना तथा इसे संग्रहित करने वाले एक समझौते पर हस्ताक्षर किए। इस संघर्ष के दौरान ही, गांधी जी को जनता ने बापू पिता और महात्मा (महान आत्मा) के नाम से संबोधित किया। खेड़ा में सरदार पटेल ने अंग्रेजों के साथ विचार विमर्श के लिए किसानों का नेतृत्व किया जिसमें अंग्रेजों ने राजस्व संग्रहण से मुक्ति देकर सभी कैदियों को रिहा कर दिया गया था। इसके परिणामस्वरूप, गांधी की ख्याति देश भर में फैल गई।


=== असहयोग आन्दोलन ===

गांधी जी ने असहयोग, अहिंसा तथा शांतिपूर्ण प्रतिकार को अंग्रेजों के खिलाफ़ शस्त्र के रूप में उपयोग किया। पंजाब में अंग्रेजी फोजों द्वारा भारतीयों पर जलियावांला नरसंहार जिसे अमृतसर नरसंहार के नाम से भी जाना जाता है ने देश को भारी आघात पहुँचाया जिससे जनता में क्रोध और हिंसा की ज्वाला भड़क उठी। गांधीजी ने ब्रिटिश राज तथा भारतीयों द्वारा ‍प्रतिकारात्मक रवैया दोनों की की। उन्होंने ब्रिटिश नागरिकों तथा दंगों के शिकार लोगों के प्रति संवेदना व्यक्त की तथा पार्टी के आरम्भिक विरोध के बाद दंगों की भंर्त्सना की। गांधी जी के भावनात्मक भाषण के बाद अपने सिद्धांत की वकालत की कि सभी हिंसा और बुराई को न्यायोचित नहीं ठहराया जा सकता है। किंतु ऐसा इस नरसंहार और उसके बाद हुई हिंसा से गांधी जी ने अपना मन संपूर्ण सरकार आर भारतीय सरकार के कब्जे वाली संस्थाओं पर संपूर्ण नियंत्रण लाने पर केंद्रित था जो जल्‍दी ही स्वराज अथवा संपूर्ण व्यक्तिगत, आध्‍यात्मिक एवं राजनैतिक आजादी में बदलने वाला था।

दिसम्बर १९२१ में गांधी जो भारतीय राष्ट्रीय कांग्रेस.का कार्यकारी अधिकारी नियुक्त किया गया। उनके नेतृत्व में कांग्रेस को स्वराज.के नाम वाले एक नए उद्देश्‍य के साथ संगठित किया गया। पार्दी में सदस्यता सांकेतिक शुल्क का भुगताने पर सभी के लिए खुली थी। पार्टी को किसी एक कुलीन संगठन की न बनाकर इसे राष्ट्रीय जनता की पार्टी बनाने के लिए इसके अंदर अनुशासन में सुधार लाने के लिए एक पदसोपान समिति गठित की गई। गांधी जी ने अपने अहिंसात्मक मंच को स्वदेशी नीति — में शामिल करने के लिए विस्तार किया जिसमें विदेशी वस्तुओं विशेषकर अंग्रेजी वस्तुओं का बहिष्कार करना था। इससे जुड़ने वाली उनकी वकालत का कहना था कि सभी भारतीय अंग्रेजों द्वारा बनाए वस्त्रों की अपेक्षा हमारे अपने लोगों द्वारा हाथ से बनाई गई खादी पहनें। गांधी जी ने स्वतंत्रता आंदोलन को सहयोग देने के लिएपुरूषों और महिलाओं को प्रतिदिन खादी के लिए सूत कातने में समय बिताने के लिए कहा। यह अनुशासन और समर्पण लाने की ऐसी नीति थी जिससे अनिच्छा और महत्वाकाक्षा को दूर किया जा सके और इनके स्थान पर उस समय महिलाओं को शामिल किया जाए जब ऐसे बहुत से विचार आने लगे कि इस प्रकार की गतिविधियां महिलाओं के लिए सम्मानजनक नहीं हैं। इसके अलावा गांधी जी ने ब्रिटेन की शैक्षिक संस्थाओं तथा अदालतों का बहिष्कार और सरकारी नौकरियों को छोड़ने का तथा सरकार से प्राप्त तमगों और सम्मान (honours) को वापस लौटाने का भी अनुरोध किया।
असहयोग को दूर-दूर से अपील और सफलता मिली जिससे समाज के सभी वर्गों की जनता में जोश और भागीदारी बढ गई। फिर जैसे ही यह आंदोलन अपने शीर्ष पर पहुँचा वैसे फरवरी १९२२ में इसका अंत चौरी-चोरा, उत्तरप्रदेश में भयानक द्वेष के रूप में अंत हुआ। आंदोलन द्वारा हिंसा का रूख अपनाने के डर को ध्‍यान में रखते हुए और इस पर विचार करते हुए कि इससे उसके सभी कार्यों पर पानी फिर जाएगा, गांधी जी ने व्यापक असहयोग के इस आंदोलन को वापस ले लिया। गांधी पर गिरफ्तार किया गया १० मार्च, १९२२, को राजद्रोह के लिए गांधी जी पर मुकदमा चलाया गया जिसमें उन्हें छह साल कैद की सजा सुनाकर जैल भेद दिया गया। १८ मार्च, १९२२ से लेकर उन्होंने केवल २ साल ही जैल में बिताए थे कि उन्हें फरवरी १९२४ में आंतों के ऑपरेशन के लिए रिहा कर दिया गया।
गांधी जी के एकता वाले व्यक्तित्व के बिना इंडियन नेशनल कांग्रेस उसके जेल में दो साल रहने के दौरान ही दो दलों में बंटने लगी जिसके एक दल का नेतृत्व सदन में पार्टी की भागीदारी के पक्ष वाले चित्त रंजन दास तथा मोतीलाल नेहरू ने किया तो दूसरे दल का नेतृत्व इसके विपरीत चलने वाले चक्रवर्ती राजगोपालाचार्य और सरदार वल्लभ भाई पटेल ने किया। इसके अलावा, हिंदुओं और मुसलमानों के बीच अहिंसा आंदोलन की चरम सीमा पर पहुँचकर सहयोग टूट रहा था। गांधी जी ने इस खाई को बहुत से साधनों से भरने का प्रयास किया जिसमें उन्होंने १९२४ की बसंत में सीमित सफलता दिलाने वाले तीन सप्ताह का उपवास करना भी शामिल था।


=== स्वराज और नमक सत्याग्रह (नमक मार्च) ===
गांधी जी सक्रिय राजनीति से दूर ही रहे और १९२० की अधिकांश अवधि तक वे स्वराज पार्टी और इंडियन नेशनल कांग्रेस के बीच खाई को भरने में लगे रहे और इसके अतिरिक्त वे अस्पृश्यता, शराब, अज्ञानता और गरीबी के खिलाफ आंदोलन छेड़ते भी रहे। उन्होंने पहले १९२८ में लौटे .एक साल पहले अंग्रेजी सरकार ने सर जॉन साइमन के नेतृत्व में एक नया संवेधानिक सुधार आयोग बनाया जिसमें एक भी सदस्य भारतीय नहीं था। इसका परिणाम भारतीय राजनैतिक दलों द्वारा बहिष्कार निकला। दिसम्बर १९२८ में गांधी जी ने कलकत्ता में आयोजित कांग्रेस के एक अधिवेशन में एक प्रस्ताव रखा जिसमें भारतीय साम्राज्य को सत्ता प्रदान करने के लिए कहा गया था अथवा ऐसा न करने के बदले अपने उद्देश्य के रूप में संपूर्ण देश की आजादी के लिए असहयोग आंदोलन का सामना करने के लिए तैयार रहें। गांधी जी ने न केवल युवा वर्ग सुभाष चंद्र बोस तथा जवाहरलाल नेहरू जैसे पुरूषों द्वारा तत्काल आजादी की मांग के विचारों को फलीभूत किया बल्कि अपनी स्वयं की मांग को दो साल की बजाए एक साल के लिए रोक दिया। अंग्रेजों ने कोई जवाब नहीं दिया।.नहीं ३१ दिसम्बर १९२९, भारत का झंडा फहराया गया था लाहौर में है।२६ जनवरी १९३० का दिन लाहौर में भारतीय स्वतंत्रता दिवस के रूप में इंडियन नेशनल कांग्रेस ने मनाया। यह दिन लगभग प्रत्येक भारतीय संगठनों द्वारा भी मनाया गया। इसके बाद गांधी जी ने मार्च १९३० में नमक पर कर लगाए जाने के विरोध में नया सत्याग्रह चलाया जिसे १२ मार्च से ६ अप्रेल तक नमक आंदोलन के याद में ४०० किलोमीटर (२४८ मील) तक का सफर अहमदाबाद से दांडी, गुजरात तक चलाया गया ताकि स्वयं नमक उत्पन्न किया जा सके। समुद्र की ओर इस यात्रा में हजारों की संख्‍या में भारतीयों ने भाग लिया। भारत में अंग्रेजों की पकड़ को विचलित करने वाला यह एक सर्वाधिक सफल आंदोलन था जिसमें अंग्रेजों ने ८०,००० से अधिक लोगों को जेल भेजा।
लार्ड एडवर्ड इरविन द्वारा प्रतिनिधित्व वाली सरकार ने गांधी जी के साथ विचार विमर्श करने का निर्णय लिया। यह इरविन गांधी की सन्धि मार्च १९३१ में हस्ताक्षर किए थे। सविनय अवज्ञा आंदोलन को बंद करने के लिए ब्रिटिश सरकार ने सभी राजनैतिक कैदियों को रिहा करने के लिए अपनी रजामन्दी दे दी। इस समझौते के परिणामस्वरूप गांधी को भारतीय राष्ट्रीय कांग्रेस के एकमात्र प्रतिनिधि के रूप में लंदन में आयोजित होने वाले गोलमेज सम्मेलन में भाग लेने के लिए आमन्त्रित किया गया। यह सम्मेलन गांधी जी और राष्ट्रीयवादी लोगों के लिए घोर निराशाजनक रहा, इसका कारण सत्ता का हस्तांतरण करने की बजाय भारतीय कीमतों एवं भारतीय अल्पसंख्‍यकों पर केन्द्रित होना था। इसके अलावा, लार्ड इरविन के उत्तराधिकारी लार्ड विलिंगटन, ने राष्‍ट्रवादियों के आंदोलन को नियंत्रित एवं कुचलने का एक नया अभियान आरम्भ करदिया। गांधी फिर से गिरफ्तार कर लिए गए और सरकार ने उनके अनुयाईयों को उनसे पूर्णतया दूर रखते हुए गांधी जी द्वारा प्रभावित होने से रोकने की कोशिश की। लेकिन, यह युक्ति सफल नहीं थी।


=== दलित आंदोलन और निश्चय दिवस ===

१९३२ में, दलित नेता और प्रकांड विद्वान डॉ॰ बाबासाहेब अम्बेडकर के चुनाव प्रचार के माध्यम से, सरकार ने अछूतों को एक नए संविधान के अंतर्गत अलग निर्वाचन मंजूर कर दिया। इसके विरोध में दलित हतों के विरोधी गांधी जी ने सितंबर १९३२ में छ: दिन का अनशन ले लिया जिसने सरकार को सफलतापूर्वक दलित से राजनैतिक नेता बने पलवंकर बालू द्वारा की गई मध्‍यस्ता वाली एक समान व्यवस्था को अपनाने पर बल दिया। अछूतों के जीवन को सुधारने के लिए गांधी जी द्वारा चलाए गए इस अभियान की शुरूआत थी। गांधी जी ने इन अछूतों को हरिजन का नाम दिया जिन्हें वे भगवान की संतान मानते थे। ८ मई १९३३ को गांधी जी ने हरिजन आंदोलन में मदद करने के लिए आत्म शुद्धिकरण का २१ दिन तक चलने वाला उपवास किया। यह नया अभियान दलितों को पसंद नहीं आया तथापि वे एक प्रमुख नेता बने रहे। डॉ॰ अम्बेडकर ने गांधी जी द्वारा हरिजन शब्द का उपयोग करने की स्पष्ट निंदा की, कि दलित सामाजिक रूप से अपरिपक्व हैं और सुविधासंपन्न जाति वाले भारतीयों ने पितृसत्तात्मक भूमिका निभाई है। अम्बेडकर और उसके सहयोगी दलों को भी महसूस हुआ कि गांधी जी दलितों के राजनीतिक अधिकारों को कम आंक रहे हैं। हालांकि गांधी जी एक वैश्य जाति में पैदा हुए फिर भी उन्होनें इस बात पर जोर दिया कि वह डॉ॰ अम्बेडकर जैसे दलित मसिहा के होते हुए भी वह दलितों के लिए आवाज उठा सकता है। भारतीय स्वतंत्रता संग्राम के दिनों में हिन्दुस्तान की सामाजिक बुराइयों में में छुआछूत एक प्रमुख बुराई थी जिसके के विरूद्ध महात्मा गांधी और उनके अनुयायी संघर्षरत रहते थे। उस समय देश के प्रमुख मंदिरों में हरिजनों का प्रवेश पूर्णतः प्रतिबंधित था। केरल राज्य का जनपद त्रिशुर दक्षिण भारत की एक प्रमुख धार्मिक नगरी है। यहीं एक प्रतिष्ठित मंदिर है गुरुवायुर मंदिर, जिसमें कृष्ण भगवान के बालरूप के दर्शन कराती भगवान गुरूवायुरप्पन की मूर्ति स्थापित है। आजादी से पूर्व अन्य मंदिरों की भांति इस मंदिर में भी हरिजनों के प्रवेश पर पूर्ण प्रतिबंध था।
केरल के गांधी समर्थक श्री केलप्पन ने महात्मा की आज्ञा से इस प्रथा के विरूद्ध आवाज उठायी और अंततः इसके लिये सन् १९३३ ई0 में सविनय अवज्ञा प्रारम्भ की गयी। मन्दिर के ट्रस्टियों को इस बात की ताकीद की गयी कि नये वर्ष का प्रथम दिवस अर्थात १ जनवरी १९३४ को अंतिम निश्चय दिवस के रूप में मनाया जायेगा और इस तिथि पर उनके स्तर से कोई निश्चय न होने की स्थिति मे महात्मा गांधी तथा श्री केलप्पन द्वारा आन्दोलनकारियों के पक्ष में आमरण अनशन किया जा सकता है। इस कारण गुरूवायूर मन्दिर के ट्रस्टियो की ओर से बैठक बुलाकर मन्दिर के उपासको की राय भी प्राप्त की गयी। बैठक मे ७७ प्रतिशत उपासको के द्वारा दिये गये बहुमत के आधार पर मन्दिर में हरिजनों के प्रवेश को स्वीकृति दे दी गयी और इस प्रकार १ जनवरी १९३४ से केरल के श्री गुरूवायूर मन्दिर में किये गये निश्चय दिवस की सफलता के रूप में हरिजनों के प्रवेश को सैद्वांतिक स्वीकृति मिल गयी। गुरूवायूर मन्दिर जिसमें आज भी गैर हिन्दुओं का प्रवेश वर्जित है तथापि कई धर्मों को मानने वाले भगवान भगवान गुरूवायूरप्पन के परम भक्त हैं। महात्मा गांधी की प्रेरणा से जनवरी माह के प्रथम दिवस को निश्चय दिवस के रूप में मनाया गया और किये गये निश्चय को प्राप्त किया गया। ।
१९३४ की गर्मियों में, उनकी जान लेने के लिए उन पर तीन असफल प्रयास किए गए थे।
जब कांग्रेस पार्टी के चुनाव लड़ने के लिए चुना और संघीय योजना के अंतर्गत सत्ता स्वीकार की तब गांधी जी ने पार्टी की सदस्यता से इस्तीफा देने का निर्णय ले लिया। वह पार्टी के इस कदम से असहमत नहीं थे किंतु महसूस करते थे कि यदि वे इस्तीफा देते हैं तब भारतीयों के साथ उसकी लोकप्रियता पार्टी की सदस्यता को मजबूत करने में आसानी प्रदान करेगी जो अब तक कम्यूनिसटों, समाजवादियों, व्यापार संघों, छात्रों, धार्मिक नेताओं से लेकर व्यापार संघों और विभिन्न आवाजों के बीच विद्यमान थी। इससे इन सभी को अपनी अपनी बातों के सुन जाने का अवसर प्राप्त होगा। गांधी जी राज के लिए किसी पार्टी का नेतृत्व करते हुए प्रचार द्वारा कोई ऐसा लक्ष्‍य सिद्ध नहीं करना चाहते थे जिसे राज के साथ अस्थायी तौर पर राजनैतिक व्यवस्‍था के रूप में स्वीकार कर लिया जाए।
गांधी जी नेहरू प्रेजीडेन्सी और कांग्रेस के लाहौर अधिवेशन के साथ ही १९३६ में भारत लौट आए। हालांकि गांधी की पूर्ण इच्छा थी कि वे आजादी प्राप्त करने पर अपना संपूर्ण ध्‍यान केन्द्रित करें न कि भारत के भविष्य के बारे में अटकलों पर। उसने कांग्रेस को समाजवाद को अपने उद्देश्‍य के रूप में अपनाने से नहीं रोका। १९३८ में पार्टी अध्यक्ष पद के लिए चुने गए सुभाष बोस के साथ गांधी जी के मतभेद थे। बोस के साथ मतभेदों में गांधी के मुख्य बिंदु बोस की लोकतंत्र में प्रतिबद्धता की कमी तथा अहिंसा में विश्वास की कमी थी। बोस ने गांधी जी की आलोचना के बावजूद भी दूसरी बार जीत हासिल की किंतु कांग्रेस को उस समय छोड़ दिया जब सभी भारतीय नेताओं ने गांधी जी द्वारा लागू किए गए सभी सिद्धातों का परित्याग कर दिया गया।


=== द्वितीय विश्व युद्ध और भारत छोड़ो आन्दोलन ===
द्वितीय विश्व युद्ध १९३९ में जब छिड़ने नाजी जर्मनी आक्रमण पोलैंड.आरम्भ में गांधी जी ने अंग्रेजों के प्रयासों को अहिंसात्मक नैतिक सहयोग देने का पक्ष लिया किंतु दूसरे कांग्रेस के नेताओं ने युद्ध में जनता के प्रतिनिधियों के परामर्श लिए बिना इसमें एकतरफा शामिल किए जाने का विरोध किया। कांग्रेस के सभी चयनित सदस्यों ने सामूहिक तौर पर अपने पद से इस्तीफा दे दिया। लंबी चर्चा के बाद, गांधी ने घोषणा की कि जब स्वयं भारत को आजादी से इंकार किया गया हो तब लोकतांत्रिक आजादी के लिए बाहर से लड़ने पर भारत किसी भी युद्ध के लिए पार्टी नहीं बनेगी। जैसे जैसे युद्ध बढता गया गांधी जी ने आजादी के लिए अपनी मांग को अंग्रेजों को भारत छोड़ो आन्दोलन नामक विधेयक देकर तीव्र कर दिया। यह गांधी तथा कांग्रेस पार्टी का सर्वाधिक स्पष्ट विद्रोह था जो भारत देश से अंग्रेजों को खदेड़ने पर लक्षित था।
गांधी जी के दूसरे नम्बर पर बैठे जवाहरलाल नेहरू की पार्टी के कुछ सदस्यों तथा कुछ अन्य राजनैतिक भारतीय दलों ने आलोचना की जो अंग्रेजों के पक्ष तथा विपक्ष दोनों में ही विश्‍वास रखते थे। कुछ का मानना था कि अपने जीवन काल में अथवा मौत के संघर्ष में अंग्रेजों का विरोध करना एक नश्वर कार्य है जबकि कुछ मानते थे कि गांधी जी पर्याप्त कोशिश नहीं कर रहे हैं। भारत छोड़ो इस संघर्ष का सर्वाधिक शक्तिशाली आंदोलन बन गया जिसमें व्यापक हिंसा और गिरफ्तारी हुई। पुलिस की गोलियों से हजारों की संख्‍या में स्वतंत्रता सेनानी या तो मारे गए या घायल हो गए और हजारों गिरफ्तार कर लिए गए। गांधी और उनके समर्थकों ने स्पष्ट कर दिया कि वह युद्ध के प्रयासों का समर्थन तब तक नहीं देंगे तब तक भारत को तत्‍काल आजादी न दे दी जाए। उन्होंने स्पष्ट किया कि इस बार भी यह आन्दोलन बन्द नहीं होगा यदि हिंसा के व्यक्तिगत कृत्यों को मूर्त रूप दिया जाता है। उन्होंने कहा कि उनके चारों ओर अराजकता का आदेश असली अराजकता से भी बुरा है। उन्होंने सभी कांग्रेसियों और भारतीयों को अहिंसा के साथ करो या मरो  (अंग्रेजी में डू ऑर डाय) के द्वारा अन्तिम स्वतन्त्रता के लिए अनुशासन बनाए रखने को कहा।
गांधी जी और कांग्रेस कार्यकारणी समिति के सभी सदस्यों को अंग्रेजों द्वारा मुबंई में ९ अगस्त १९४२ को गिरफ्तार कर लिया गया। गांधी जी को पुणे के आंगा खां महल में दो साल तक बंदी बनाकर रखा गया। यही वह समय था जब गांधी जी को उनके निजी जीवन में दो गहरे आघात लगे। उनका ५० साल पुराना सचिव महादेव देसाई ६ दिन बाद ही दिल का दौरा पड़ने से मर गए और गांधी जी के १८ महीने जेल में रहने के बाद २२ फरवरी १९४४ को उनकी पत्नी कस्तूरबा गांधी का देहांत हो गया। इसके छ: सप्ताह बाद गांधी जी को भी मलेरिया का भयंकर शिकार होना पड़ा। उनके खराब स्वास्थ्‍य और जरूरी उपचार के कारण ६ मई १९४४ को युद्ध की समाप्ति से पूर्व ही उन्हें रिहा कर दिया गया। राज उन्हें जेल में दम तोड़ते हुए नहीं देखना चाहते थे जिससे देश का क्रोध बढ़ जाए। हालांकि भारत छोड़ो आंदोलन को अपने उद्देश्य में आशिंक सफलता ही मिली लेकिन आंदोलन के निष्‍ठुर दमन ने १९४३ के अंत तक भारत को संगठित कर दिया। युद्ध के अंत में, ब्रिटिश ने स्पष्ट संकेत दे दिया था कि संत्ता का हस्तांतरण कर उसे भारतीयों के हाथ में सोंप दिया जाएगा। इस समय गांधी जी ने आंदोलन को बंद कर दिया जिससे कांग्रेसी नेताओं सहित लगभग १००,००० राजनैतिक बंदियों को रिहा कर दिया गया।


=== स्वतंत्रता और भारत का विभाजन ===
गांधी जी ने १९४६ में कांग्रेस को ब्रिटिश केबीनेट मिशन (British Cabinet Mission) के प्रस्ताव को ठुकराने का परामर्श दिया क्योकि उसे मुस्लिम बाहुलता वाले प्रांतों के लिए प्रस्तावित समूहीकरण के प्रति उनका गहन संदेह होना था इसलिए गांधी जी ने प्रकरण को एक विभाजन के पूर्वाभ्यास के रूप में देखा। हालांकि कुछ समय से गांधी जी के साथ कांग्रेस द्वारा मतभेदों वाली घटना में से यह भी एक घटना बनी (हालांकि उसके नेत्त्व के कारण नहीं) चूंकि नेहरू और पटेल जानते थे कि यदि कांग्रेस इस योजना का अनुमोदन नहीं करती है तब सरकार का नियंत्रण मुस्लिम लीग के पास चला जाएगा। १९४८ के बीच लगभग ५००० से भी अधिक लोगों को हिंसा के दौरान मौत के घाट उतार दिया गया। गांधी जी किसी भी ऐसी योजना के खिलाफ थे जो भारत को दो अलग अलग देशों में विभाजित कर दे। भारत में रहने वाले बहुत से हिंदुओं और सिक्खों एवं मुस्लिमों का भारी बहुमत देश के बंटवारे के पक्ष में था। इसके अतिरिक्त मुहम्मद अली जिन्ना, मुस्लिम लीग के नेता ने, पश्चिम पंजाब, सिंध, उत्तर पश्चिम सीमांत प्रांत और पूर्वी बंगाल में व्यापक सहयोग का परिचय दिया। व्यापक स्तर पर फैलने वाले हिंदु मुस्लिम लड़ाई को रोकने के लिए ही कांग्रेस नेताओं ने बंटवारे की इस योजना को अपनी मंजूरी दे दी थी। कांगेस नेता जानते थे कि गांधी जी बंटवारे का विरोध करेंगे और उसकी सहमति के बिना कांग्रेस के लिए आगे बझना बसंभव था चुकि पाटर्ठी में गांधी जी का सहयोग और संपूर्ण भारत में उनकी स्थिति मजबूत थी। गांधी जी के करीबी सहयोगियों ने बंटवारे को एक सर्वोत्तम उपाय के रूप में स्वीकार किया और सरदार पटेल ने गांधी जी को समझाने का प्रयास किया कि नागरिक अशांति वाले युद्ध को रोकने का यही एक उपाय है। मज़बूर गांधी ने अपनी अनुमति दे दी।
उन्होंने उत्तर भारत के साथ-साथ बंगाल में भी मुस्लिम और हिंदु समुदाय के नेताओं के साथ गर्म रवैये को शांत करने के लिए गहन विचार विमर्श किया। १९४७ के भारत-पाकिस्तान युद्ध के बावजूद उन्हें उस समय परेशान किया गया जब सरकार ने पाकिस्तान को विभाजन परिषद द्वारा बनाए गए समझौते के अनुसार ५५ करोड़ रू0 न देने का निर्णय लियाथा। सरदार पटेल जैसे नेताओं को डर था कि पाकिस्तान इस धन का उपयोग भारत के खिलाफ़ जंग छेड़ने में कर सकता है। जब यह मांग उठने लगी कि सभी मुस्लिमों को पाकिस्तान भेजा जाए और मुस्लिमों और हिंदु नेताओं ने इस पर असंतोष व्य‍क्त किया और एक दूसरे के साथ समझौता करने से मना करने से गांधी जी को गहरा सदमा पहुंचा। उन्होंने दिल्ली में अपना पहला आमरण अनशन आरंभ किया जिसमें साम्प्रदायिक हिंसा को सभी के लिए तत्काल समाप्त करने और पाकिस्तान को 55 करोड़ रू0 का भुगतान करने के लिए कहा गया था। गांधी जी को डर था कि पाकिस्तान में अस्थिरता और असुरक्षा से भारत के प्रति उनका गुस्सा और बढ़ जाएगा तथा सीमा पर हिंसा फैल जाएगी। उन्हें आगे भी डर था कि हिंदु और मुस्लिम अपनी शत्रुता को फिर से नया कर देंगे और उससे नागरिक युद्ध हो जाने की आशंका बन सकती है। जीवन भर गांधी जी का साथ देने वाले सहयोगियों के साथ भावुक बहस के बाद गांधी जी ने बात का मानने से इंकार कर दिया और सरकार को अपनी नीति पर अडिग रहना पड़ा तथा पाकिस्तान को भुगतान कर दिया। हिंदु मुस्लिम और सिक्ख समुदाय के नेताओं ने उन्हें विश्‍वास दिलाया कि वे हिंसा को भुला कर शांति लाएंगे। इन समुदायों में राष्ट्रीय स्वयंसेवक संघ और हिंदू महासभा शामिल थे। इस प्रकार गांधी जी ने संतरे का जूस पीकर अपना अनशन तोड़ दिया।


== हत्या ==

मैनचेस्टर गार्जियन, १८ फरवरी, १९४८, की गलियों से ले जाते हुआ दिखाया गया था।
३० जनवरी, १९४८, गांधी की उस समय नाथूराम गोडसे द्वारा गोली मारकर हत्या कर दी गई जब वे नई दिल्ली के बिड़ला भवन (बिरला हाउस के मैदान में रात चहलकदमी कर रहे थे। गांधी का हत्यारा नाथूराम गौड़से हिन्दू राष्ट्रवादी थे जिनके कट्टरपंथी हिंदु महासभा के साथ संबंध थे जिसने गांधी जी को पाकिस्तान को भुगतान करने के मुद्दे को लेकर भारत को कमजोर बनाने के लिए जिम्मेदार ठहराया था। गोड़से और उसके उनके सह षड्यंत्रकारी नारायण आप्टे को बाद में केस चलाकर सजा दी गई तथा १५ नवंबर १९४९ को इन्हें फांसी दे दी गई। राज घाट, नई दिल्ली, में गांधी जी के स्मारक पर "देवनागरी में  हे राम " लिखा हुआ है। ऐसा व्यापक तौर पर माना जाता है कि जब गांधी जी को गोली मारी गई तब उनके मुख से निकलने वाले ये अंतिम शब्द थे। हालांकि इस कथन पर विवाद उठ खड़े हुए हैं।जवाहरलाल नेहरू ने रेडियो के माध्यम से राष्ट्र को संबोधित किया :
गांधी जी की राख को एक अस्थि-रख दिया गया और उनकी सेवाओं की याद दिलाने के लिए संपूर्ण भारत में ले जाया गया। इनमें से अधिकांश को इलाहाबाद में संगम पर १२ फरवरी १९४८ को जल में विसर्जित कर दिया गया किंतु कुछ को अलग पवित्र रूप में रख दिया गया। १९९७ में, तुषार गाँधी ने बैंक में नपाए गए एक अस्थि-कलश की कुछ सामग्री को अदालत के माध्यम से, इलाहाबाद में संगम नामक स्थान पर जल में विसर्जित कर दिया। ३० जनवरी२००८ को दुबई में रहने वाले एक व्यापारी द्वारा गांधी जी की राख वाले एक अन्य अस्थि-कलश को मुंबई संग्रहालय में भेजने के उपरांत उन्हें गिरगाम चौपाटी नामक स्थान पर जल में विसर्जित कर दिया गया। एक अन्य अस्थि कलश आगा खान जो पुणे में है, (जहाँ उन्होंने १९४२ से कैद करने के लिए किया गया था १९४४) वहां समाप्त हो गया और दूसरा आत्मबोध फैलोशिप झील मंदिर में लॉस एंजिल्स. रखा हुआ है। इस परिवार को पता है कि इस पवित्र राख का राजनीतिक उद्देश्यों के लिए दुरूपयोग किया जा सकता है लेकिन उन्हें यहां से हटाना नहीं चाहती हैं क्योंकि इससे मन्दिरों . को तोड़ने का खतरा पैदा हो सकता है।
12 फरवरी वर्ष 1948 में महात्मा गांधी के अस्थि कलश जिन 12 तटों पर विसर्जित किए गए थे, त्रिमोहिनी संगम भी उनमें से एक है |


== गांधी के सिद्धान्त ==


=== सत्य ===
गांधी जी ने अपना जीवन सत्य, या सच्चाई की व्यापक खोज में समर्पित कर दिया। उन्होंने इस लक्ष्य को प्राप्त करने के लिए अपनी स्वयं की गल्तियों और खुद पर प्रयोग करते हुए सीखने की कोशिश की। उन्होंने अपनी आत्मकथा को सत्य के प्रयोग का नाम दिया। 
गांधी जी ने कहा कि सबसे महत्वपूर्ण लड़ाई लड़ने के लिए अपने दुष्टात्माओं , भय और असुरक्षा जैसे तत्वों पर विजय पाना है। गांधी जी ने अपने विचारों को सबसे पहले उस समय संक्षेप में व्य‍क्त किया जब उन्होंने कहा भगवान ही सत्य है| बाद में उन्होंने अपने इस कथन को सत्य ही भगवान है में बदल दिया। इस प्रकार , सत्य में गांधी के दर्शन है " परमेश्वर "|""""जय परमात्मा""""                                      एक बार वायसराय लार्ड कर्जेन ने कहा था कि सत्य की कल्पना भारत में यूरोप से आई है।  इस पर गांधी जी बड़े ही क्षुब्ध हुए और उन्होंने वायसराय को लिखा, “आपका विचार गलत है।  भारत में सत्य की प्रतिष्ठा बहुत प्राचीन काल से चली आ रही है। सत्य परमात्मा का रूप माना जाता है।”


=== अहिंसा ===
हालांकि गांधी जी अहिंसा के सिद्धांत के प्रवर्तक बिल्कुल नहीं थे फिर भी इसे बड़े पैमाने  पर राजनैतिक क्षेत्र में इस्तेमाल करने वाले वे पहले व्यक्ति थे। अहिंसा (non-violence),  अहिंसा (ahimsa) और अप्रतिकार  (non-resistance) का भारतीय धार्मिक विचारों में एक लंबा इतिहास है और इसके हिंदु, बौद्ध, जैन, यहूदी और ईसाई समुदायों में बहुत सी अवधारणाएं हैं। गांधी जी ने अपनी आत्मकथा द स्टोरी ऑफ़ माय एक्सपेरिमेंट्स विथ ट्रुथ "  (The Story of My Experiments with Truth)में दर्शन और अपने जीवन के मार्ग का वर्णन किया है। उन्हें कहते हुए बताया गया था :

जब मैं निराश होता हूं तब मैं याद करता हूं कि हालांकि इतिहास सत्य का मार्ग होता है किंतु प्रेम  इसे सदैव जीत लेता है। यहां अत्याचारी और हतयारे भी हुए हैं और कुछ समय के लिए वे अपराजय लगते थे किंतु अंत में उनका पतन ही होता है -इसका सदैव विचार करें। 
" मृतकों, अनाथ तथा बेघरों के लिए इससे क्या फर्क पड़ता है कि स्वतंत्रता और लोकतंत्र के पवित्र नाम के नीचे संपूर्णवाद का पागल विनाश छिपा है। 
एक आंख के लिए दूसरी आंख पूरी दुनिया को अंधा बना देगी।
मरने के लिए मैरे पास बहुत से कारण है किंतु मेरे पास किसी को मारने का कोई भी कारण नहीं है "
इन सिद्धातों को लागू करने में गांधी जी ने इन्हें दुनिया को दिखाने के लिए सर्वाधिक तार्किक सीमा पर ले जाने से भी मुंह नहीं मोड़ा जहां सरकार, पुलिस और सेनाए भी अहिंसात्मक बन गईं थीं।" फॉर पसिफिस्ट्स." नामक पुस्तक से उद्धरण लिए गए हैं। 

विज्ञान का युद्ध किसी व्यक्ति को तानाशाही , शुद्ध और सरलता की ओर ले जाता है।  अहिंसा का विज्ञान अकेले ही किसी व्यक्ति को शुद्ध लोकतंत्र के मार्ग की ओर ले जा सकता है।प्रेम पर आधारित शक्ति सजा के डर से उत्पन्न शक्ति से हजार गुणा अधिक और स्थायी होती है। यह कहना निन्दा करने जैसा होगा कि कि अहिंसा का अभ्यास केवल व्यक्तिगत तौर पर किया जा सकता है और व्यक्तिवादिता वाले देश इसका कभी भी अभ्यास नहीं कर सकते हैं। शुद्ध अराजकता का निकटतम दृष्टिकोण अहिंसा पर आधारित लोकतंत्र होगा; संपूर्ण अहिंसा के आधार पर संगठित और चलने वाला कोई समाज शुद्ध अराजकता वाला समाज होगा।
मैं ने भी स्वीकार किया कि एक अहिंसक राज्य में भी पुलिस बल की जरूरत अनिवार्य हो सकती है। पुलिस रैंकों का गठन अहिंसा में विश्‍वास रखने वालों से किया जाएगा। लोग उनकी हर संभव मदद करेंगे और  आपसी सहयोग के माध्यम से वे किसी भी उपद्रव का आसानी से सामना कर लेंगे ...श्रम और पूंजी तथा हड़तालों के बीव हिंसक झगड़े बहुत कम होंगे और अहिंसक राज्यों में तो बहुत कम होंगे क्योंकि अहिंसक समाज की बाहुलता का प्रभाव समाज में प्रमुख तत्वों का सम्मान करने के लिए महान होगा। इसी प्रकार साम्प्रदायिक अव्यवस्था के लिए कोई जगह नहीं होगी;। 
शांति एवं अव्यवस्था के समय सशस्त्र सैनिकों की तरह सेना का कोई अहिंसात्मक कार्य उनका यह कर्तव्य होगा कि वे विजय दिलाने वाले समुदायों को एकजुट करें जिसमें शांति का प्रसार, तथा ऐसी गतिविधियों का समावेश हो जो किसी भी व्यक्ति को उसके चर्च अथवा खंड में संपर्क बनाए रखते हुए अपने साथ मिला लें। इस प्रकार की सैना को किसी भी आपात स्थिति से लड़ने के लिए तैयार रहना चाहिए तथा भीड़ के क्रोध को शांत करने के लिए उसके पास मरने के लिए सैनिकों की पर्याप्त नफरी भी होनी चाहिए;सत्याग्रह (सत्यबल) के बिग्रेड को प्रत्येक गांव तथा शहर तक भवनों के प्रत्येक ब्लॉक में संगठित किया जा सकता हैयदि अहिंसात्मक समाज पर हमला किया जाता है तब अहिंसा के दो मार्ग खुलते हैं। अधिकार पाने के लिए हमलावर से सहयोग न करें बल्कि समर्पण करने की अपेक्षा मृत्यु को गले लगाना पसंद करें। दूसरा तरीका होगा ऐसी जनता द्वारा  अहिंसक प्रतिरोध करना हो सकता है जिन्हें अहिंसक तरीके से प्रशिक्षित किया गया हो ...इस अप्रत्याशित प्रदर्शन की अनंत राहों पर आदमियों और महिलाओं को हमलावर की इच्छा  लिए आत्मसमर्पण करने की बजाए आसानी से मरना अच्छा लगता है और अंतंत: उसे तथा उसकी सैनिक बहादुरी के समक्ष पिघलना जरूर पड़ता है। ऐसे किसी देश अथवा समूह जिसने अहिंसा को अपनी अंतिम नीति बना लिया है उसे परमाणु बम भी अपना दास नहीं बना सकता है। उस देश में अहिंसा का स्तर खुशी-खुशी गुजरता है तब वह प्राकृतिक तौर पर इतना अधिक बढ़ जाता है कि उसे सार्वभोमिक आदर मिलने लगता है। 
इन विचारों के अनुरूप १९४० में जब नाजी जर्मनी द्वारा अंग्रेजों के द्वीपों पर किए गए हमले आसन्न दिखाई दिए तब गांधी जी ने अंग्रेजों को शांति और युद्ध में अहिंसा की निम्नलिखित नीति का अनुसरण करने को कहा। 

मैं आपसे हथियार रखने के लिए कहना पसंद करूंगा क्योंकि ये आपको अथवा मानवता को बचाने में बेकार हैं।आपको हेर हिटलर और सिगनोर मुसोलिनी को आमंत्रित करना होगा कि उन्हें देशों से जो कुछ चाहिए आप उन्हें अपना अधिकार कहते हैं।यदि इन सज्जनों को अपने घर पर रहने का चयन करना है तब आपको उन्हें खाली करना होगा।यदि वे तुम्हें आसानी से रास्ता नहीं देते हैं तब आप अपने आपको , पुरूषों को महिलाओं को और बच्चों की बलि देने की अनुमति देंगे किंतु अपनी  निष्ठा के प्रति झुकने से इंकार करेंगे। 
१९४६ में युद्ध के बाद दिए गए एक साक्षात्कार में उन्होंने इससे भी आगे एक विचार का प्रस्तुतीकरण किया। 

यहूदियों को अपने लिए स्वयं कसाई का चाकू दे देना चाहिए था।उन्हें अपने आप को समुद्री चट्टानों से समुद्र के अंदर फैंक देना चाहिए था। 
फिर भी गांधी जी को पता था कि इस प्रकार के अहिंसा के स्तर को अटूट विश्वास और साहस की जरूरत होगी और इसके लिए उसने महसूस कर लिया था कि यह हर किसी के पास नहीं होता है। इसलिए उन्होंने प्रत्येक व्यक्ति को परामर्श दिया कि उन्हें अहिंसा को अपने पास रखने की जरूरत नहीं है खास तौर पर उस समय जब इसे कायरता के संरक्षण के लिए उपयोग में किया गया हो। 

गांधी जी ने अपने सत्याग्रह आंदोलन  में ऐसे लोगों को दूर ही रखा जो हथियार उठाने से डरते थे अथवा प्रतिरोध करने में स्वयं की अक्षमता का अनुभव करते थे। उन्होंने लिखा कि :

मैं मानता हूं कि जहां डरपोक और हिंसा में से किसी एक को चुनना हो तो मैं हिंसा के पक्ष में अपनी राय दूंगा।
प्रत्येक सभा पर मैं तब तक चेतावनी दोहराता रहता था जब तक वन्हें यह अहसास नहीं हो जाता है कि वे एक ऐसे अंहिसात्मक बल के अधिकार में आ गए हैं जिसके अधिकार में वे पहले भी थे और वे उस प्रयोग के आदि हो चुके थे और उनका मानना था कि उन्हें अहिंसा से कुछ लेना देना नहीं हैं तथा फिर से हथियार उठा लिए थे। खुदाई खिदमतगार  (Khudai Khidmatgar)के बारे में ऐसा कभी नहीं कहना चाहिए कि जो एक बार इतने बहादुर थे कि बादशाह खान  (Badshah Khan)के प्रभाव में अब वे डरपोक बन गए। वीरता केवल अच्छे निशाने वालों में ही नहीं होती है बल्कि मृत्यु को हरा देने वालों में तथा  अपनी छातियों को गोली  खाने के लिए सदा तैयार रहने वालों में भी होती है।


=== शाकाहारी रवैया ===
बाल्यावस्था में गांधी को मांस खाने का अनुभव मिला। ऐसा उनकी उत्तराधिकारी जिज्ञासा के कारण ही था जिसमें उसके उत्साहवर्धक मित्र शेख मेहताब का भी  योगदान था।
वेजीटेरियनिज्म का विचार भारत की हिंदु और जैन प्रथाओं में कूट-कूट कर भरा हुआ था तथा उनकी मातृभूमि गुजरात में ज्यादातर हिंदु शाकाहारी ही थे। इसी तरह जैन भी थे। गांधी का परिवार भी इससे अछूता नहीं था। पढाई के लिए लंदन आने से पूर्व गांधी जी ने अपनी माता पुतलीबाई और अपने चाचा बेचारजी स्वामी से एक वायदा किया था कि वे मांस खाने, शराब पीने से तथा संकीणता से दूर रहेंगे। उन्होने अपने वायदे रखने के लिए उपवास किए और ऐसा करने से सबूत कुछ ऐसा मिला जो भोजन करने से नहीं मिल सकता था, उन्होंने अपने जीवन पर्यन्त दर्शन के लिए आधार जो प्राप्त कर लिया था। जैसे जैसे गांधी जी व्यस्क होते गए वे पूर्णतया शाकाहारी बन गए। उन्होंने द मोरल बेसिस ऑफ वेजीटेरियनिज्म तथा इस विषय पर बहुत सी लेख भी लिखें हैं जिनमें से कुछ लंदन वेजीटेरियन सोसायटी के प्रकाशन द वेजीटेरियन में प्रकाशित भी हुए हैं। गांधी जी स्वयं इस अवधि में बहुत सी महान विभूतियों से प्रेरित हुए और लंदन वेजीटेरियन सोसायटी के चैयरमेन डॉ० जोसिया ओल्डफील्ड के मित्र बन गए। 
हेनरी स्टीफन साल्ट
हेनरी स्टीफन ‍साल्ट  (Henry Stephens Salt)की कृतियों को पढने और प्रशंसा करने के बाद  युवा मोहनदास गांधी शाकाहारी प्रचारक से मिले और उनके साथ पत्राचार किया। गांधी जी ने लंदन में रहते समय और उसके बाद शाकाहारी भोजन की वकालत करने में काफी समय बिताया। गांधी जी का कहना था कि शाकाहारी भोजन न केवल शरीर की जरूरतों को पूरा करता है बल्कि यह आर्थिक प्रयोजन की भी पूर्ति करता है जो मांस से होती है और फिर भी मांस अनाज, सब्जियों और फलों से अधिक मंहगा होता है। इसके अलावा  कई भारतीय जो  आय  कम होने  की वजह से संघर्ष कर रहे थे, उस समय  जो शाकाहारी  के रूप में दिखाई दे रहे थे वह आध्यात्मिक परम्परा ही नहीं व्यावहारिकता के कारण भी था.वे बहुत देर तक खाने से परहेज रखते थे , और राजनैतिक विरोध के रूप में उपवास रखते थे उन्होंने अपनी मृत्यु तक खाने से इनकार किया जब तक उनका मांग पुरा नही होता उनकी आत्मकथा में यह नोट किया  गया है  कि शाकाहारी होना ब्रह्मचर्य में गहरी प्रतिबद्धता होने की शुरूआती सीढ़ी है, बिना कुल नियंत्रण ब्रह्मचर्य में उनकी  सफलता लगभग असफल है |
गाँधी जी शुरू से फलाहार (frutarian), करते थे  लेकिन अपने चिकित्सक की सलाह से  बकरी का दूध पीना शुरू किया था|वे कभी भी दुग्ध-उत्पाद का सेवन नहीं करते थे क्योंकि पहले उनका मानना था की  दूध मनुष्य का प्राकृतिक आहार नहीं होता और उन्हें गाय के चीत्कार से घृणा (cow blowing), थी  और सबसे महत्वपूर्ण कारण था शपथ जो उन्होंने अपनी स्वर्गीय माँ से किया था|


=== ब्रह्मचर्य ===
जब गाँधी जी सोलह वर्ष के हुए तब उनके  पिताश्री का स्वास्थ्य बहुत ख़राब था. उनके पिता की बीमारी के समय वे सदा उपस्थित रहते थे क्योंकि वे अपने माता-पिता के प्रति अत्यंत समर्पित थे. यद्यपि, गाँधी जी को कुछ समय की मुक्ति देने के लिए एक दिन उनके चाचा जी आए वे आराम के लिए शयनकक्ष पहुंचे जहाँ उनकी शारीरिक अभिलाषाएं जागृत हुई और उन्होंने अपनी पत्नी से प्रेम किया नौकर के जाने के पश्चात् थोडी ही देर में ख़बर आई की गाँधी के पिता का अभी अभी देहांत हो गया है.गाँधी जी को जबरदस्त अपराध अनुभव हुआ और इसके लिए वे अपने आप को कभी क्षमा नहीं कर सकते थे उन्होंने इस घटना का उल्लेख दोहरी शर्म  में  किया इस घटना का  गाँधी  पर महत्वपूर्ण प्रभाव पड़ा  और वे  ३६ वर्ष की आयु में  ब्रह्मचर्य (celibate) की और  मुड़ने लगे, जबकि उनका विवाह हो चुका था.यह निर्णय ब्रहमचर्य  (Brahmacharya) के दर्शन से पुरी तरह प्रभावित था आध्यात्मिक और व्यवहारिक शुद्धता बड़े पैमाने पर ब्रह्मचर्य और वैराग्यवाद (asceticism) से जुदा होता है.गाँधी ने ब्रह्मचर्य को भगवान् के करीब आने  और अपने को पहचानने का प्राथमिक  आधार के रूप में देखा था  अपनी आत्मकथा में वे अपनी बचपन की दुल्हन कस्तूरबा (Kasturba) के साथ  अपनी  कामेच्छा और इर्ष्या के संघर्षो को बतातें  हैं  उन्होंने महसूस किया कि यह उनका व्यक्तिगत दायित्व है की उन्हें ब्रह्मचर्य रहना है ताकि वे बजाय हवस  के प्रेम को सिख पायें गाँधी के लिए, ब्रह्मचर्य का अर्थ था  "इन्द्रियों के अंतर्गत  विचारों, शब्द और कर्म पर नियंत्रण".


=== सादगी ===
गाँधी जी का मानना था कि अगर  एक व्यक्ति समाज सेवा में कार्यरत है तो उसे साधारण जीवन (simple life) की ओर ही बढ़ना चाहिए जिसे वे ब्रह्मचर्य के लिए आवश्यक मानते थे।  उनकी सादगी (simplicity) ने पश्चमी जीवन शैली को त्यागने पर मजबूर करने लगा और वे दक्षिण  अफ्रीका में फैलने लगे थे इसे वे "ख़ुद को शुन्य के स्थिति में लाना" कहते हैं जिसमे अनावश्यक खर्च, साधारण जीवन शैली को अपनाना  और अपने वस्त्र स्वयं  धोना आवश्यक है.एक अवसर पर जन्मदार की और से सम्मुदय के लिए  उनकी अनवरत सेवा के लिए  प्रदान किए गए उपहार को भी वापस कर देते हैं.गाँधी सप्ताह में एक दिन मौन धारण करते थे.उनका मानना था कि बोलने के परहेज से उन्हें आतंरिक शान्ति (inner peace) मिलती है। उनपर यह प्रभाव हिंदू मौन सिद्धांत का है, (संस्कृत: - मौन) और शान्ति (संस्कृत: -शान्ति)वैसे दिनों में वे कागज पर लिखकर दूसरों के साथ संपर्क करते थे। 37 वर्ष की आयु से साढ़े तीन वर्षों तक गांधी जी ने अख़बारों को पढ़ने से इंकार कर दिया जिसके जवाब में उनका कहना था कि जगत की आज जो स्थिर अवस्था है उसने उसे अपनी स्वयं की आंतरिक अशांति की तुलना में अधिक भ्रमित किया है। 
जॉन रस्किन (John Ruskin) की अन्टू दिस लास्ट (Unto This Last), पढ़ने के बाद उन्होंने अपने जीवन शैली में परिवर्तन करने का फैसला किया तथा एक समुदाय बनाया जिसे अमरपक्षी अवस्थापन कहा जाता था।
दक्षिण अफ्रीका, जहाँ  से उन्होंने  वकालत पूरी की थी तथा  धन और सफलता के साथ जुड़े थे। वहां  से लौटने के पश्चात्  उन्होंने  पश्चमी शैली के वस्त्रों का  त्याग किया। उन्होंने भारत के सबसे गरीब इंसान के द्वारा जो वस्त्र पहने जाते हैं उसे स्वीकार किया, तथा घर में बने हुए कपड़े (खादी) पहनने की वकालत भी की। गाँधी और उनके अनुयायियों ने अपने कपड़े सूत के द्वारा ख़ुद बुनने के अभ्यास को अपनाया और दूसरो को भी ऐसा करने के लिए प्रोत्साहित किया हालाँकि भारतीय श्रमिक बेरोज़गारी के कारण बहुधा आलसी थे, वे अक्सर अपने कपड़े उन  औद्योगिक निर्माताओं से खरीदते थे जिसका उद्देश्य ब्रिटिश हितों को पुरा करना था।  गाँधी का मत था कि अगर भारतीय अपने कपड़े ख़ुद बनाने लगे, तो यह भारत में बसे ब्रिटिशों को आर्थिक झटका लगेगा। फलस्वरूप,  बाद में चरखा (spinning wheel) को भारतीय राष्ट्रीय झंडा में शामिल किया गया। अपने साधारण जीवन को दर्शाने के लिए उन्होंने बाद में अपनी बाकी जीवन में धोती पहनने का निर्णय लिया।


=== विश्वास ===
गाँधी का जन्म हिंदू धर्म में हुआ, उनके पूरे जीवन में  अधिकतर सिद्धान्तों की उत्पति हिंदुत्व से हुआ। साधारण हिंदू की तरह वे सारे धर्मों को समान रूप से मानते थे, और इसलिए उन्होंने धर्म-परिवर्तन के सारे तर्कों एवं प्रयासों को अस्वीकृत किया। वे ब्रह्मज्ञान के जानकार थे और सभी प्रमुख धर्मो को विस्तार से पढ़ते थे। उन्होंने हिंदू धर्म के बारे में निम्नलिखित बातें कही हैं-

हिंदू धर्म, इसे जैसा मैंने समझा है, मेरी आत्मा को पूरी तरह तृप्त करता है, मेरे प्राणों को आप्लावित कर देता है,... जब संदेह मुझे घेर लेता है, जब निराशा मेरे सम्मुख आ खड़ी होती है, जब क्षितिज पर प्रकाश की एक किरण भी दिखाई नहीं देती, तब मैं 'भगवद्गीता' की शरण में जाता हूँ और उसका कोई-न-कोई श्लोक मुझे सांत्वना दे जाता है, और मैं घोर विषाद के बीच भी तुरंत मुस्कुराने लगता हूं। मेरे जीवन में अनेक बाह्य त्रासदियां घटी है और यदि उन्होंने मेरे ऊपर कोई प्रत्यक्ष या अमिट प्रभाव नहीं छोड़ा है तो मैं इसका श्रेय 'भगवद्गीता' के उपदेशों को देता हूँ।गाँधी ने भगवद् गीता की व्याख्या गुजराती में भी की है.महादेव देसाई ने गुजराती पाण्डुलिपि का अतिरिक्त भूमिका तथा विवरण के साथ  अंग्रेजी में अनुवाद किया है गाँधी के द्वारा  लिखे गए प्राक्कथन के साथ इसका प्रकाशन १९४६ में हुआ था .गाँधी का मानना था कि प्रत्येक धर्म के मूल में सत्य और प्रेम होता है। उनका कहना है कि  'कुरान', 'बाइबिल', 'जेन्द-अवेस्ता', 'तालमुड', अथवा 'गीता' किसी भी माध्यम से देखिए, हम सबका ईश्वर एक ही है, और वह सत्य तथा प्रेम स्वरूप है। ढोंग, कुप्रथा आदि पर भी उन्होंने सभी धर्मो के सिद्धान्तों के प्रति सवालिया रुख अख्तियार किया। वे एक अथक समाज सुधारक थे। उनकी कुछ टिप्पणियां विभिन्न धर्मो के सन्दर्भ में इस प्रकार हैं :

किंतु जिस तरह मैं ईसाई धर्म को स्वीकार नहीं कर सका, उसी तरह हिंदू धर्म की संपूर्णता के विषय में अथवा उसके सर्वोपरि होने के विषय में भी मैं उस समय निश्चय नहीं कर सका। हिंदू धर्म की त्रुटियां मेरी आँखों के सामने तैरती रहती थीं। यदि अस्पृश्यता हिंदू धर्म का अंग है जो ऐसा जान पड़ा कि वह एक सड़ा और बाद में जोड़ा गया अंग है। अनेक संप्रदायों और अनेक जाति-भेदों का होना भी मेरी समझ में नहीं आता था। केवल वेद ही ईश्वर-प्रणीत हैं, इस बात का क्या अर्थ है। यदि वेद ईश्वर-प्रणीत हैं, तो बाइबिल और कुरान क्यों नहीं हैं?मुझे प्रभावित करने के लिए जिस तरह ईसाई मित्र प्रयत्नशील थे, उसी प्रकार मुसलमान मित्र भी प्रयत्नशील थे। अब्दुल्ला सेठ मुझे इस्लाम का अध्ययन करने के लिए प्रोत्साहित करते रहते थे। उसकी खूबियों की चर्चा तो वे किया ही करते थे।जितनी जल्दी हम नैतिक आधार से हारेंगे उतनी ही जल्दी हममे धार्मिक युद्ध समाप्त हो जायेगा ऐसी कोई बात नहीं है कि धर्म नैतिकता के ऊपर हो। उदाहरण के तौर पर कोई मनुष्य असत्यवादी, क्रूर या असंयमी हो और वह यह दावा करे कि परमेश्वर उसके साथ हैं कभी हो ही नहीं सकता।मुहम्मद की बातें ज्ञान का खजाना है, सिर्फ़ मुसलमानों के लिए ही नहीं बल्कि पूरी मानव जाति के लिए।बाद में उनसे जब पूछा गया कि क्या तुम हिंदू हो, उन्होंने  कहा: 

"हाँ मैं हूँ। मैं एक ईसाई, मुस्लिम, बौद्ध और यहूदी भी हूँ।"एक दूसरे के प्रति गहरा आदर भाव होने के बावजूद गाँधी और रवीन्द्रनाथ ठाकुर एक से अधिक बार लम्बी बहस में लगे रहे। ये वाद-विवाद दोनों के दार्शनिक मतभेद को दर्शाते हैं। ये दोनों ही उस समय के प्रसिद्ध भारतीय चिन्तक थे। १५ जनवरी १९३४ को बिहार में    
आये भीषण भूकंप के संदर्भ में गांधी जी ने सर्वप्रथम २४ जनवरी १९३४ को तिन्नवल्ली की सार्वजनिक सभा में कहा था कि भले ही आप मुझे अंधविश्वासी ही कहें, मगर मुझ जैसा आदमी यही मानेगा कि भगवान ने हमें हमारे पापों का दंड देने के लिए इस भयंकर भूकंप को भेजा है।... बिहार का यह संकट तो केवल शरीर का नाश करने वाला है, मगर अस्पृश्यता-जनित संकट तो हमारी आत्मा को नष्ट कर रहा है। इसलिए बिहार की इस विपत्ति से हमें यह सीख लेनी चाहिए कि अपनी चंद शेष सांसो के रहते हुए हम अस्पृश्यता के इस कलंक से मुक्ति पाकर अपने-आपको अपने सिरजनहार के समक्ष स्वच्छ हृदय लेकर उपस्थित होने योग्य बना लें। २५ जनवरी को भी उन्होंने लोगों को इस घटना के संदर्भ में अस्पृश्यता को महापाप मानकर त्यागने की प्रेरणा दी तथा २६ जनवरी को मदुरा में व्यापारियों द्वारा आयोजित स्वागत समारोह में उन्होंने कहा कि मुझे तो यह विश्वास होता जा रहा है कि हम पर यह विपत्ति अस्पृश्यता के इस महापाप के फलस्वरूप ही आई है। मैं आपसे विनती करता हूँ कि आप मेरी बात पर मन ही मन हँस कर ऐसा न सोचें कि मैं तो आपके अंधविश्वास की वृत्ति को जगा रहा हूँ। मैं ऐसा कुछ नहीं कर रहा हूँ।... मैं भले ही अंधविश्वासी कहा जाऊँ, लेकिन जिस बात को मैं अपने हृदय की गहराई में महसूस कर रहा हूँ, उसे आपसे कहे बिना रह नहीं सकता।... यदि आप भी मेरी ही तरह इस बात में विश्वास करते हों तो आप निर्णय करने में तत्परता बरतेंगे और यह मानेंगे कि आज हम जैसी अस्पृश्यता बरतते हैं वैसी अस्पृश्यता का विधान हिंदू शास्त्रों में नहीं है। आप मेरे इस विचार से सहमत होंगे कि किसी भी मनुष्य को अस्पृश्य मानना एक भयंकर पाप है। मनुष्य का अहंकार ही उससे ऐसा कहता है कि वह अन्य लोगों से श्रेष्ठ है।गांधी जी के इस विचार को अंधविश्वास को बढ़ावा देने में सक्षम होने के कारण अविवेकपूर्ण मानते हुए रवीन्द्रनाथ टैगोर ने लिखा कि भौतिक आपदाओं का निश्चित और एकमात्र मूल कारण कुछ खास भौतिक तथ्यों के योग से होता है।... हमारे पाप अथवा त्रुटियाँ चाहे कितनी भी भयंकर क्यों न हों इनमें इतना बल नहीं है कि सृष्टि के ढांचे को तहस-नहस कर सकें।इसके उत्तर में गांधी जी ने विस्तारपूर्वक अपना दृष्टिकोण स्पष्ट करते हुए लिखा कि ब्रह्मांड में हो रही प्राकृतिक घटनाओं और मानवीय व्यवहार के पारस्परिक संबंध में मेरा जीवंत विश्वास है और उस विश्वास के कारण मैं ईश्वर के अधिकाधिक निकट आता गया हूँ, मुझमें विनम्रता आई है और मैं अपने को ईश्वर के सम्मुख उपस्थित करने के लिए अधिकाधिक तैयार होता गया हूँ। यदि मैं अपने घोर अज्ञान के कारण उस विश्वास का उपयोग अपने विरोधियों की निंदा करने के लिए करूँ तो निश्चय ही ऐसा विश्वास पतनकारी अंधविश्वास बन जायेगा।


=== सन्दर्भ ===


== लेखन कार्य एवं प्रकाशन ==
गाँधी जी पर हवाई बहस करने एवं मनमाना निष्कर्ष निकालने की अपेक्षा यह युगीन आवश्यकता ही नहीं वरन् समझदारी का तकाजा भी है कि गाँधीजी की मान्यताओं के आधार की प्रामाणिकता को ध्यान में रखा जाए। सामान्य से विशिष्ट तक -- सभी संदर्भों में दस्तावेजी रूप प्राप्त गाँधी जी का लिखा-बोला प्रायः प्रत्येक शब्द अध्ययन के लिए उपलब्ध है। इसलिए स्वभावतः यह आवश्यक है कि इनके मद्देनजर ही किसी बात को यथोचित मुकाम की ओर ले जाया जाए। लिखने की प्रवृत्ति गाँधीजी में आरम्भ से ही थी। अपने संपूर्ण जीवन में उन्होंने वाचिक की अपेक्षा कहीं अधिक लिखा है। चाहे वह टिप्पणियों के रूप में हो या पत्रों के रूप में। कई पुस्तकें लिखने के अतिरिक्त उन्होंने कई पत्रिकाएँ भी निकालीं और उनमें प्रभूत लेखन किया। उनके महत्त्वपूर्ण लेखन कार्य को निम्न बिंदुओं के अंतर्गत देखा जा सकता है:


=== पत्रिकाएँ ===
गाँधी जी एक सफल लेखक थे। कई दशकों तक वे अनेक पत्रों का संपादन कर चुके थे जिसमे हरिजन, इंडियन ओपिनियन, यंग इंडिया आदि सम्मिलित हैं। जब वे भारत में वापस आए तब उन्होंने 'नवजीवन' नामक मासिक पत्रिका निकाली। बाद में नवजीवन का प्रकाशन हिन्दी में भी हुआ। इसके अलावा उन्होंने लगभग हर रोज व्यक्तियों और समाचार पत्रों को पत्र लिखा


=== प्रमुख प्रकाशित पुस्तकें ===
गाँधी जी द्वारा मौलिक रूप से लिखित पुस्तकें चार हैं-- हिंद स्वराज, दक्षिण अफ्रीका के सत्याग्रह का इतिहास, सत्य के प्रयोग (आत्मकथा), तथा गीता पदार्थ कोश सहित संपूर्ण गीता की टीका। गाँधी जी आमतौर पर गुजराती में लिखते थे, परन्तु अपनी किताबों का हिन्दी और अंग्रेजी में भी अनुवाद करते या करवाते थे।


==== हिंद स्वराज ====
हिंद स्वराज (मूलतः हिंद स्वराज्य) नामक अल्पकाय ग्रंथरत्न गाँधीजी ने इंग्लैंड से लौटते समय किल्डोनन कैसिल नामक जहाज पर गुजराती में लिखा था और उनके दक्षिण अफ्रीका पहुँचने पर इंडियन ओपिनियन में प्रकाशित हुआ था। आरम्भ के बारह अध्याय 11 दिसंबर 1909 के अंक में और शेष 18 दिसंबर 1909 के अंक में। पुस्तक रूप में इसका प्रकाशन पहली बार जनवरी 1910 में हुआ था और भारत में बम्बई सरकार द्वारा 24 मार्च 1910 को इसके प्रचार पर प्रतिबंध लगा दिया गया था। बम्बई सरकार की इस कार्रवाई का जवाब गाँधीजी ने इसका अंग्रेजी अनुवाद प्रकाशित करके दिया। इस पुस्तक के परिशिष्ट-1 में पुस्तक में प्रतिपादित विषय के अधिक अध्ययन के लिए 20 पुस्तकों की सूची भी दी गयी है जिससे गाँधीजी के तत्कालीन अध्ययन के विस्तार की एक झलक भी मिलती है।


==== दक्षिण अफ्रीका के सत्याग्रह का इतिहास ====
'दक्षिण अफ्रीका के सत्याग्रह का इतिहास' मूलतः गुजराती में 'दक्षिण आफ्रिकाना सत्याग्रहनो इतिहास' नाम से 26 नवंबर 1923 को, जब वे यरवदा जेल में थे, लिखना शुरू किया। 5 फरवरी 1924 को रिहा होने के समय तक उन्होंने प्रथम 30 अध्याय लिख डाले थे। यह इतिहास लेखमाला के रूप में 13 अप्रैल 1924 से 22 नवंबर 1925 तक 'नवजीवन' में प्रकाशित हुआ। पुस्तक के रूप में इसके दो खंड क्रमशः 1924 और 1925 में छपे। वालजी देसाई द्वारा किये गये अंग्रेजी अनुवाद का प्रथम संस्करण अपेक्षित संशोधनों के साथ एस० गणेशन मद्रास ने 1928 में और द्वितीय और तृतीय संस्करण नवजीवन प्रकाशन मन्दिर, अहमदाबाद ने 1950 और 1961 में प्रकाशित किया था।


==== सत्य के प्रयोग (आत्मकथा) ====
आत्मकथा के मूल गुजराती अध्याय धारावाहिक रूप से 'नवजीवन' के अंकों में प्रकाशित हुए थे। 29 नवंबर 1925 के अंक में 'प्रस्तावना' के प्रकाशन से उसका आरम्भ हुआ और 3 फरवरी 1929 के अंक में 'पूर्णाहुति' शीर्षक अंतिम अध्याय से उसकी समाप्ति। गुजराती अध्यायों के प्रकाशन के साथ ही हिन्दी नवजीवन में उनका हिन्दी अनुवाद और यंग इंडिया में उनका अंग्रेजी अनुवाद भी दिया जाता रहा। तदनुसार 'प्रस्तावना' का अनुवाद 'हिन्दी नवजीवन' के 3 दिसंबर 1925 के अंक में प्रकाशित हुआ था। हिन्दी अनुवाद में आत्मकथा का पहला खंड पुस्तक के रूप में पहली बार सस्ता साहित्य मंडल, दिल्ली से सन् 1928 में प्रकाशित हुआ था। गाँधी जी की रचनाओं के स्वत्वाधिकारी नवजीवन ट्रस्ट ने अपनी ओर से उसके हिन्दी अनुवाद का प्रकाशन सन् 1957 में किया था।


==== गीता माता ====
श्रीमद्भगवद्गीता से गाँधी जी का हार्दिक लगाव प्रायः आजीवन रहा। गीता पर उनका चिंतन-मनन तथा लेखन भी लंबे समय तक चलते रहा। सम्पूर्ण गीता का गुजराती अनुवाद, प्रस्तावना सहित, उन्होंने जून 1929 में पूरा किया था और 12 मार्च 1930 को नवजीवन प्रकाशन मन्दिर, अहमदाबाद से 'अनासक्ति योग' नाम से उसका पुस्तकाकार प्रकाशन हुआ था। उसका हिंदी, बांग्ला एवं मराठी में अनुवाद भी तत्काल हो गया था। अंग्रेजी अनुवाद इसके बाद जनवरी 1931 में संपन्न हुआ था तथा पहले यंग इंडिया के अंक में प्रकाशित हुआ था।गीता के प्रत्येक श्लोक का अनुवाद सामान्य पाठकों के लिए सहज बोधगम्य न होने से गाँधी जी ने गीता के प्रत्येक अध्याय के भावों को सामान्य पाठकों के लिए सहज बोधगम्य रूप में लिखा। यरवदा सेंट्रल जेल में 1930 और 1932 में प्रत्येक सप्ताह पत्र के रूप में ये भाव भी नारायणदास गांधी को भेजे जाते रहे ताकि उन्हें आश्रम की प्रार्थना सभाओं में पढ़े जायें। इन्हीं का प्रकाशन बाद में पुस्तक रूप में 'गीता-बोध' नाम से हुआ। इनके अतिरिक्त भी उन्होंने गीता पर प्रार्थना सभाओं में अनेक प्रवचन दिये थे। गीता से गाँधी जी का जुड़ाव इस कदर था कि अपने अत्यंत व्यस्त जीवन के बावजूद उन्होंने गीता के प्रत्येक पद का अक्षर क्रम से कोश तैयार किया जिसमें पद के अर्थ के साथ-साथ उनके प्रयोग-स्थल भी निर्दिष्ट थे। इन समस्त सामग्रियों का एकत्र प्रकाशन ही 'गीता माता' के नाम से हुआ है।


==== सम्पूर्ण गांधी वाङ्मय ====
गाँधी जी के लिखित एवं वाचिक समग्र साहित्य के प्रकाशन हेतु भारत सरकार द्वारा एक ग्रंथमाला के प्रकाशन का निर्णय प्रकाशन जगत में निःसंदेह एक ऐतिहासिक कदम रहा है। इस ग्रंथमाला का उद्देश्य गाँधी जी ने दिन-प्रति-दिन और वर्ष-प्रति-वर्ष जो कुछ कहा और लिखा उस सबको एकत्र करना था।इसी निर्णय के तहत अनेक अधीत विद्वानों के सहयोग से सम्पूर्ण गांधी वाङ्मय का प्रकाशन हुआ। यह प्रकाशन तीन भाषाओं में हुआ। अंग्रेजी में पहली बार 100 खंडों में (THE COLLECTED WORKS OF MAHATMA GANDHI नाम से) तथा संशोधित रूप (pdf) में 98 खंडों में इसका प्रकाशन हुआ है। हिन्दी में 97 खंडों में तथा गुजराती में 70 खंडों में यह प्रकाशकीय महाकुंभ संपन्न हुआ है। सम्पूर्ण गांधी वाङ्मय (हिन्दी) में दो अतिरिक्त वैशिष्ट्य भी सम्मिलित हैं। एक तो यह कि प्रत्येक खंड के अंत में अक्षरक्रम से शब्दानुक्रमणिका दी गयी है जिससे अध्ययन तथा विभिन्न कार्यवश पुनरावलोकन में अत्यंत सुविधा हो गयी है; तथा दूसरी यह कि प्रत्येक खंड के अंत में गाँधी जी का तारीखवार जीवन-वृत्तान्त दिया गया है, जिससे सरलतापूर्वक एक नज़र में गाँधीजी के जीवन की सभी महत्वपूर्ण घटनाओं एवं बातों की संक्षिप्त जानकारी उपलब्ध हो जाती है।


==== अन्य ====
इनके अतिरिक्त गाँधी जी के समग्र साहित्य से चुनिंदा अंशों के संचयन तथा विभिन्न विषयों पर केन्द्रित छोटी-छोटी पुस्तिकाओं का भी विभिन्न नामों से प्रकाशन होते रहा है। इनमें दो संचयन अति प्रसिद्ध तथा अत्युपयोगी रहे हैं और इन दोनों का प्रकाशन भी गाँधी जी के जीवन-काल में ही (अंग्रेजी में, 1945 एवं 1947 में) हो गया था:

महात्मा गांधी के विचार - सं०- आर० के० प्रभु एवं यू० आर० राव (नेशनल बुक ट्रस्ट, नयी दिल्ली)
मेरे सपनों का भारत - सं०- आर० के० प्रभु (नवजीवन प्रकाशन मन्दिर, अहमदाबाद)गाँधी जी ने जॉन रस्किन की अन्टू दिस लास्ट (Unto This Last) की गुजराती में व्याख्या भी की है। अन्तिम निबंध को उनका अर्थशास्त्र से सम्बंधित कार्यक्रम कहा जा सकता है उन्होंने शाकाहार, भोजन और स्वास्थ्य, धर्म, सामाजिक सुधार पर भी विस्तार से लिखा है।
सन् २००० में गाँधी जी के संपूर्ण कार्य (CWMG) का संशोधित संस्करण विवादों के घेरे में आ गया क्योंकि गाँधी जी के अनुयायियों ने सरकार पर राजनीतिक उद्देश्यों के लिए परिवर्तन शामिल करने का आरोप लगाया.


=== गाँधी पर पुस्तकें ===
कई जीवनी लेखकों ने गाँधी के जीवन-वर्णन का कार्य किया है। उनमें से दो कार्य व्यापक एवं अपने आप में उदाहरण हैं:

MAHATMA (In 8 Volumes)  By: D. G. Tendulkar (First Edition : January 1954) [The Publications Division, govt. of India, New Delhi]
MAHATMA GANDHI -(In 10 Volumes) {The Early Phase to Last Phase} - By Pyarelal & Sushila Nayar [Navajivan Publishing House, Ahmedabad]इस दूसरे महाग्रंथ के अंतिम खण्ड (Last Phase) का चार खण्डों में हिन्दी अनुवाद महात्मा गांधी : पूर्णाहुति नाम से प्रकाशित है।
कर्नल जी बी अमेरिकी सेना के सिंह ने कहा कि अपनी तथ्यात्मक शोध पुस्तक 'देवत्व के मुखौटे के पीछे गाँधी' (गाँधी: बिहाइंड द मास्क ऑफ़ डिविनिटी) के मूल भाषण और लेखन के लिए उन्होंने अपने २० वर्ष लगा दिए।


=== कथित समलैंगिक प्रेम संबंध ===
२०११ में प्रकाशित किताब ग्रेट सोल: महात्मा गांधी एंड हिज़ स्ट्रगल विद इंडिया में पुलित्ज़र पुरस्कार विजेता लेखक जोसेफ़ लेलीवेल्ड ने महात्मा गाँधी और उनके दक्षिण अफ़्रीका में रहे सहयोगी हर्मन केलेनबाख के रिश्तों को अनन्य प्रेम सम्बन्ध बताया है। पुस्तक के प्रकाशन के समय इस कारण से भारत में विवाद हो गया, और गाँधी के गृह राज्य गुजरात की विधान सभा ने एक संकल्प के माध्यम से इस पुस्तक की बिक्री प्र रोक लगा दी। लेलीवेल्ड के मुताबिक, उनकी पुस्तक के आधार पर गाँधी की समलैंगी या द्विलिंगी होने के लगाए जा रहे आंकलन गलत हैं। उन्होंने कहा, "यह किताब ये नहीं कहती कि गाँधी समलैंगिक या द्विलिंगी थे।यह [किताब] कहती है कि [गाँधी] ब्रह्मचारी थे, और कैलेनबाख से गहरे [दिल] से जुडे थे। और यह कोई नई जानकारी नहीं है।" (यथाशब्द)।


== गांधी और कालेनबाख ==
महात्मा गांधी और उनके दक्षिण अफ्रीकी मित्र हरमन कालेनबाख से संबंधित दस्तावेजों को 1.28 मिलियन डॉलर (करीब 6.88 करोड़ रुपए) में खरीद कर भारत लाया गया है। 2012 में नीलाम होने से पहले भारत सरकार ने इन्हें सोदबी नीलामी घर से गोपनीय करार में खरीदा था। कालेनबाख दक्षिण अफ्रीका में जिम्नास्ट, बॉडी बिल्डर और आर्किटेक्ट थे। उन्होंने एमके गांधी को कुछ ऐसे भी पत्र भेजे थे जिन्हें कुछ समीक्षक ‘प्रेम पत्र’ कहते हैं। इन दोनों लोगों का रिश्ता काफी विवादित रहा था।


== अनुयायियों और प्रभाव ==
महत्वपूर्ण नेता और राजनीतिक गतिविधियाँ गाँधी से प्रभावित थीं। अमेरिका के नागरिक अधिकार आन्दोलन के नेताओं में मार्टिन लूथर किंग और जेम्स लाव्सन गाँधी के लेखन जो उन्हीं के सिद्धांत अहिंसा को विकसित करती है, से काफी आकर्षित हुए थे। विरोधी-रंगभेद कार्यकर्ता और दक्षिण अफ्रीका के पूर्व राष्ट्रपति नेल्सन मंडेला, गाँधी जी से प्रेरित थे। और दुसरे लोग खान अब्दुल गफ्फेर खान,स्टीव बिको और औंग सू कई (Aung San Suu Kyi) हैं।गाँधी का जीवन तथा उपदेश कई लोगों को प्रेरित करती है जो गाँधी को अपना गुरु मानते है या जो गाँधी के विचारों का प्रसार करने में अपना जीवन समर्पित कर देते हैं। यूरोप के, रोमेन रोल्लांड पहला व्यक्ति था जिसने १९२४ में अपने किताब महात्मा गाँधी में गाँधी जी पर चर्चा की थी और ब्राज़ील की अराजकतावादी (anarchist) और नारीवादी मारिया लासर्दा दे मौरा ने अपने कार्य शांतिवाद में गाँधी के बारें में लिखा.१९३१ में उल्लेखनीय भौतिक विज्ञानी अलबर्ट आइंस्टाइन, गाँधी के साथ पत्राचार करते थे और अपने बाद के पत्रों में उन्हें "आने वाले पीढियों का आदर्श" कहा.लांजा देल वस्तो (Lanza del Vasto) महात्मा गाँधी के साथ रहने के इरादे से सन १९३६ में भारत आया; और बाद में गाँधी दर्शन को फैलाने के लिए वह यूरोप वापस आया और १९४८ में उसने कम्युनिटी ऑफ़ द आर्क की स्थापना की। (गाँधी के आश्रम से प्रभावित होकर)मदेलिने स्लेड (मीराबेन) ब्रिटिश नौसेनापति की बेटी थी जिसने अपना अधिक से अधिक व्यस्क जीवन गाँधी के भक्त के रूप में भारत में बिताया था।
इसके अतिरिक्त, ब्रिटिश संगीतकार जॉन लेनन ने गाँधी का हवाला दिया जब वे अहिंसा पर अपने विचारों को व्यक्त कर रहे थे। २००७ में केन्स लिओंस अन्तर राष्ट्रीय विज्ञापन महोत्सव (Cannes Lions International Advertising Festival), अमेरिका के पूर्व उपराष्ट्रपति और पर्यावरणविद अल गोर ने उन पर गाँधी के प्रभाव को बताया। 


== विरासत ==
२ अक्टूबर गाँधी का जन्मदिन है इसलिए गाँधी जयंती के अवसर पर भारत में राष्ट्रीय अवकाश होता है १५ जून २००७ को यह घोषणा की गई थी कि "संयुक्त राष्ट्र महासभा " एक प्रस्ताव की घोषणा की, कि २ अक्टूबर (2 October) को "अंतर्राष्ट्रीय अंहिसा दिवस" के रूप में मनाया जाएगा.अक्सर पश्चिम में महात्मा शब्द का अर्थ ग़लत रूप में ले लिया जाता है उनके अनुसार यह संस्कृत से लिया गया है जिसमे महा का अर्थ महान और आत्म का अर्थ आत्मा होता है। ज्यादातर सूत्रों के अनुसार जैसे दत्ता और रोबिनसन के रबिन्द्रनाथ टगोर: संकलन में कहा गया है कि रवीन्द्रनाथ टैगोर ने सबसे पहले गाँधी को महात्मा का खिताब दिया था। अन्य सूत्रों के अनुसार नौतामलाल भगवानजी मेहता ने २१ जनवरी १९१५ में उन्हें यह खिताब दिया था। हालाँकि गाँधी ने अपनी आत्मकथा में कहा है कि उन्हें कभी नहीं लगा कि वे इस सम्मान के योग्य हैं। मानपत्र के अनुसार, गाँधी को उनके न्याय और सत्य के सराहनीये बलिदान के लिए महात्मा नाम मिला है।१९३० में टाइम पत्रिका ने महात्मा गाँधी को वर्ष का पुरूष का नाम दियाI १९९९ में गाँधी अलबर्ट आइंस्टाइन जिन्हे सदी का पुरूष नाम दिया गया के मुकाबले द्वितीय स्थान जगह पर थे। टाइम पत्रिका ने दलाई लामा, लेच वालेसा, डॉ मार्टिन लूथर किंग, जूनियर, सेसर शावेज़, औंग सान सू कई, बेनिग्नो अकुइनो जूनियर, डेसमंड टूटू और नेल्सन मंडेला को गाँधी के अहिंसा के आद्यात्मिक उत्तराधिकारी के रूप में कहा गया. भारत सरकार प्रति वर्ष उल्लेखनीय सामाजिक कार्यकर्ताओं, विश्व के नेताओं और नागरिकों को महात्मा गाँधी शान्ति पुरस्कार से पुरस्कृत करती है। दक्षिण अफ्रीकी नेल्सन मंडेला, जो कि जातीय मतभेद और पार्थक्य के उन्मूलन में संघर्षरत रहे, इस पुरूस्कार को पाने वाले पहले गैर-भारतीय थे।
१९९६ में, भारत सरकार ने महात्मा गाँधी की श्रृंखला के नोटों के मुद्रण को १, ५, १०, २०, ५०, १००, ५०० और १००० के अंकन के रूप में आरम्भ किया। आज जितने भी नोट इस्तेमाल में हैं उनपर महात्मा गाँधी का चित्र है। १९६९ में यूनाइटेड किंगडम ने डाक टिकेट की एक श्रृंखला महात्मा गाँधी के शत्वर्शिक जयंती के उपलक्ष्य में जारी की। 

यूनाइटेड किंगडम में ऐसे अनेक गाँधी जी की प्रतिमाएँ उन ख़ास स्थानों पर हैं जैसे लन्दन विश्वविद्यालय कालेज के पास ताविस्तोक चौक, लन्दन जहाँ पर उन्होंने कानून की शिक्षा प्राप्त की। यूनाइटेड किंगडम में जनवरी ३० को “राष्ट्रीय गाँधी स्मृति दिवस” मनाया जाता है।संयुक्त राज्य में, गाँधी की प्रतिमाएँ न्यू यार्क शहर में यूनियन स्क्वायर के बहार और अटलांटा में मार्टिन लूथर किंग जूनियर राष्ट्रीय ऐतिहासिक स्थल और वाशिंगटन डी.सी में भारतीय दूतावास के समीप मेसासुशैट्स मार्ग में हैं। भारतीय दूतावास के समीप पितर्मरित्ज़्बर्ग, दक्षिण अफ्रीका, जहाँ पर १८९३ में गाँधी को प्रथम-श्रेणी से निकल दिया गया था वहां उनकी स्मृति में एक प्रतिमा स्थापित की गए है। गाँधी की प्रतिमाएँ मदाम टुसौड के मोम संग्रहालय, लन्दन में, न्यू यार्क और विश्व के अनेक शहरों में स्थापित हैं।
गाँधी को कभी भी शान्ति का नोबेल पुरस्कार प्राप्त नहीं हुआ, हालाँकि उनको १९३७ से १९४८ के बीच, पाँच बार मनोनीत किया गया जिसमे अमेरिकन फ्रेंड्स सर्विस कमिटी द्वारा दिया गया नामांकन भी शामिल है . दशको उपरांत नोबेल समिति ने सार्वजानिक रूप में यह घोषित किया कि उन्हें अपनी इस भूल पर खेद है और यह स्वीकार किया कि पुरूस्कार न देने की वजह विभाजित राष्ट्रीय विचार थे। महात्मा गाँधी को यह पुरुस्कार १९४८ में दिया जाना था, परन्तु उनकी हत्या के कारण इसे रोक देना पड़ा.उस साल दो नए राष्ट्र भारत और पाकिस्तान में युद्ध छिड़ जाना भी एक जटिल कारण था। गाँधी के मृत्यु वर्ष १९४८ में पुरस्कार इस वजह से नहीं दिया गया कि कोई जीवित योग्य उम्मीदवार नहीं था और जब १९८९ में दलाई लामा को पुरस्कृत किया गया तो समिति के अध्यक्ष ने यह कहा कि "यह महात्मा गाँधी की याद में श्रद्धांजलि का ही हिस्सा है।"
बिरला भवन (या बिरला हॉउस), नई दिल्ली जहाँ पर ३०जन्वरी, १९४८ को गाँधी की हत्या की गयी का अधिग्रहण भारत सरकार ने १९७१ में कर लिया तथा १९७३ में गाँधी स्मृति के रूप में जनता के लिए खोल दिया। यह उस कमरे को संजोय हुए है जहाँ गाँधी ने अपने आख़िर के चार महीने बिताये और वह मैदान भी जहाँ रात के टहलने के लिए जाते वक्त उनकी हत्या कर दी गयी। एक शहीद स्तम्भ अब उस जगह को चिन्हित करता हैं जहाँ पर उनकी हत्या कर दी गयी थी।
प्रति वर्ष ३० जनवरी को, महात्मा गाँधी के पुण्यतिथि पर कई देशों के स्कूलों में अहिंसा और शान्ति का स्कूली दिन (DENIP मनाया जाता है जिसकी स्थापना १९६४ स्पेन में हुयी थी। वे देश जिनमें दक्षिणी गोलार्ध कैलेंडर इस्तेमाल किया जाता हैं, वहां ३० मार्च को इसे मनाया जाता है।


== आदर्श और आलोचनाएँ ==
गाँधी के कठोर अहिंसा का नतीजा शांतिवाद (pacifism) है, जो की राजनैतिक क्षेत्र से आलोचना का एक मूल आधार है।


=== विभाजन की संकल्पना ===
नियम के रूप में गाँधी विभाजन की अवधारणा के खिलाफ थे क्योंकि यह उनके धार्मिक एकता के दृष्टिकोण के प्रतिकूल थी। ६ अक्टूबर १९४६ में हरिजन में उन्होंने भारत का विभाजन पाकिस्तान बनाने के लिए, के बारे में लिखा:

(पाकिस्तान की मांग) जैसा की मुस्लीम लीग द्वारा प्रस्तुत किया गया गैर-इस्लामी है और मैं इसे पापयुक्त कहने से नही हिचकूंगा इस्लाम मानव जाति के भाईचारे और एकता के लिए खड़ा है, न कि मानव परिवार के ऐक्य का अवरोध करने के लिए॰इस वजह से जो यह चाहते हैं कि भारत दो युद्ध के समूहों में बदल जाए वे भारत और इस्लाम दोनों के दुश्मन हैं। वे मुझे टुकडों में काट सकते हैं पर मुझे उस चीज़ के लिए राज़ी नहीं कर सकते जिसे मैं ग़लत समझता हूँ[...] हमें आस नही छोडनी चाहिए, इसके बावजूद कि ख्याली बाते हो रही हैं कि हमें मुसलमानों को अपने प्रेम के कैद में अबलाम्बित कर लेना चाहिए॰
फिर भी, जैक होमर गाँधी के जिन्ना के साथ पाकिस्तान के विषय को लेकर एक लंबे पत्राचार पर ध्यान देते हुए कहते हैं- "हालाँकि गांधी वैयक्तिक रूप में विभाजन के खिलाफ थे, उन्होंने सहमति का सुझाव दिया जिसके तहत कांग्रेस और मुस्लिम लीग अस्थायी सरकार के नीचे समझौता करते हुए अपनी आजादी प्राप्त करें जिसके बाद विभाजन के प्रश्न का फैसला उन जिलों के जनमत द्वारा होगा जहाँ पर मुसलमानों की संख्या ज्यादा है।".
भारत के विभाजन के विषय को लेकर यह दोहरी स्थिति रखना, गाँधी ने इससे हिन्दुओं और मुसलमानों दोनों तरफ़ से आलोचना के आयाम खोल दिए। मुहम्मद अली जिन्ना तथा समकालीन पाकिस्तानियों ने गाँधी को मुस्लमान राजनैतिक हक़ को कम कर आंकने के लिए निंदा की। विनायक दामोदर सावरकार और उनके सहयोगियों ने गाँधी की निंदा की और आरोप लगाया कि वे राजनैतिक रूप से मुसलमानों को मनाने में लगे हुए हैं तथा हिन्दुओं पर हो रहे अत्याचार के प्रति वे लापरवाह हैं और पाकिस्तान के निर्माण के लिए स्वीकृति दे दी है (हालाँकि सार्वजानिक रूप से उन्होंने यह घोषित किया था कि विभाजन से पहले मेरे शरीर को दो हिस्सों में काट दिया जाएगा). यह आज भी राजनैतिक रूप से विवादस्पद है, जैसे कि पाकिस्तानी-अमरीकी इतिहासकार आयेशा जलाल यह तर्क देती हैं कि विभाजन की वजह गाँधी और कांग्रेस मुस्लीम लीग के साथ सत्ता बांटने में इक्छुक नहीं थे, दुसरे मसलन हिंदू राष्ट्रवादी राजनेता प्रवीण तोगडिया भी गाँधी के इस विषय को लेकर नेतृत्व की आलोचना करते हैं, यह भी इंगित करते हैं कि उनके हिस्से की अत्यधिक कमजोरी की वजह से भारत का विभाजन हुआ।
गाँधी ने १९३० के अंत-अंत में विभाजन को लेकर इस्राइल के निर्माण के लिए फिलिस्तीन के विभाजन के प्रति भी अपनी अरुचि जाहिर की थी। २६ अक्टूबर १९३८ को उन्होंने हरिजन में कहा था:

मुझे कई पत्र प्राप्त हुए जिनमे मुझसे पूछा गया कि मैं घोषित करुँ कि जर्मनी में यहूदियों के उत्पीडन और अरब-यहूदियों के बारे में क्या विचार रखता हूँ (persecution of the Jews in Germany). ऐसा नही कि इस कठिन प्रश्न पर अपने विचार मैं बिना झिझक के दे पाउँगा. मेरी सहानुभूति यहुदिओं के साथ है। मैं उनसे दक्षिण अफ्रीका से ही नजदीकी रूप से परिचित हूँ कुछ तो जीवन भर के लिए मेरे साथी बन गए हैं। इन मित्रों के द्वारा ही मुझे लंबे समय से हो रहे उत्पीडन के बारे में जानकारी मिली. वे ईसाई धर्म के अछूत रहे हैं पर मेरी सहानुभूति मुझे न्याय की आवश्यकता से विवेकशून्य नही करती यहूदियों के लिए एक राष्ट्र की दुहाई मुझे ज्यादा आकर्षित नही करती. जिसकी मंजूरी बाईबल में दी गयी और जिस जिद से वे अपनी वापसी में फिलिस्तीन को चाहने लगे हैं। क्यों नही वे, पृथ्वी के दुसरे लोगों से प्रेम करते हैं, उस देश को अपना घर बनाते जहाँ पर उनका जन्म हुआ और जहाँ पर उन्होंने जीविकोपार्जन किया। फिलिस्तीन अरबों का हैं, ठीक उसी तरह जिस तरह इनलैंड अंग्रेजों का और फ्रांस फ्रंसिसिओं का. यहूदियों को अरबों पर अधिरोपित करना अनुचित और अमानवीय है जो कुछ भी आज फिलिस्तीन में हो रहा हैं उसे किसी भी आचार संहिता से सही साबित नही किया जा सकता.


=== हिंसक प्रतिरोध की अस्वीकृति ===
जो लोग हिंसा के जरिये आजादी हासिल करना चाहते थे गाँधी उनकी आलोचना के कारण भी थोड़ा सा राजनैतिक आग की लपेट में भी आ गये भगत सिंह, सुखदेव, उदम सिंह, राजगुरु की फांसी के ख़िलाफ़ उनका इनकार कुछ दलों में उनकी निंदा का कारण बनी। इस आलोचना के लिए गाँधी ने कहा,"एक ऐसा समय था जब लोग मुझे सुना करते थे की किस तरह अंग्रेजो से बिना हथियार लड़ा जा सकता है क्योंकि तब हथयार नही थे।..पर आज मुझे कहा जाता है कि मेरी अहिंसा किसी काम की नही क्योंकि इससे हिंदू-मुसलमानों के दंगो को नही रोका जा सकता इसलिए आत्मरक्षा के लिए सशस्त्र हो जाना चाहिए."उन्होंने अपनी बहस कई लेखो में की, जो की होमर जैक्स के द गाँधी रीडर: एक स्रोत उनके लेखनी और जीवन का . १९३८ में जब पहली बार "यहूदीवाद और सेमेटीसम विरोधी" लिखी गई, गाँधी ने १९३० में हुए जर्मनी में यहूदियों पर हुए उत्पीडन (persecution of the Jews in Germany) को सत्याग्रह के अंतर्गत बताया उन्होंने जर्मनी में यहूदियों द्वारा सहे गए कठिनाइयों के लिए अहिंसा के तरीके को इस्तेमाल करने की पेशकश यह कहते हुए की

अगर मैं एक यहूदी होता और जर्मनी में जन्मा होता और अपना जीविकोपार्जन वहीं से कर रहा होता तो जर्मनी को अपना घर मानता इसके वावजूद कि कोई सभ्य जर्मन मुझे धमकाता कि वह मुझे गोली मार देगा या किसी अंधकूपकारागार में फ़ेंक देगा, मैं तडीपार और मतभेदीये आचरण के अधीन होने से इंकार कर दूँगा . और इसके लिए मैं यहूदी भाइयों का इंतज़ार नाहे करूंगा कि वे आयें और मेरे वैधानिक प्रैत्रोध में मुझसे जुडें, बल्कि मुझे आत्मविश्वास होगा कि आख़िर में सभी मेरा उदहारण मानने के लिए बाध्य हो जायेंगे. यहाँ पर जो नुस्खा दिया गया है अगर वह एक भी यहूदी या सारे यहूदी स्वीकार कर लें, तो उनकी स्थिति जो आज है उससे बदतर नही होगी. और अगर दिए गए पीडा को वे स्वेच्छापूर्वक सह लें तो वह उन्हें अंदरूनी शक्ति और आनन्द प्रदान करेगा और हिटलर की सुविचारित हिंसा भी यहूदियों की एक साधारण नर संहार के रूप में निष्कर्षित हो तथा यह उसके अत्याचारों की घोषणा के खिलाफ पहला जवाब होगी. अगर यहूदियों का दिमाग स्वेच्छयापूर्वक पीड़ा सहने के लिए तयार हो, मेरी कल्पना है कि संहार का दिन भी धन्यवाद ज्ञापन और आनन्द के दिन में बदल जाएगा जैसा कि जिहोवा ने गढा.. एक अत्याचारी के हाथ में अपनी ज़ाति को देकर किया। इश्वर का भय रखने वाले, मृत्यु के आतंक से नही डरते.
गाँधी की इन वक्तव्यों के कारण काफ़ी आलोचना हुयी जिनका जवाब उन्होंने "यहूदियों पर प्रश्न" लेख में दिया साथ में उनके मित्रों ने यहूदियों को किए गए मेरे अपील की आलोचना में समाचार पत्र कि दो कर्तने भेजीं दो आलोचनाएँ यह संकेत करती हैं कि मैंने जो यहूदियों के खिलाफ हुए अन्याय का उपाय बताया, वह बिल्कुल नया नहीं है।...मेरा केवल यह निवेदन हैं कि अगर हृदय से हिंसा को त्याग दे तो निष्कर्षतः वह अभ्यास से एक शक्ति सृजित करेगा जो कि बड़े त्याग कि वजह से है। उन्होंने आलोचनाओं का उत्तर "यहूदी मित्रो को जवाब" और "यहूदी और फिलिस्तीन" में दिया यह जाहिर करते हुए कि "मैंने हृदय से हिंसा के त्याग के लिए कहा जिससे निष्कर्षतः अभ्यास से एक शक्ति सृजित करेगा जो कि बड़े त्याग कि वजह से है।यहूदियों की आसन्न आहुति को लेकर गाँधी के बयान ने कई टीकाकारों की आलोचना को आकर्षित किया।मार्टिन बूबर (Martin Buber), जो की स्वयं यहूदी राज्य के एक विरोधी हैं ने गाँधी को २४ फरवरी, १९३९ को एक तीक्ष्ण आलोचनात्मक पत्र लिखा. बूबर ने दृढ़ता के साथ कहा कि अंग्रेजों द्वारा भारतीय लोगों के साथ जो व्यवहार किया गया वह नाजियों द्वारा यहूदियों के साथ किए गए व्यवहार से भिन्न है, इसके अलावा जब भारतीय उत्पीडन के शिकार थे, गाँधी ने कुछ अवसरों पर बल के प्रयोग का समर्थन किया।गाँधी ने १९३० में जर्मनी में यहूदियों (persecution of the Jews in Germany) के उत्पीडन को सत्याग्रह के भीतर ही सन्दर्भित कहा। नवम्बर १९३८ में उपरावित यहूदियों के नाजी उत्पीडन के लिए उन्होंने अहिंसा के उपाय को सुझाया:

आभास होता है कि यहूदियों के जर्मन उत्पीडन का इतिहास में कोई सामानांतर नही. पुराने जमाने के तानाशाह कभी इतने पागल नही हुए जितना कि हिटलर हुआ और इसे वे धार्मिक उत्साह के साथ करते हैं कि वह एक ऐसे अनन्य धर्म और जंगी राष्ट्र को प्रस्तुत कर रहा है जिसके नाम पर कोई भी अमानवीयता मानवीयता का नियम बन जाती है जिसे अभी और भविष्य में पुरुस्कृत किया जायेगा. जाहिर सी बात है कि एक पागल परन्तु निडर युवा द्वारा किया गया अपराध सारी जाति पर अविश्वसनीय उग्रता के साथ पड़ेगा.यदि कभी कोई न्यायसंगत युद्ध मानवता के नाम पर, तो एक पुरी कॉम के प्रति जर्मनी के ढीठ उत्पीडन के खिलाफ युद्ध को पूर्ण रूप से उचित कहा जा सकता हैं। पर मैं किसी युद्ध में विश्वास नही रखता. इसे युद्ध के नफा-नुकसान के बारे में चर्चा मेरे अधिकार क्षेत्र में नही है। परन्तु जर्मनी द्वारा यहूदियों पर किए गए इस तरह के अपराध के खिलाफ युद्ध नही किया जा सकता तो जर्मनी के साथ गठबंधन भी नही किया जा सकता यह कैसे हो सकता हैं कि ऐसे देशों के बीच गठबंधन हो जिसमे से एक न्याय और प्रजातंत्र का दावा करता हैं और दूसरा जिसे दोनों का दुश्मन घोषित कर दिया गया है?"


=== दक्षिण अफ्रीका के प्रारंभिक लेख ===
गाँधी के दक्षिण अफ्रीका को लेकर शुरुआती लेख काफी विवादस्पद हैं ७ मार्च, १९०८ को, गाँधी ने इंडियन ओपिनियन में दक्षिण अफ्रीका में उनके कारागार जीवन के बारे में लिखा "काफिर शासन में ही असभ्य हैं - कैदी के रूप में तो और भी. वे कष्टदायक, गंदे और लगभग पशुओं की तरह रहते हैं।" १९०३ में अप्रवास के विषय को लेकर गाँधी ने टिप्पणी की कि "मैं मानता हूँ कि जितना वे अपनी जाति की शुद्धता पर विश्वास करते हैं उतना हम भी...हम मानते हैं कि दक्षिण अफ्रीका में जो गोरी जाति है उसे ही श्रेष्ट जाति होनी चाहिए॰" दक्षिण अफ्रीका में अपने समय के दौरान गाँधी ने बार-बार भारतीयों का अश्वेतों के साथ सामाजिक वर्गीकरण को लेकर विरोध किया, जिनके बारे में वे वर्णन करते हैं कि " निसंदेह पूर्ण रूप से काफिरों से श्रेष्ठ हैं". यह ध्यान देने योग्य हैं कि गाँधी के समय में काफिर का वर्तमान में (a different connotation) इस्तेमाल हो रहे अर्थ से एक अलग अर्थ था (its present-day usage). गाँधी के इन कथनों ने उन्हें कुछ लोगों द्वारा नसलवादी होने के आरोप को लगाने का मौका दिया है।इतिहास के दो प्रोफ़ेसर सुरेन्द्र भाना और गुलाम वाहेद, जो दक्षिण अफ्रीका के इतिहास पर महारत रखते हैं, ने अपने मूलग्रन्थ द मेकिंग ऑफ़ अ पोलिटिकल रिफोर्मार : गाँधी इन साऊथ अफ्रीका, १८९३ - १९१४ में इस विवाद की जांच की है।(नई दिल्ली:मनोहर,२००५). अध्याय एक के केन्द्र में,"गाँधी, औपनिवेशिक स्थिति में जन्मे अफ्रीकी और भारतीय" जो कि "श्वेत आधिपत्य" में अफ्रीकी और भारतीय समुदायों के संबंधों पर है तथा उन नीतियों पर जिनकी वजह से विभाजन हुआ (और वे तर्क देते हैं कि इन समुदायों के बीच संघर्ष लाजिमी सा है) इस सम्बन्ध के बारे में वे कहते हैं, "युवा गाँधी १८९० में उन विभाजीय विचारों से प्रभावित थे जो कि उस समय प्रबल थीं।" साथ ही साथ वे यह भी कहते हैं, "गाँधी के जेल के अनुभव ने उन्हें उन लोगों कि स्थिति के प्रति अधीक संवेदनशील बना दिया था।..आगे गाँधी दृढ़ हो गए थे; वे अफ्रीकियों के प्रति अपने अभिव्यक्ति में पूर्वाग्रह को लेकर बहुत कम निर्णायक हो गए और वृहत स्तर पर समान कारणों के बिन्दुओं को देखने लगे थे। जोहान्सबर्ग जेल में उनके नकारात्मक दृष्टिकोण में ढीठ अफ्रीकी कैदी थे न कि आम अफ्रीकी."दक्षिण अफ्रीका के पूर्व राष्ट्रपति नेल्सन मंडेला गाँधी के अनुयायी हैं, २००३ में गाँधी के आलोचकों द्वारा प्रतिमा के अनावरण को रोकने की कोशिश के बावजूद उन्होंने उसे जोहान्सबर्ग में अनावृत किया। भाना और वाहेद ने अनावरण के इर्द-गिर्द होने वाली घटनाओं पर द मेकिंग ऑफ़ अ पोलिटिकल रिफोर्मार : गाँधी इन साऊथ अफ्रीका, १९१३-१९१४ में टिप्पणी किया है। अनुभाग " दक्षिण अफ्रीका के लिए गाँधी के विरासत" में वे लिखते हैं " गाँधी ने दक्षिण अफ्रीका के सक्रिय कार्यकर्ताओं के आने वाली पीढियों को श्वेत अधिपत्य को ख़त्म करने के लिये प्रेरित किया। यह विरासत उन्हें नेल्सन मंडेला से जोड़ती हैं।.माने यह कि जिस कम को गाँधी ने शुरू किया था उसे मंडेला ने खत्म किया।" वे जारी रखते हैं उन विवादों का हवाला देते हुए जो गाँधी कि प्रतिमा के अनावरण के दौरान उठे थे। गाँधी के प्रति इन दो दृष्टिकोणों के प्रतिक्रिया स्वरुप, भाना और वाहेद तर्क देते हैं : वे लोग को दक्षिण अफ्रीका में रंगभेद के पश्चात अपने राजनैतिक उद्देश्य के लिए गाँधी को सही ठहराना चाहते हैं वे उनके बारे में कई तथ्यों को नजरंदाज करते हुए कारण  में कुछ ज्यादा मदद नहीं करते; और जो उन्हें केवल एक नस्लवादी कहते हैं वे भी ग़लत बयानी के उतने ही दोषी हैं/विकृति के उतने ही दोषी हैं।"


=== राज्य विरोधी ===
गाँधी राज विरोधी (anti statist) उस रूप में थे जहाँ उनका दृष्टिकोण उस भारत का हैं जो कि किसी सरकार के अधीन न हो।  उनका विचार था कि एक देश में सच्चे स्वशासन (self rule) का अर्थ है कि प्रत्यक व्यक्ति ख़ुद पर शासन करता हैं तथा कोई ऐसा राज्य नहीं जो लोगों पर कानून लागु कर सके।  कुछ मौकों पर उन्होंने स्वयं को एक दार्शनिक अराजकतावादी कहा है (philosophical anarchist). उनके अर्थ में एक स्वतंत्र भारत का अस्तित्व उन हजारों छोटे छोटे आत्मनिर्भर समुदायों से है (संभवतः टालस्टोय का विचार) जो बिना दूसरो के अड़चन बने ख़ुद पर राज्य करते हैं। इसका यह मतलब नहीं था कि ब्रिटिशों द्वारा स्थापित प्रशाशनिक ढांचे को भारतीयों को स्थानांतरित कर देना जिसके लिए उन्होंने कहा कि हिंदुस्तान को इंगलिस्तान बनाना है. ब्रिटिश ढंग के संसदीय तंत्र पर कोई विश्वास न होने के कारण वे भारत में आजादी के बाद कांग्रेस पार्टी को भंग कर प्रत्यक्ष लोकतंत्र (direct democracy) प्रणाली को स्थापित करना चाहते थे।


== गांधी जी की आलोचना ==
गांधी के सिद्धान्तों और करनी को लेकर प्रयः उनकी आलोचना भी की जाती है। उनकी आलोचना के मुख्य बिन्दु हैं-

जुलु विद्रोह में अंग्रेजों का साथ देना 
दोनो विश्वयुद्धों में अंग्रेजों का साथ देना,
खिलाफत आन्दोलन जैसे साम्प्रदायिक आन्दोलन को राष्ट्रीय आन्दोलन बनाना,
अंग्रेजों के विरुद्ध सशस्त्र क्रान्तिकारियों के हिंसात्मक कार्यों की निन्दा करना,
गांधी-इरविन समझौता - जिससे भारतीय क्रन्तिकारी आन्दोलन को बहुत धक्का लगा,
भारतीय राष्ट्रीय कांग्रेस के अध्यक्ष पद पर सुभाष चन्द्र बोस के चुनाव पर नाखुश होना,
चौरीचौरा काण्ड के बाद असहयोग आन्दोलन को सहसा रोक देना,
भारत की स्वतंत्रता के बाद नेहरू को प्रधानमंत्री का दावेदार बनाना,
स्वतंत्रता के बाद पाकिस्तान को ५५ करोड़ रूपये देने की जिद पर अनशन करना,
भीमराव आम्बेडकर, महात्मा गांधी को जाति प्रथा का समर्थक समझते थे।


== इन्हें भी देखें ==
महात्मा गांधी का शिक्षा दर्शन
महात्मा गांधी की हत्या


=== आगे के अध्ययन के लिए ===
भाना, सुरेन्द्र और गुलाम वाहेद.द मेकिंग ऑफ़ अ पोलिटिकल रिफोर्मार : गाँधी इन साऊथ अफ्रीका, १८९३-१९१४. नई दिल्ली: मनोहर, २००५
बोंदुरंत्त, जुआअन वी. हिंसा की जीत: गाँधीवादी दर्शन का संघर्ष. प्रिन्सटन यूपी, १९९८ आईएसबीऍन ०-६९१-०२२८१-X
चेर्नस, ईरा. अमरीकी अहिंसा: विचारों का इतिहास, सातवाँ अध्याय .आईएसबीऍन १-५७०७५-५४७-७
चड्ढा, योगेश .गाँधी: एक जीवन .आईएसबीऍन ०-४७१-३५०६२-१
डेलटन, डेनिस (ईडी) .महात्मा गाँधी: चुनिन्दा राजनैतिक लेख .इंडियानापोलिस/कैमब्रिज : हैकट प्रकशन कंपनी (Hackett Publishing Company), १९९६ आईएसबीऍन ०-८७२२०-३३०-१
गाँधी, महात्मा .महात्मा गाँधी के संचित लेख .नई दिल्ली: प्रकाशन विभाग, सुचना एवम प्रसारण मंत्रालय, भारत सरकार, १९९४.
इश्वरण, एकनाथ (Eswaran, Eknath).गाँधी एक मनुष्य .आईएसबीऍन ०-९१५१३२-९६-६
फिशर, लुईस .द एसेनसियल गांधी : उनके जीवन, कार्यों और विचारों का संग्रह . प्राचीन: न्यूयार्क, २००२. (पुनर्मुद्रित संस्करण), आईएसबीऍन १-४०००-३०५०-१
गाँधी, एम.के. गाँधी के पाठक: उनके जीवन और लेखन का एक स्रोत पुस्तक.होमर जैक (ईडी) ग्रोव प्रेस, न्यू योर्क, १९५६
गाँधी, राजमोहन .पटेल: एक जीवन .नवजीवन प्रकाशन घर, १९९० आईएसबीऍन ८१-७२२९-१३८-८
हंट, जेम्स डी. लन्दन में गाँधी . नई दिल्ली: प्रोमिला एवं कंपनी, प्रकाशक, १९७८
मान्न, बर्नहार्ड, महात्मा गाँधी और पाउलो फरेरी के शैक्षणिक अवधारणाएं. क्लौबें, बी, में .(ईडी) राजनैतिक समाजीकरण एव शिक्षा में अंतर्राष्ट्रीय अध्ययन बीडी. ८.हैम्बर्ग १९९६.आईएसबीऍन ३-९२६९५२-९७-०
रूहे, पीटर. गाँधी: एक छायाचित्र जीवनी .आईएसबीऍन ०-७१४८-९२७९-३
शार्प, जीन. गाँधी एक राजनैतिक नीतिज्ञ के रूप में, अपने मूल्यों और राजनैतिक निबंधो के साथ. बोस्टन: एक्सटेंडिंग होराइज़ोन पुस्तकें, १९७९.
सोफ्री, गियान्नी. गाँधी और भारत: केन्द्र में एक सदी (१९९५) आईएसबीएन १-९००६२४-१२-५
गौरडन, हैम.आध्यात्मिक साम्राज्यवाद से अस्वीकृति: गाँधी को बूबर के पत्रों की झलकी .'सार्वभौमिक अध्ययन की पत्रिका, २२ जून १९९९.
गाँधी, एम.के "दक्षिण अफ्रीका में सत्याग्रह"


== सन्दर्भ ==


== बाहरी कड़ियाँ ==
महात्मा गांधी विरासत पोर्टल
हिंदी के सबसे बड़े पैरोकार महात्मा गांधी
गांधीजी और हिन्दी
Nine facts you may not know about Mahatma Gandhi
Even Gandhi, apostle of peace, wanted Indians to fight in World War I

'''

In [44]:
# record start time
start = time.time()

# We will generate all possible mcq from single page in json format
prompt = "Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text:"+" "+item+'''\n Give me output in this JSON array format:[{"question": string, "options":List[string], "answer":string}]'''
prompt_template= f'''[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> {prompt}[/INST]'''

# print("\n\n*** Generate:")

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=4096,
    do_sample=True,
)


print(pipe(prompt_template)[0]['generated_text'])
# record end time
end = time.time()

temp = end-start
print(temp)
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes

total_time_for_generation = "%dh:%dm:%ds" %(hours,minutes,seconds)

if hours == 0:
    print('The time of execution of downloading and loading the model : %dm:%ds' %(minutes,seconds))

else:
    print('The time of execution of downloading and loading the model : %dh:%dm:%ds' %(hours,minutes,seconds))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


*** Pipeline:


OutOfMemoryError: CUDA out of memory. Tried to allocate 1022.00 MiB. GPU 0 has a total capacty of 47.54 GiB of which 663.00 MiB is free. Process 3284230 has 46.87 GiB memory in use. Of the allocated memory 43.16 GiB is allocated by PyTorch, and 3.40 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [35]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Mon Apr  1 05:55:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:01:00.0 Off |                  Off |
| 30%   45C    P2              76W / 300W |  48251MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [45]:
import gc
import torch

def flush():
  gc.collect()
  torch.cuda.empty_cache()
  torch.cuda.reset_peak_memory_stats()

In [46]:
flush()

In [38]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Mon Apr  1 05:56:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:01:00.0 Off |                  Off |
| 30%   37C    P8              21W / 300W |  46783MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [43]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Mon Apr  1 05:57:13 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:01:00.0 Off |                  Off |
| 30%   37C    P8              21W / 300W |  38931MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [55]:
flush()

In [56]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Mon Apr  1 05:59:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:01:00.0 Off |                  Off |
| 30%   36C    P8              20W / 300W |  46543MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [63]:
# record start time
start = time.time()

# We will generate all possible mcq from single page in json format
prompt = "Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text:"+" "+item+'''\n Give me output in this JSON array format:[{"question": string, "options":List[string], "answer":string}]'''
prompt_template= f'''[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> {prompt}[/INST]'''

# print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda(4)
# output = model.generate(inputs=input_ids, do_sample=True,max_length=32768)
# output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
# print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@",tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    do_sample=True,
    max_new_tokens=4096
)

# print("######",pipe(prompt_template)[0]['generated_text'])
gen_text = pipe(prompt_template)[0]['generated_text']
print(pipe(prompt_template)[0]['generated_text'])


# record end time
end = time.time()

temp = end-start
print(temp)
hours = temp//3600
temp = temp - 3600*hours
minutes = temp//60
seconds = temp - 60*minutes

total_time_for_generation = "%dh:%dm:%ds" %(hours,minutes,seconds)

if hours == 0:
    print('The time of execution of downloading and loading the model : %dm:%ds' %(minutes,seconds))

else:
    print('The time of execution of downloading and loading the model : %dh:%dm:%ds' %(hours,minutes,seconds))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


*** Pipeline:


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text: 
Mohandas Karamchand Gandhi (2 October 1869 – 30 January 1948) was an Indian lawyer, anti-colonial nationalist and political ethicist who employed nonviolent resistance to lead the successful campaign for India's independence from British rule. He inspired movements for civil rights and freedom across the world. The honorific Mahātmā (from Sanskrit 'great-souled, venerable'), first applied to him in South Africa in 1914, is now used throughout the world.
Born and raised in a Hindu family in coastal Gujarat, Gandhi trained in the law at the Inner Temple, London, and was called to the bar at age 22 in June 1891. After two uncertain years in India, where he was unable to start a successful law practice, he moved to South Africa in 1893 to represe

In [62]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Mon Apr  1 06:02:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:01:00.0 Off |                  Off |
| 30%   44C    P5              55W / 300W |  43113MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [65]:
gen_text

'[INST] <<SYS>> You have Phd in history and best in generating Multiple choice questions with demanded JSON format. <</SYS>> Generate as many as possible difficulty level hard Multiple choice questions with four options and answer using this text: \nMohandas Karamchand Gandhi (2 October 1869 – 30 January 1948) was an Indian lawyer, anti-colonial nationalist and political ethicist who employed nonviolent resistance to lead the successful campaign for India\'s independence from British rule. He inspired movements for civil rights and freedom across the world. The honorific Mahātmā (from Sanskrit \'great-souled, venerable\'), first applied to him in South Africa in 1914, is now used throughout the world.\nBorn and raised in a Hindu family in coastal Gujarat, Gandhi trained in the law at the Inner Temple, London, and was called to the bar at age 22 in June 1891. After two uncertain years in India, where he was unable to start a successful law practice, he moved to South Africa in 1893 to r

In [66]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Mon Apr  1 06:18:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:01:00.0 Off |                  Off |
| 30%   49C    P8              22W / 300W |  45557MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [67]:
type(gen_text)

str